In [ ]:
!pip -q install requests


In [ ]:
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM = {"name":"wheezards","pl":"Python","email":"tarstars@gmail.com"}

resp = requests.post(f"{BASE}/register", json=TEAM, timeout=30)
resp.raise_for_status()
team = resp.json()  # {"id": "..."}
team_id = team["id"]
print("REGISTERED. Your SECRET id:", team_id)

# Save locally in the current runtime (ephemeral unless you copy it to Drive)
pathlib.Path("/content/icfp_id.json").write_text(json.dumps(team))
print("Saved to /content/icfp_id.json")


REGISTERED. Your SECRET id: tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ
Saved to /content/icfp_id.json


In [ ]:
import json, pathlib, requests

In [ ]:
import json, requests, pathlib

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Load id (or paste it directly if you prefer)
team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
team_id = team["id"]

# 1) Select toy problem
r = requests.post(f"{BASE}/select", json={"id": team_id, "problemName": "probatio"})
r.raise_for_status()
print("Selected:", r.json())

# 2) First exploration: empty route, then a couple doors
plans = ["", "0", "1", "2"]i
r = requests.post(f"{BASE}/explore", json={"id": team_id, "plans": plans})
r.raise_for_status()
data = r.json()
print("Results:", data["results"])
print("queryCount:", data["queryCount"])


Selected: {'problemName': 'probatio'}
Results: [[0], [0, 2], [0, 2], [0, 2]]
queryCount: 5


In [ ]:
o

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/icfp_id.json /content/drive/MyDrive/icfp_id.json
print("Backed up to Drive: /content/drive/MyDrive/icfp_id.json")


MessageError: Error: credential propagation was unsuccessful

In [ ]:
# === One-shot observation + map reconstruction for "probatio" ===
import json, pathlib, requests
from itertools import product

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Load your secret id saved earlier (adjust path if needed)
team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
TEAM_ID = team["id"]

# Prefixes we want to probe (candidate state reps)
S = [""] + list("012345") + \
    ["".join(p) for p in product("012", repeat=2)] + \
        ["03","30"]  # a couple of cross checks

        # Suffixes to discriminate behaviours (keep short to stay cheap)
        E = ["", "0", "1", "2", "3", "4", "5"]

        # Build the batched plan set and dedup
        plans = []
        seen = set()
        for s in S:
            for e in E:
                    p = s + e
                            if p not in seen:
                                        seen.add(p)
                                                    plans.append(p)

                                                    # One /explore call to measure everything
                                                    resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=60)
                                                    resp.raise_for_status()
                                                    data = resp.json()
                                                    results = dict(zip(plans, data["results"]))
                                                    print("queryCount after this call:", data["queryCount"])

                                                    # Build rows: Row(s)[e] = label at position |s| in the run of s+e
                                                    def row_of(s):
                                                        return tuple(results[s+e][len(s)] for e in E)

                                                        rows = {s: row_of(s) for s in S}

                                                        # Identify distinct states by unique rows (Moore machine view)
                                                        row_to_idx = {}
                                                        idx_to_rep = []
                                                        for s in sorted(S, key=len):
                                                            r = rows[s]
                                                                if r not in row_to_idx:
                                                                        row_to_idx[r] = len(idx_to_rep)
                                                                                idx_to_rep.append(s)

                                                                                print(f"Observed {len(idx_to_rep)} distinct states (expect 3 for probatio).")
                                                                                for i, rep in enumerate(idx_to_rep):
                                                                                    print(f"State {i}: rep='{rep}', label={rows[rep][0]}  row={rows[rep]}")

                                                                                    # Helper to map any prefix to a state index
                                                                                    def idx_of(prefix):
                                                                                        return row_to_idx[rows[prefix]]

                                                                                        # Build transitions delta[i][a] using representatives
                                                                                        n = len(idx_to_rep)
                                                                                        d


IndentationError: unexpected indent (ipython-input-2475718913.py, line 17)

In [ ]:
# === One-shot observation + map reconstruction for "probatio" ===
import json, pathlib, requests
from itertools import product

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Load your secret id saved earlier (adjust path if needed)
team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
TEAM_ID = team["id"]

# Prefixes we want to probe (candidate state reps)
S = [""] + list("012345") + \
    ["".join(p) for p in product("012", repeat=2)] + \
    ["03","30"]  # a couple of cross checks

# Suffixes to discriminate behaviours (keep short to stay cheap)
E = ["", "0", "1", "2", "3", "4", "5"]

# Build the batched plan set and dedup
plans = []
seen = set()
for s in S:
    for e in E:
        p = s + e
        if p not in seen:
            seen.add(p)
            plans.append(p)

# One /explore call to measure everything
resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=60)
resp.raise_for_status()
data = resp.json()
results = dict(zip(plans, data["results"]))
print("queryCount after this call:", data["queryCount"])

# Build rows: Row(s)[e] = label at position |s| in the run of s+e
def row_of(s):
    return tuple(results[s+e][len(s)] for e in E)

rows = {s: row_of(s) for s in S}

# Identify distinct states by unique rows (Moore machine view)
row_to_idx = {}
idx_to_rep = []
for s in sorted(S, key=len):
    r = rows[s]
    if r not in row_to_idx:
        row_to_idx[r] = len(idx_to_rep)
        idx_to_rep.append(s)

print(f"Observed {len(idx_to_rep)} distinct states (expect 3 for probatio).")
for i, rep in enumerate(idx_to_rep):
    print(f"State {i}: rep='{rep}', label={rows[rep][0]}  row={rows[rep]}")

# Helper to map any prefix to a state index
def idx_of(prefix):
    return row_to_idx[rows[prefix]]

# Build transitions delta[i][a] using representatives
n = len(idx_to_rep)
delta = [[None]*6 for _ in range(n)]
for i, s in enumerate(idx_to_rep):
    for a in range(6):
        sp = s + str(a)
        if sp not in rows:
            rows[sp] = row_of(sp)
        delta[i][a] = idx_of(sp)

# Room labels come from E[0] == "" column
rooms = [rows[idx_to_rep[i]][0] for i in range(n)]

# Starting state is the one represented by ""
starting_idx = idx_of("")

# Build undirected connections, pairing doors on both ends
connections = []
seen_edges = set()
for i in range(n):
    for a in range(6):
        j = delta[i][a]
        for bb in range(6):
            if delta[j][bb] == i:
                key = tuple(sorted([(i,a),(j,bb)]))
                if key not in seen_edges:
                    seen_edges.add(key)
                    connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":bb}})
                break

map_obj = {"rooms": rooms, "startingRoom": starting_idx, "connections": connections}
print("\n=== Candidate map ===")
print(json.dumps(map_obj, indent=2))


queryCount after this call: 115
Observed 3 distinct states (expect 3 for probatio).
State 0: rep='', label=0  row=(0, 0, 0, 0, 0, 0, 0)
State 1: rep='0', label=2  row=(2, 2, 2, 2, 2, 2, 2)
State 2: rep='3', label=1  row=(1, 1, 1, 1, 1, 1, 1)


KeyError: '040'

In [ ]:
# === Repair & finalize "probatio" using your existing session ===
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Load TEAM_ID if needed
try:
    TEAM_ID
except NameError:
    team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
    TEAM_ID = team["id"]

# Reuse results if you already have it; otherwise start fresh
try:
    results
except NameError:
    results = {}

# We’ll use the same discriminator set
E = ["", "0", "1", "2", "3", "4", "5"]

# Representatives we already know exist from your logs:
# State 0: rep=''  label=0
# State 1: rep='0' label=2
# State 2: rep='3' label=1
reps = ["", "0", "3"]

def fetch(plans):
    plans = [p for p in plans if p not in results]
    if not plans:
        return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=60)
    r.raise_for_status()
    data = r.json()
    results.update(dict(zip(plans, data["results"])))
    print(f"Fetched {len(plans)} plans; queryCount now: {data['queryCount']}")

# Ensure we have measures for reps and all their one-step successors, for all suffixes E
needed = set()
for s in reps:
    for e in E:
        needed.add(s + e)
    for a in "012345":
        sp = s + a
        for e in E:
            needed.add(sp + e)

fetch(sorted(needed))

# Build rows for representatives (vectors over E)
def row_for(s):
    return tuple(results[s + e][len(s)] for e in E)

rows = {s: row_for(s) for s in reps}

# Index states by their row; should be exactly 3 for "probatio"
row_to_idx = {rows[reps[i]]: i for i in range(3)}

# Build transitions by matching the successor row to one of the representative rows
n = 3
delta = [[None]*6 for _ in range(n)]
for i, s in enumerate(reps):
    for a in range(6):
        sp = s + str(a)
        r = row_for(sp)
        j = row_to_idx[r]  # in probatio, successors must match one of our 3 rows
        delta[i][a] = j

# Room labels (take E[0] == "" component)
rooms = [rows[s][0] for s in reps]
starting_idx = 0  # '' is our index 0 in reps

# Build undirected connections (pair the opposite door)
connections = []
seen_edges = set()
for i


In [ ]:
# === Repair & finalize "probatio" using your existing session ===
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Load TEAM_ID if needed
try:
    TEAM_ID
    except NameError:
        team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
            TEAM_ID = team["id"]

            # Reuse results if you already have it; otherwise start fresh
            try:
                results
                except NameError:
                    results = {}

                    # We’ll use the same discriminator set
                    E = ["", "0", "1", "2", "3", "4", "5"]

                    # Representatives we already know exist from your logs:
                    # State 0: rep=''  label=0
                    # State 1: rep='0' label=2
                    # State 2: rep='3' label=1
                    reps = ["", "0", "3"]

                    def fetch(plans):
                        plans = [p for p in plans if p not in results]
                            if not plans:
                                    return
                                        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=60)
                                            r.raise_for_status()
                                                data = r.json()
                                                    results.update(dict(zip(plans, data["results"])))
                                                        print(f"Fetched {len(plans)} plans; queryCount now: {data['queryCount']}")

                                                        # Ensure we have measures for reps and all their one-step successors, for all suffixes E
                                                        needed = set()
                                                        for s in reps:
                                                            for e in E:
                                                                    needed.add(s + e)
                                                                        for a in "012345":
                                                                                sp = s + a
                                                                                        for e in E:
                                                                                                    needed.add(sp + e)

                                                                                                    fetch(sorted(needed))

                                                                                                    # Build rows for representatives (vectors over E)
                                                                                                    def row_for(s):
                                                                                                        return tuple(results[s + e][len(s)] for e in E)

                                                                                                        rows = {s: row_for(s) for s in reps}

                                                                                                        # Index states by their row; should be exactly 3 for "probatio"
                                                                                                        row_to_idx = {rows[reps[i]]: i for i in range(3)}

                                                                                                        # Build transitions by matching the successor row to one of the representative rows
                                                                                                        n = 3
                                                                                                        delta = [[None]*6 for _ in range(n)]
                                                                                                        for i, s in enumerate(reps):
                                                                                                            for a in range(6):
                                                                                                                    sp = s + str(a)
                                                                                                                            r = row_for(sp)
                                                                                                                                    j = row_to_idx[r]  # in probatio, successors must match one of our 3 rows
                                                                                                                                            delta[i][a] = j

                                                                                                                                            # Room labels (take E[0] == "" component)
                                                                                                                                            rooms = [rows[s][0] for s in reps]
                                                                                                                                            starting_idx = 0  # '' is our index 0 in reps

                                                                                                                                            # Build undirected connections (pair the opposite door)
                                                                                                                                            connections = []
                                                                                                                                            seen_edges = set()
                                                                                                                                            for i in range(n):
                                                                                                                                                for a in range(6):
                                                                                                                                                        j = delta[i][a]
                                                                                                                                                                # find partner door at j that returns to i
                                                                                                                                                                        b = None
                                                                                                                                                                                for bb in range(6):
                                                                                                                                                                                            if delta[j][bb] == i:
                                                                                                                                                                                                            b = bb
                                                                                                                                                                                                                            break
                                                                                                                                                                                                                                    key = tuple(sorted([(i, a), (j, b)]))
                                                                                                                                                                                                                                            if key not in seen_edges:
                                                                                                                                                                                                                                                        seen_edges.add(key)
                                                                                                                                                                                                                                                                    connections.append({
                                                                                                                                                                                                                                                                                    "from": {"room": i, "door": a},
                                                                                                                                                                                                                                                                                                    "to":   {"room": j, "door": b}
                                                                                                                                                                                                                                                                                                                })

                                                                                                                                                                                                                                                                                                                map_obj = {"rooms": rooms, "startingRoom": starting_idx, "connections": connections}
                                                                                                                                                                                                                                                                                                                print("\n=== Candidate map ===")
                                                                                                                                                                                                                                                                                                                print(json.dumps(map_obj, indent=2))

                                                                                                                                                                                                                                                                                                                # Optional submission
                                                                                                                                                                                                                                                                                                                SUBMIT = False
                                                                                                                                                                                                                                                                                                                if SUBMIT:
                                                                                                                                                                                                                                                                                                                    r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=30)
                                                                                                                                                                                                                                                                                                                        r.raise_for_status()
                                                                                                                                                                                                                                                                                                                            print("Guess result:", r.json())


SyntaxError: invalid syntax (ipython-input-869946188.py, line 9)

In [ ]:
# === Repair & finalize "probatio" (simpler version) ===
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# Always load your team ID from the file
team = json.loads(pathlib.Path("/content/icfp_id.json").read_text())
TEAM_ID = team["id"]

# Start fresh with results (we'll fetch everything we need)
results = {}

# Discriminator set
E = ["", "0", "1", "2", "3", "4", "5"]

# Representatives you already discovered: '', '0', '3'
reps = ["", "0", "3"]

def fetch(plans):
    # Remove any plans we've already fetched
    plans = [p for p in plans if p not in results]
    if not plans:
        return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=60)
    r.raise_for_status()
    data = r.json()
    results.update(dict(zip(plans, data["results"])))
    print(f"Fetched {len(plans)} plans; queryCount now:", data["queryCount"])

# Build the full set of plans to fetch: each rep + all suffixes, and each one-step successor + all suffixes
needed = set()
for s in reps:
    for e in E:
        needed.add(s + e)
    for a in "012345":
        sp = s + a
        for e in E:
            needed.add(sp + e)

# Fetch whatever is missing
fetch(sorted(needed))

# Compute rows for the reps
def row_for(s):
    return tuple(results[s + e][len(s)] for e in E)

rows = {s: row_for(s) for s in reps}

# Map each unique row to a state index
row_to_idx = {rows[reps[i]]: i for i in range(3)}

# Build transitions
n = 3
delta = [[None]*6 for _ in range(n)]
for i, s in enumerate(reps):
    for a in range(6):
        sp = s + str(a)
        r = row_for(sp)
        delta[i][a] = row_to_idx[r]

# Room labels and starting room
rooms = [rows[s][0] for s in reps]
starting_idx = 0

# Build undirected connections
connections = []
seen_edges = set()
for i in range(n):
    for a in range(6):
        j = delta[i][a]
        for bb in range(6):
            if delta[j][bb] == i:
                key = tuple(sorted([(i,a),(j,bb)]))
                if key not in seen_edges:
                    seen_edges.add(key)
                    connections.append({
                        "from":{"room":i,"door":a},
                        "to":{"room":j,"door":bb}
                    })
                break

map_obj = {"rooms": rooms, "startingRoom": starting_idx, "connections": connections}
print("\n=== Candidate map ===")
print(json.dumps(map_obj, indent=2))

# Optional submission
SUBMIT = False
if SUBMIT:
    r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=30)
    r.raise_for_status()
    print("Guess result:", r.json())


Fetched 115 plans; queryCount now: 231

=== Candidate map ===
{
  "rooms": [
    0,
    2,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 5
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 1,
        "door

In [ ]:
SUBMIT = True


In [ ]:
r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=30)
r.raise_for_status()
print("Guess result:", r.json())


Guess result: {'correct': True}


In [ ]:
# === Compact Active Learner for the next problem (query-efficient) ===
import json, pathlib, requests, random
from collections import defaultdict

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# 0) TEAM ID & problem
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]

# Pick your next problem here:
PROBLEM = "n020"   # ← change to the official next problem name you want
# If you’re unsure of names, pick one you know, or re-run with another.

# 1) Select
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": PROBLEM}, timeout=30)
r.raise_for_status()
print("Selected:", r.json())

# 2) Learner setup
E = [""]                       # discriminators (we’ll add short ones only when needed)
S = {""}                       # prefixes (candidate state reps)
results = {}                   # plan -> label sequence cache
ROW = {}                       # prefix -> row tuple over E
alphabet = "012345"

MAX_SUFFIX_LEN = 2             # keep suffixes short (<= 2)
MAX_PREFIX_DEPTH = 4           # keep prefixes shallow to keep routes short
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

def explore(plans):
    """Batch /explore; cache all results."""
    todo = [p for p in plans if p not in results]
    if not todo:
        return
    rr = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
    rr.raise_for_status()
    data = rr.json()
    for p, seq in zip(todo, data["results"]):
        results[p] = seq
    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

def ensure_row(s):
    """Ensure we measured ROW[s] over all suffixes E."""
    plans = [s + e for e in E]
    explore(plans)
    ROW[s] = tuple(results[s+e][len(s)] for e in E)

def row_eq(s,t):
    return ROW[s] == ROW[t]

# 3) Initial measurement
ensure_row("")

# 4) Main learning loop:
#    - closure: for every s in S and each door a, ensure s+a is measured and see if it’s a new state row
#    - if ambiguous states exist (equal rows but behave differently downstream), add a short suffix to E to split them
#    - stop when no new states found and no ambiguity remains
def successors(prefix):
    if len(prefix) >= MAX_PREFIX_DEPTH:    # don’t let prefixes explode
        return []
    return [prefix + a for a in alphabet]

def discover_new_states():
    """Return number of new states added to S in this pass."""
    new_count = 0
    current_rows = {ROW[s] for s in S}
    needed_prefixes = []
    for s in list(S):
        for sp in successors(s):
            needed_prefixes.append(sp)
    # measure all sp over current E
    miss = []
    for sp in needed_prefixes:
        for e in E:
            if sp+e not in results:
                miss.append(sp+e)
    explore(miss)

    # record rows for each sp and add if new
    for sp in needed_prefixes:
        if sp not in ROW:
            ROW[sp] = tuple(results[sp+e][len(sp)] for e in E)
        if ROW[sp] not in current_rows:
            S.add(sp)         # new state discovered
            current_rows.add(ROW[sp])
            new_count += 1
    return new_count

def find_ambiguity():
    """If two different prefixes have equal rows but lead differently under some a, return (s,t). Else None."""
    # map row -> canonical rep
    row_to_rep = {}
    for s in sorted(S, key=len):
        r = ROW[s]
        if r not in row_to_rep:
            row_to_rep[r] = s
    # for each state-rep, compare 1-step successors
    reps = list(row_to_rep.values())
    for i in range(len(reps)):
        for j in range(i+1, len(reps)):
            s, t = reps[i], reps[j]
            if not row_eq(s,t):
                continue
            # same row; check successors’ rows with current E
            for a in alphabet:
                sa, ta = s+a, t+a
                # ensure rows exist for sa/ta
                for u in (sa, ta):
                    if u not in ROW:
                        ensure_row(u)
                if ROW[sa] != ROW[ta]:
                    return (s, t)  # ambiguous: equal rows but distinguishable next-step structure
    return None

def add_distinguishing_suffix(s, t):
    """Try to add a short suffix e that distinguishes s and t."""
    # Try all length-1 suffixes, then all length-2
    tried = set(E)
    candidates = [d for d in alphabet if d not in tried]
    if not candidates:
        candidates = []
    if MAX_SUFFIX_LEN >= 2:
        candidates += [a+b for a in alphabet for b in alphabet if a+b not in tried]
    for e in candidates:
        # measure s+e and t+e
        explore([s+e, t+e])
        # extend E and recompute their rows
        oldE = list(E)
        E.append(e)
        ROW[s] = tuple(results[s+ee][len(s)] for ee in E)
        ROW[t] = tuple(results[t+ee][len(t)] for ee in E)
        if ROW[s] != ROW[t]:
            print("Added distinguishing suffix:", e)
            return True
        # didn’t help—roll back suffix addition
        E.pop()
        # NOTE: rows remain consistent for oldE
    return False

rounds = 0
while True:
    rounds += 1
    new_states = discover_new_states()
    amb = find_ambiguity()
    if amb:
        s,t = amb
        if not add_distinguishing_suffix(s,t):
            # Fallback: add a random short suffix (rare)
            cand = "".join(random.choice(alphabet) for _ in range(1 if random.random()<0.7 else 2))
            print("Fallback suffix:", cand)
            E.append(cand)
            # refresh rows for all known s
            explore([s+cand for s in S])
            for s in S:
                ROW[s] = tuple(results[s+e][len(s)] for e in E)
        continue
    if new_states == 0:
        break

# 5) Build compact state set and transitions
# representatives: pick shortest prefix for each unique row
row_to_rep = {}
for s in sorted(S, key=len):
    r = ROW[s]
    if r not in row_to_rep:
        row_to_rep[r] = s
reps = list(row_to_rep.values())
reps.sort(key=len)
idx_of = {row: i for i, row in enumerate([ROW[s] for s in reps])}

# transitions via row lookup
n = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, s in enumerate(reps):
    for a in range(6):
        sp = s + str(a)
        # ensure row measured for sp
        if sp not in ROW:
            ensure_row(sp)
        delta[i][a] = idx_of[ROW[sp]]

# labels from E’s empty-suffix column
# find index of "" in E (always present)
e0 = E.index("")
rooms = [ROW[s][e0] for s in reps]

# starting state index
start_idx = idx_of[ROW[""]]

# undirected connections (pair opposite doors)
connections, seen_edges = [], set()
for i in range(n):
    for a in range(6):
        j = delta[i][a]
        # find partner door at j that returns to i
        b = None
        for bb in range(6):
            if delta[j][bb] == i:
                b = bb
                break
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen_edges:
            seen_edges.add(key)
            connections.append({
                "from": {"room": i, "door": a},
                "to":   {"room": j, "door": b}
            })

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== Learned map ===")
print("States discovered:", n, " (|E| =", len(E), ")")
print(json.dumps(map_obj, indent=2))

# 6) Submit (set True when you're ready)
SUBMIT = False
if SUBMIT:
    rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
    rr.raise_for_status()
    print("Guess result:", rr.json())


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/select

In [ ]:
import json, pathlib, requests
BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]

# 1) Minimal SELECT to test the pipe (probatio always exists)
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
print("probatio status:", r.status_code, r.text)  # should be 200

# 2) Now try your chosen problem name; print the server's error body if it fails
PROBLEM = "primus"   # <-- replace with an exact name from the scoreboard
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": PROBLEM}, timeout=30)
print("select status:", r.status_code)
print("response:", r.text)
r.raise_for_status()


probatio status: 200 {"problemName":"primus"}
select status: 200
response: {"problemName":"primus"}


In [ ]:
# === Compact learner for 'primus' (n≈6) ===
import json, pathlib, requests, random

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]
alphabet = "012345"

# Discriminators and prefixes (grow only as needed)
E = [""]                 # start with just epsilon
S = {""}                 # start-state representative
results = {}             # plan -> label sequence
ROW = {}                 # prefix -> row over E

MAX_PREFIX_DEPTH = 4     # keeps routes short
MAX_SUFFIX_LEN = 2
random.seed(42)

def explore(plans):
    todo = [p for p in plans if p not in results]
        if not todo: return
            r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
                r.raise_for_status()
                    data = r.json()
                        for p, seq in zip(todo, data["results"]):
                                results[p] = seq
                                    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

                                    def ensure_row(s):
                                        explore([s+e for e in E])
                                            ROW[s] = tuple(results[s+e][len(s)] for e in E)

                                            def successors(s):
                                                if len(s) >= MAX_PREFIX_DEPTH: return []
                                                    return [s+a for a in alphabet]

                                                    # Initial measurement
                                                    ensure_row("")

                                                    def discover_new_states():
                                                        newc = 0
                                                            current = {ROW[s] for s in S}
                                                                # ensure one-step successors measured (under current E)
                                                                    need = []
                                                                        for s in list(S):
                                                                                for sp in successors(s):
                                                                                            for e in E:
                                                                                                            if sp+e not in results:
                                                                                                                                need.append(sp+e)
                                                                                                                                    explore(need)
                                                                                                                                        # add new distinct rows as new state reps
                                                                                                                                            for s in list(S):
                                                                                                                                                    for sp in successors(s):
                                                                                                                                                                if sp not in ROW:
                                                                                                                                                                                ROW[sp] = tuple(results[sp+e][len(sp)] for e in E)
                                                                                                                                                                                            if ROW[sp] not in current:
                                                                                                                                                                                                            S.add(sp); current.add(ROW[sp]); newc += 1
                                                                                                                                                                                                                return newc

                                                                                                                                                                                                                def add_distinguishing_suffix(s, t):
                                                                                                                                                                                                                    tried = set(E)
                                                                                                                                                                                                                        cands = [d for d in alphabet if d not in tried]
                                                                                                                                                                                                                            if MAX_SUFFIX_LEN >= 2:
                                                                                                                                                                                                                                    cands += [a+b for a in alphabet for b in alphabet if a+b not in tried]
                                                                                                                                                                                                                                        for e in cands:
                                                                                                                                                                                                                                                explore([s+e, t+e])
                                                                                                                                                                                                                                                        E.append(e)
                                                                                                                                                                                                                                                                old_s, old_t = ROW[s], ROW[t]
                                                                                                                                                                                                                                                                        ROW[s] = tuple(results[s+ee][len(s)] for ee in E)
                                                                                                                                                                                                                                                                                ROW[t] = tuple(results[t+ee][len(t)] for ee in E)
                                                                                                                                                                                                                                                                                        if ROW[s] != ROW[t]:
                                                                                                                                                                                                                                                                                                    print("Added distinguishing suffix:", e)
                                                                                                                                                                                                                                                                                                                return True
                                                                                                                                                                                                                                                                                                                        E.pop()
                                                                                                                                                                                                                                                                                                                                ROW[s], ROW[t] = old_s, old_t
                                                                                                                                                                                                                                                                                                                                    return False

                                                                                                                                                                                                                                                                                                                                    def find_ambiguity():
                                                                                                                                                                                                                                                                                                                                        # find two reps with equal rows but differing 1-step structure
                                                                                                                                                                                                                                                                                                                                            row_to_rep = {}
                                                                                                                                                                                                                                                                                                                                                for s in sorted(S, key=len):
                                                                                                                                                                                                                                                                                                                                                        r = ROW[s]
                                                                                                                                                                                                                                                                                                                                                                if r not in row_to_rep:
                                                                                                                                                                                                                                                                                                                                                                            row_to_rep[r] = s
                                                                                                                                                                                                                                                                                                                                                                                reps = list(row_to_rep.values())
                                                                                                                                                                                                                                                                                                                                                                                    for i in range(len(reps)):
                                                                                                                                                                                                                                                                                                                                                                                            for j in range(i+1, len(reps)):
                                                                                                                                                                                                                                                                                                                                                                                                        s,t = reps[i], reps[j]
                                                                                                                                                                                                                                                                                                                                                                                                                    if ROW[s] != ROW[t]: continue
                                                                                                                                                                                                                                                                                                                                                                                                                                for a in alphabet:
                                                                                                                                                                                                                                                                                                                                                                                                                                                sa, ta = s+a, t+a
                                                                                                                                                                                                                                                                                                                                                                                                                                                                for u in (sa, ta):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    if u not in ROW: ensure_row(u)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    if ROW[sa] != ROW[ta]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        return s,t
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            return None

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # Main loop
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            while True:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                new_states = discover_new_states()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    amb = find_ambiguity()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if amb:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                s,t = amb
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if not add_distinguishing_suffix(s,t):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # fallback: add a short random suffix to E and refresh
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                import random
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            e = random.choice(list(alphabet))
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        print("Fallback suffix:", e)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    E.append(e)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                explore([s+e for s in S])
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            for s in S:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ROW[s] = tuple(results[s+ee][len(s)] for ee in E)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    continue
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if new_states == 0:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                break

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Build representatives (shortest prefix per distinct row)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                row_to_rep = {}
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                for s in sorted(S, key=len):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    row_to_rep.setdefault(ROW[s], s)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    reps = [row_to_rep[r] for r in row_to_rep]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    reps.sort(key=len)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    idx_of_row = {ROW[s]: i for i,s in enumerate(reps)}

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # Transitions
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    n = len(reps)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    delta = [[None]*6 for _ in range(n)]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    for i, s in enumerate(reps):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        for a in range(6):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                sp = s+str(a)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if sp not in ROW: ensure_row(sp)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                delta[i][a] = idx_of_row[ROW[sp]]

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Labels and starting index
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                e0 = E.index("")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                rooms = [ROW[s][e0] for s in reps]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                start_idx = idx_of_row[ROW[""]]

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Undirected connections
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                connections, seen = [], set()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                for i in range(n):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    for a in range(6):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            j = delta[i][a]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    b = next(bb for bb in range(6) if delta[j][bb] == i)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            key = tuple(sorted([(i,a),(j,b)]))
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    if key not in seen:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                seen.add(key)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            print("\n=== Learned map ===")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            print("States discovered:", n, " |E|:", len(E))
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            import json
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            print(json.dumps(map_obj, indent=2))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # Submit when ready
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            SUBMIT = False
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            if SUBMIT:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    r.raise_for_status()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        print("Guess result:", r.json())


IndentationError: unexpected indent (ipython-input-230231126.py, line 20)

In [ ]:
# === Compact learner for 'primus' (n≈6) ===
import json, pathlib, requests, random

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]
alphabet = "012345"

# Discriminators and prefixes (grow only as needed)
E = [""]                 # start with just epsilon
S = {""}                 # start-state representative
results = {}             # plan -> label sequence
ROW = {}                 # prefix -> row over E

MAX_PREFIX_DEPTH = 4     # keeps routes short
MAX_SUFFIX_LEN = 2
random.seed(42)

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
    r.raise_for_status()
    data = r.json()
    for p, seq in zip(todo, data["results"]):
        results[p] = seq
    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

def ensure_row(s):
    explore([s+e for e in E])
    ROW[s] = tuple(results[s+e][len(s)] for e in E)

def successors(s):
    if len(s) >= MAX_PREFIX_DEPTH: return []
    return [s+a for a in alphabet]

# Initial measurement
ensure_row("")

def discover_new_states():
    newc = 0
    current = {ROW[s] for s in S}
    # ensure one-step successors measured (under current E)
    need = []
    for s in list(S):
        for sp in successors(s):
            for e in E:
                if sp+e not in results:
                    need.append(sp+e)
    explore(need)
    # add new distinct rows as new state reps
    for s in list(S):
        for sp in successors(s):
            if sp not in ROW:
                ROW[sp] = tuple(results[sp+e][len(sp)] for e in E)
            if ROW[sp] not in current:
                S.add(sp); current.add(ROW[sp]); newc += 1
    return newc

def add_distinguishing_suffix(s, t):
    tried = set(E)
    cands = [d for d in alphabet if d not in tried]
    if MAX_SUFFIX_LEN >= 2:
        cands += [a+b for a in alphabet for b in alphabet if a+b not in tried]
    for e in cands:
        explore([s+e, t+e])
        E.append(e)
        old_s, old_t = ROW[s], ROW[t]
        ROW[s] = tuple(results[s+ee][len(s)] for ee in E)
        ROW[t] = tuple(results[t+ee][len(t)] for ee in E)
        if ROW[s] != ROW[t]:
            print("Added distinguishing suffix:", e)
            return True
        E.pop()
        ROW[s], ROW[t] = old_s, old_t
    return False

def find_ambiguity():
    # find two reps with equal rows but differing 1-step structure
    row_to_rep = {}
    for s in sorted(S, key=len):
        r = ROW[s]
        if r not in row_to_rep:
            row_to_rep[r] = s
    reps = list(row_to_rep.values())
    for i in range(len(reps)):
        for j in range(i+1, len(reps)):
            s,t = reps[i], reps[j]
            if ROW[s] != ROW[t]: continue
            for a in alphabet:
                sa, ta = s+a, t+a
                for u in (sa, ta):
                    if u not in ROW: ensure_row(u)
                if ROW[sa] != ROW[ta]:
                    return s,t
    return None

# Main loop
while True:
    new_states = discover_new_states()
    amb = find_ambiguity()
    if amb:
        s,t = amb
        if not add_distinguishing_suffix(s,t):
            # fallback: add a short random suffix to E and refresh
            import random
            e = random.choice(list(alphabet))
            print("Fallback suffix:", e)
            E.append(e)
            explore([s+e for s in S])
            for s in S:
                ROW[s] = tuple(results[s+ee][len(s)] for ee in E)
        continue
    if new_states == 0:
        break

# Build representatives (shortest prefix per distinct row)
row_to_rep = {}
for s in sorted(S, key=len):
    row_to_rep.setdefault(ROW[s], s)
reps = [row_to_rep[r] for r in row_to_rep]
reps.sort(key=len)
idx_of_row = {ROW[s]: i for i,s in enumerate(reps)}

# Transitions
n = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, s in enumerate(reps):
    for a in range(6):
        sp = s+str(a)
        if sp not in ROW: ensure_row(sp)
        delta[i][a] = idx_of_row[ROW[sp]]

# Labels and starting index
e0 = E.index("")
rooms = [ROW[s][e0] for s in reps]
start_idx = idx_of_row[ROW[""]]

# Undirected connections
connections, seen = [], set()
for i in range(n):
    for a in range(6):
        j = delta[i][a]
        b = next(bb for bb in range(6) if delta[j][bb] == i)
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== Learned map ===")
print("States discovered:", n, " |E|:", len(E))
import json
print(json.dumps(map_obj, indent=2))

# Submit when ready
SUBMIT = False
if SUBMIT:
    r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
    r.raise_for_status()
    print("Guess result:", r.json())


Fetched 1 plans; queryCount: 2
Fetched 6 plans; queryCount: 9
Fetched 18 plans; queryCount: 28


StopIteration: 

In [ ]:
import requests, json

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

resp = requests.get(f"{BASE}/select", timeout=30)
resp.raise_for_status()
problems = resp.json()

print("Raw response:")
print(json.dumps(problems, indent=2))

# Try to print a clean list of names if present
print("\nProblem names detected:")
names = []
for item in problems:
    if isinstance(item, str):
        names.append(item)
    elif isinstance(item, dict):
        # Common keys used by endpoints
        for key in ("name", "problemName", "id", "title"):
            if key in item:
                names.append(item[key])
                break
        else:
            names.append(str(item))
    else:
        names.append(str(item))

for i, name in enumerate(names):
    print(f"{i:2d}. {name}")


Raw response:
[
  {
    "problem": "probatio",
    "size": 3
  },
  {
    "problem": "primus",
    "size": 6
  },
  {
    "problem": "secundus",
    "size": 12
  },
  {
    "problem": "tertius",
    "size": 18
  },
  {
    "problem": "quartus",
    "size": 24
  },
  {
    "problem": "quintus",
    "size": 30
  }
]

Problem names detected:
 0. {'problem': 'probatio', 'size': 3}
 1. {'problem': 'primus', 'size': 6}
 2. {'problem': 'secundus', 'size': 12}
 3. {'problem': 'tertius', 'size': 18}
 4. {'problem': 'quartus', 'size': 24}
 5. {'problem': 'quintus', 'size': 30}


In [ ]:
# === Patch: refine E until all back-edges exist, then rebuild transitions+connections ===
import requests, json

alphabet = "012345"

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
    r.raise_for_status()
    data = r.json()
    for p, seq in zip(todo, data["results"]):
        results[p] = seq
    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

def ensure_row(s):
    explore([s+e for e in E])
    ROW[s] = tuple(results[s+e][len(s)] for e in E)

def refresh_all_rows():
    # measure all prefixes we already know under the (possibly larger) E
    need = []
    for s in S:
        for e in E:
            if s+e not in results:
                need.append(s+e)
    explore(need)
    for s in S:
        ROW[s] = tuple(results[s+e][len(s)] for e in E)

def rebuild_states_and_delta():
    # representatives: shortest prefix for each distinct row
    row_to_rep = {}
    for s in sorted(S, key=len):
        row_to_rep.setdefault(ROW[s], s)
    reps = [row_to_rep[r] for r in row_to_rep]
    reps.sort(key=len)

    idx_of_row = {ROW[s]: i for i, s in enumerate(reps)}
    n = len(reps)

    # transitions via row lookup
    delta = [[None]*6 for _ in range(n)]
    for i, s in enumerate(reps):
        for a in range(6):
            sp = s + str(a)
            if sp not in ROW: ensure_row(sp)
            delta[i][a] = idx_of_row[ROW[sp]]
    return reps, idx_of_row, delta

def all_back_edges_exist(delta):
    n = len(delta)
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if not any(delta[j][bb] == i for bb in range(6)):
                return False
    return True

# 1) Rebuild once with your current E
reps, idx_of_row, delta = rebuild_states_and_delta()

# 2) If any missing back-edges, add distinguishing suffixes and retry
added = []
if not all_back_edges_exist(delta):
    # try all single-digit suffixes not in E, then length-2 if needed
    cand = [d for d in alphabet if d not in E]
    cand += [a+b for a in alphabet for b in alphabet if (a+b) not in E]
    for e in cand:
        E.append(e)
        added.append(e)
        refresh_all_rows()
        reps, idx_of_row, delta = rebuild_states_and_delta()
        if all_back_edges_exist(delta):
            print("Added distinguishing suffix(es):", added)
            break

# 3) Build final map
e0 = E.index("")
rooms = [ROW[s][e0] for s in reps]
start_idx = idx_of_row[ROW[""]]

connections, seen = [], set()
for i in range(len(reps)):
    for a in range(6):
        j = delta[i][a]
        # guaranteed to exist now
        b = next(bb for bb in range(6) if delta[j][bb] == i)
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== Refined map ===")
print("States discovered:", len(reps), "|E|:", len(E), "Added suffixes:", added)
print(json.dumps(map_obj, indent=2))

# Uncomment to submit
# SUBMIT = True
# if SUBMIT:
#     r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
#     r.raise_for_status()
#     print("Guess result:", r.json())


KeyError: (3,)

In [ ]:
# === Robust refine+rebuild: closure under successors + add suffixes until all back-edges exist ===
import requests, json

alphabet = "012345"

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
    r.raise_for_status()
    data = r.json()
    for p, seq in zip(todo, data["results"]):
        results[p] = seq
    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

def ensure_row(s):
    # ensure ROW[s] over current E
    missing = [s+e for e in E if s+e not in results]
    explore(missing)
    ROW[s] = tuple(results[s+e][len(s)] for e in E)

def refresh_all_rows_for(prefixes):
    need = []
    for s in prefixes:
        for e in E:
            if s+e not in results:
                need.append(s+e)
    explore(need)
    for s in prefixes:
        ROW[s] = tuple(results[s+e][len(s)] for e in E)

def build_closed_representatives():
    """
    Return (reps, idx_of_row) where reps is closed under 1-step successors:
    every row seen at s or s+a has a representative in reps.
    """
    # Start from current S (known prefixes)
    work = set(S)
    # Ensure rows exist for all in work
    refresh_all_rows_for(work)

    # Map row -> representative (pick shortest prefix)
    row_to_rep = {}
    for s in sorted(work, key=len):
        row_to_rep.setdefault(ROW[s], s)

    # Closure under successors: if any s+a yields a row with no rep, add that sp as a rep
    changed = True
    while changed:
        changed = False
        reps_now = list(row_to_rep.values())
        # ensure rows for all successors of current reps
        succs = set()
        for s in reps_now:
            for a in alphabet:
                succs.add(s + a)
        refresh_all_rows_for(succs)

        for s in reps_now:
            for a in alphabet:
                sp = s + a
                r = ROW[sp]
                if r not in row_to_rep:
                    row_to_rep[r] = sp  # adopt sp as representative for this new row
                    changed = True

    reps = list(row_to_rep.values())
    reps.sort(key=len)
    idx_of_row = {ROW[s]: i for i, s in enumerate(reps)}
    return reps, idx_of_row

def build_delta(reps, idx_of_row):
    n = len(reps)
    delta = [[None]*6 for _ in range(n)]
    for i, s in enumerate(reps):
        for a in range(6):
            sp = s + str(a)
            if sp not in ROW: ensure_row(sp)
            delta[i][a] = idx_of_row[ROW[sp]]
    return delta

def all_back_edges_exist(delta):
    n = len(delta)
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if not any(delta[j][bb] == i for bb in range(6)):
                return False
    return True

# Try to close & rebuild; if back-edges missing, add short suffixes and retry
added_suffixes = []
while True:
    reps, idx_of_row = build_closed_representatives()
    delta = build_delta(reps, idx_of_row)
    if all_back_edges_exist(delta):
        break
    # add a distinguishing suffix and refresh
    # try all single digits not yet in E, then length-2
    cands = [d for d in alphabet if d not in E] + [a+b for a in alphabet for b in alphabet if (a+b) not in E]
    if not cands:
        raise RuntimeError("Ran out of candidate suffixes to distinguish states.")
    e = cands[0]
    E.append(e)
    added_suffixes.append(e)
    # refresh rows for all known prefixes and their 1-step successors
    refresh_all_rows_for(S | {s+a for s in S for a in alphabet})

# Build final map
e0 = E.index("")
rooms = [ROW[s][e0] for s in reps]
start_idx = idx_of_row[ROW[""]]

connections, seen = [], set()
for i in range(len(reps)):
    for a in range(6):
        j = delta[i][a]
        b = next(bb for bb in range(6) if delta[j][bb] == i)  # now guaranteed
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== Refined map ===")
print("States discovered:", len(reps), "|E|:", len(E), "Added suffixes:", added_suffixes)
print(json.dumps(map_obj, indent=2))

# Optional submit
# SUBMIT = True
# if SUBMIT:
#     rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
#     rr.raise_for_status()
#     print("Guess result:", rr.json())


Fetched 21 plans; queryCount: 50
Fetched 21 plans; queryCount: 72
Fetched 21 plans; queryCount: 94
Fetched 21 plans; queryCount: 116
Fetched 21 plans; queryCount: 138
Fetched 21 plans; queryCount: 160
Fetched 21 plans; queryCount: 182
Fetched 21 plans; queryCount: 204
Fetched 21 plans; queryCount: 226
Fetched 21 plans; queryCount: 248
Fetched 21 plans; queryCount: 270
Fetched 21 plans; queryCount: 292
Fetched 21 plans; queryCount: 314
Fetched 21 plans; queryCount: 336
Fetched 21 plans; queryCount: 358
Fetched 21 plans; queryCount: 380
Fetched 21 plans; queryCount: 402
Fetched 21 plans; queryCount: 424
Fetched 21 plans; queryCount: 446
Fetched 21 plans; queryCount: 468
Fetched 21 plans; queryCount: 490
Fetched 21 plans; queryCount: 512
Fetched 21 plans; queryCount: 534
Fetched 21 plans; queryCount: 556
Fetched 21 plans; queryCount: 578
Fetched 21 plans; queryCount: 600
Fetched 21 plans; queryCount: 622
Fetched 21 plans; queryCount: 644
Fetched 21 plans; queryCount: 666
Fetched 21 plans;

RuntimeError: Ran out of candidate suffixes to distinguish states.

In [ ]:
# Reselect primus to start fresh
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
print("select:", r.status_code, r.text)
r.raise_for_status()


select: 400 {"error":"Error: couldn't find matching record"}


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/select

In [ ]:
import json, pathlib, requests
from collections import deque

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]
alphabet = "012345"

# --- helpers ---
results = {}            # plan -> label sequence
def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return
    r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": todo}, timeout=120)
    r.raise_for_status()
    data = r.json()
    for p, seq in zip(todo, data["results"]):
        results[p] = seq
    print(f"Fetched {len(todo)} plans; queryCount:", data["queryCount"])

def label_at(plan, pos):  # pos is index into the returned path
    return results[plan][pos]

# --- learner state ---
E = [""]                 # discriminators, start minimal
S = set([""])            # discovered prefixes (candidates)
ROW = {}                 # ROW[s] = tuple(results[s+e][len(s)] for e in E)
MAX_PREFIX_DEPTH = 6     # allow deeper prefixes to reveal structure

def ensure_row(s):
    # measure s+e for current E
    explore([s+e for e in E if s+e not in results])
    ROW[s] = tuple(label_at(s+e, len(s)) for e in E)

def successors(s):
    return [s+a for a in alphabet] if len(s) < MAX_PREFIX_DEPTH else []

# 1) seed with start
ensure_row("")

# 2) BFS expand prefixes; add a prefix only if it creates a NEW row
def discover_prefixes():
    q = deque([""])
    seen = set([""])
    rows_seen = {ROW[""]}
    while q:
        s = q.popleft()
        for sp in successors(s):
            if sp in seen: continue
            seen.add(sp)
            # measure sp over E
            ensure_row(sp)
            if ROW[sp] not in rows_seen:
                S.add(sp)
                rows_seen.add(ROW[sp])
            q.append(sp)

discover_prefixes()

# 3) Build representatives (shortest prefix per distinct row)
def representatives():
    row_to_rep = {}
    for s in sorted(S, key=len):
        row_to_rep.setdefault(ROW[s], s)
    reps = [row_to_rep[r] for r in row_to_rep]
    reps.sort(key=len)
    idx_of_row = {ROW[s]: i for i, s in enumerate(reps)}
    return reps, idx_of_row

reps, idx_of_row = representatives()

def build_delta(reps, idx_of_row):
    n = len(reps)
    delta = [[None]*6 for _ in range(n)]
    for i, s in enumerate(reps):
        for a in range(6):
            sp = s+str(a)
            # ensure measured
            if sp not in ROW: ensure_row(sp)
            delta[i][a] = idx_of_row[ROW[sp]]
    return delta

def all_back_edges_exist(delta):
    n = len(delta)
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if not any(delta[j][bb] == i for bb in range(6)):
                return False
    return True

# 4) If ambiguous: find two reps with equal rows but different successor rows, add distinguishing suffix
def find_ambiguity():
    # reps with same row under current E
    row_to_list = {}
    for s in reps:
        row_to_list.setdefault(ROW[s], []).append(s)
    for row_val, group in row_to_list.items():
        if len(group) < 2: continue
        # compare successors
        for i in range(len(group)):
            for j in range(i+1, len(group)):
                s, t = group[i], group[j]
                for a in alphabet:
                    sa, ta = s+a, t+a
                    if sa not in ROW: ensure_row(sa)
                    if ta not in ROW: ensure_row(ta)
                    if ROW[sa] != ROW[ta]:
                        return s, t, a  # suffix 'a' will distinguish
    return None

# 5) Refine until back-edges exist (or no ambiguity)
while True:
    delta = build_delta(reps, idx_of_row)
    if all_back_edges_exist(delta):
        break
    amb = find_ambiguity()
    if not amb:
        # as a fallback, try length-2 distinguishing via two-step where successors mismatch
        # search pairs a,b where s+ab and t+ab rows differ
        done = False
        for a in alphabet:
            for b in alphabet:
                diffs = False
                for s in reps:
                    ensure_row(s+a+b)
                # recompute ambiguity search using 'ab'
                # measure only; deciding globally is heavy—just add the first candidate
                E.append(a+b)
                # refresh ROW for all prefixes under new E
                for s in list(S):
                    explore([s+(a+b)])
                    ROW[s] = tuple(label_at(s+e, len(s)) for e in E)
                reps, idx_of_row = representatives()
                done = True
                break
            if done: break
        if done: continue
        else:
            raise RuntimeError("Could not find distinguishing suffix — consider increasing MAX_PREFIX_DEPTH.")
    s, t, a = amb
    # add the exact distinguishing suffix
    if a not in E:
        E.append(a)
        # refresh all rows under new E
        for s0 in list(S):
            explore([s0+a])
            ROW[s0] = tuple(label_at(s0+e, len(s0)) for e in E)
        # also re-run BFS once to pick up any new rows that become visible
        discover_prefixes()
        reps, idx_of_row = representatives()
        # continue loop, rebuild delta

# 6) Build final map
e0 = E.index("")
rooms = [ROW[s][e0] for s in reps]
start_idx = idx_of_row[ROW[""]]

connections, seen = [], set()
for i in range(len(reps)):
    for a in range(6):
        j = delta[i][a]
        b = next(bb for bb in range(6) if delta[j][bb] == i)
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

print("\n=== Learned map (primus) ===")
print("States discovered:", len(reps), "|E|:", len(E))
print(json.dumps({"rooms": rooms, "startingRoom": start_idx, "connections": connections}, indent=2))

# Optionally submit:
# r = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": {"rooms": rooms, "startingRoom": start_idx, "connections": connections}}, timeout=60)
# print("Guess:", r.status_code, r.text)
app

Fetched 1 plans; queryCount: 2
Fetched 1 plans; queryCount: 4
Fetched 1 plans; queryCount: 6
Fetched 1 plans; queryCount: 8
Fetched 1 plans; queryCount: 10
Fetched 1 plans; queryCount: 12
Fetched 1 plans; queryCount: 14
Fetched 1 plans; queryCount: 16
Fetched 1 plans; queryCount: 18
Fetched 1 plans; queryCount: 20
Fetched 1 plans; queryCount: 22
Fetched 1 plans; queryCount: 24
Fetched 1 plans; queryCount: 26
Fetched 1 plans; queryCount: 28
Fetched 1 plans; queryCount: 30
Fetched 1 plans; queryCount: 32
Fetched 1 plans; queryCount: 34
Fetched 1 plans; queryCount: 36
Fetched 1 plans; queryCount: 38
Fetched 1 plans; queryCount: 40
Fetched 1 plans; queryCount: 42
Fetched 1 plans; queryCount: 44
Fetched 1 plans; queryCount: 46
Fetched 1 plans; queryCount: 48
Fetched 1 plans; queryCount: 50
Fetched 1 plans; queryCount: 52
Fetched 1 plans; queryCount: 54
Fetched 1 plans; queryCount: 56
Fetched 1 plans; queryCount: 58
Fetched 1 plans; queryCount: 60
Fetched 1 plans; queryCount: 62
Fetched 1 pl

KeyboardInterrupt: 

In [ ]:
# === Lean learner for primus (6 rooms) — batched + no tree explosion ===
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = json.loads(pathlib.Path("/content/icfp_id.json").read_text())["id"]
alphabet = "012345"

# ---- Helpers (batched explore) ----
results = {}  # plan -> label sequence

def explore(plans, chunk=200):
    # batch missing plans; print server's queryCount after each POST
    todo = [p for p in plans if p not in results]
    if not todo: return
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        r.raise_for_status()
        data = r.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = seq
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def label_at(plan, pos):
    return results[plan][pos]

# Discriminators (keep minimal)
E = [""]

def row_of(prefix):
    # with E=[""], the row is just the label at the position |prefix|
    return (label_at(prefix, len(prefix)),)

# ---- Core idea: expand ONLY representatives and their 1-step successors ----
# Start with the empty prefix as the only representative
reps =


SyntaxError: invalid syntax (ipython-input-436890943.py, line 36)

In [ ]:
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
print("select:", r.status_code, r.text)
r.raise_for_status()

select: 200 {"problemName":"primus"}


In [ ]:
BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

In [ ]:
TEAM_ID = "9QSCzqm4wPQ0qkXXQ9nLtQ"

In [ ]:
import requests, random, string

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# If you already know an ID, put it here; otherwise leave it empty:
TEAM_ID = "9QSCzqm4wPQ0qkXXQ9nLtQ".strip()

TEAM_NAME = "wheezards"
LANGUAGE  = "Python"
EMAIL     = "tarstars@gmail.com"

def try_select(team_id, problem):
    return requests.post(f"{BASE}/select", json={"id": team_id, "problemName": problem}, timeout=30)

# 1) If we have a TEAM_ID, test it on a safe problem:
valid_id = False
if TEAM_ID:
    resp = try_select(TEAM_ID, "probatio")
    print("probatio select:", resp.status_code, resp.text)
    valid_id = resp.ok

# 2) If there’s no ID or it’s invalid, register a new team (adding a suffix if needed)
if not valid_id:
    for attempt in range(3):
        suffix = "" if attempt == 0 else "-" + ''.join(random.choice(string.ascii_lowercase+string.digits) for _ in range(3))
        reg = requests.post(
            f"{BASE}/register",
            json={"name": TEAM_NAME + suffix, "pl": LANGUAGE, "email": EMAIL},
            timeout=30
        )
        print("REGISTER:", reg.status_code, reg.text)
        if reg.ok:
            TEAM_ID = reg.json()["id"]
            break
    else:
        raise RuntimeError("Unable to register a team after several attempts.")
    # Verify by selecting probatio
    resp = try_select(TEAM_ID, "probatio")
    resp.raise_for_status()

print("Using TEAM_ID:", TEAM_ID)

# 3) Now select the six‑room problem
sel = try_select(TEAM_ID, "primus")
print("primus select:", sel.status_code, sel.text)
sel.raise_for_status()


probatio select: 400 {"error":"Error: couldn't find matching record"}
REGISTER: 200 {"id":"tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ"}
Using TEAM_ID: tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ
primus select: 200 {"problemName":"primus"}


In [ ]:
# Lean learner for primus (6 rooms)
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
alphabet = "012345"
results = {}
def explore(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        r.raise_for_status()
        data = r.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = seq
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def label_at(plan, pos): return results[plan][pos]
E = [""]
def row_of(prefix): return (label_at(prefix, len(prefix)),)
reps = [""]
rows = {}
explore([""])
rows[row_of("")] = ""
TARGET_STATES = 6

while True:
    succs = [s + a for s in reps for a in alphabet]
    explore(succs)
    new_reps = []
    for s in reps:
        for a in alphabet:
            sp = s + a
            r = row_of(sp)
            if r not in rows:
                rows[r] = sp
                new_reps.append(sp)
    if not new_reps or len(rows) >= TARGET_STATES:
        break
    reps.extend(new_reps)

print("States discovered:", len(rows), "|E|=", len(E))
reps = [rows[r] for r in rows]
reps.sort(key=len)
idx_of_row = {row_of(rep): i for i, rep in enumerate(reps)}
need = [rep + a for rep in reps for a in alphabet]
explore(need)
n = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps):
    for a in range(6):
        sp = rep + str(a)
        delta[i][a] = idx_of_row[row_of(sp)]

# build rooms / startingRoom / connections…
# (same as earlier)


Fetched 1 plans; queryCount: 2
Fetched 6 plans; queryCount: 9
Fetched 12 plans; queryCount: 22
Fetched 6 plans; queryCount: 29
States discovered: 4 |E|= 1


In [ ]:
# Lean primus learner (batched, no files)
import requests, json

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ"  # use EXACTLY the id returned by /register
alphabet = "012345"

results = {}
def explore(plans, chunk=200):
    # batch missing plans
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        r.raise_for_status()
        data = r.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = seq
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def label_at(plan, pos): return results[plan][pos]
def row_of(prefix): return (label_at(prefix, len(prefix)),)

# representatives and rows
reps = [""]        # start with empty prefix
rows = {}          # row(tuple) -> representative prefix

# seed
explore([""])
rows[row_of("")] = ""

TARGET_STATES = 6  # primus has 6 rooms

# discover states by only expanding reps one step
while True:
    succs = [s + a for s in reps for a in alphabet]
    explore(succs)
    new = []
    for s in reps:
        for a in alphabet:
            sp = s + a
            r = row_of(sp)
            if r not in rows:
                rows[r] = sp
                new.append(sp)
    if not new or len(rows) >= TARGET_STATES:
        break
    reps.extend(new)

print("States discovered:", len(rows))
reps = [rows[r] for r in rows]
reps.sort(key=len)

# index by row, ensure we have rep+a measured
idx_of_row = {row_of(rep): i for i, rep in enumerate(reps)}
need = [rep + a for rep in reps for a in alphabet]
explore(need)

# transitions
n = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps):
    for a in range(6):
        sp = rep + str(a)
        delta[i][a] = idx_of_row[row_of(sp)]

# build connections with partner doors
def partner(i,a):
    j = delta[i][a]
    for b in range(6):
        if delta[j][b] == i:
            return j,b
    return None

rooms = [row_of(rep)[0] for rep in reps]
start_idx = idx_of_row[row_of("")]
connections, seen = [], set()
for i in range(n):
    for a in range(6):
        pb = partner(i,a)
        if pb is None: continue
        j,b = pb
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== Candidate map ===")
print(json.dumps(map_obj, indent=2))

# Uncomment to submit:
rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
print("Guess:", rr.status_code, rr.text)


Fetched 1 plans; queryCount: 31
Fetched 6 plans; queryCount: 38
Fetched 12 plans; queryCount: 51
Fetched 6 plans; queryCount: 58
States discovered: 4

=== Candidate map ===
{
  "rooms": [
    0,
    2,
    1,
    3
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 5
      },
  

In [ ]:
# --- Refining primus: add one distinguishing suffix if needed, rebuild, submit ---
import requests, json

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ"  # use exactly your returned id
alphabet = "012345"

results = {}
def explore(plans, chunk=200):
    # Validate
    validate_plans(plans)
    # Send only missing plans, in batches
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        if not r.ok:
            print("EXPLORE batch failed:", r.status_code, r.text[:400])
            r.raise_for_status()
        data = r.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = seq
        print(f"Fetched {len(batch)} plans; queryCount:", data.get("queryCount"))



def lab(plan, pos): return results[plan][pos]

def row_of(prefix, E):
    tup = []
    for e in E:
        pe = prefix + e
        explore([pe])
        tup.append(lab(pe, len(prefix)))
    return tuple(tup)

def learn_reps(E, target_states=6):
    reps, rows = [""], {}
    explore([""])
    rows[row_of("", E)] = ""
    while True:
        succs = [s+a for s in reps for a in alphabet]
        explore(succs)
        new = []
        for s in reps:
            for a in alphabet:
                sp = s+a
                r = row_of(sp, E)
                if r not in rows:
                    rows[r] = sp
                    new.append(sp)
        if not new or len(rows) >= target_states:
            break
        reps.extend(new)
    reps = [rows[r] for r in rows]
    reps.sort(key=len)
    return reps, rows

def build_delta(reps, E):
    idx = {row_of(rep, E): i for i,rep in enumerate(reps)}
    need = [rep+a for rep in reps for a in alphabet]
    explore(need)
    delta = [[None]*6 for _ in range(len(reps))]
    for i, rep in enumerate(reps):
        for a in range(6):
            sp = rep + str(a)
            delta[i][a] = idx[row_of(sp, E)]
    return delta

def partner(delta, i, a):
    j = delta[i][a]
    for b in range(6):
        if delta[j][b] == i:
            return j,b
    return None

# 1) start with E=[""]
E = [""]
reps, rows_map = learn_reps(E, target_states=6)
delta = build_delta(reps, E)

# 2) add one distinguishing suffix if needed
missing = [(i,a) for i in range(len(reps)) for a in range(6) if partner(delta, i, a) is None]
if missing:
    row_to_ids = {}
    for i, rep in enumerate(reps):
        row_to_ids.setdefault(row_of(rep, E), []).append(i)
    pairs = [(g[x], g[y]) for g in row_to_ids.values() if len(g)>1 for x in range(len(g)) for y in range(x+1,len(g))]
    chosen = None
    for e in alphabet:
        explore([reps[i]+e for i in range(len(reps))])
        diffs = any(row_of(reps[i]+e, [""]) != row_of(reps[j]+e, [""]) for (i,j) in pairs)
        if diffs:
            chosen = e
            break
    if chosen is not None:
        E.append(chosen)
        reps, rows_map = learn_reps(E, target_states=6)
        delta = build_delta(reps, E)

# 3) build and submit map
e0 = 0
rooms = [row_of(rep, E)[e0] for rep in reps]
idx = {row_of(rep, E): i for i,rep in enumerate(reps)}
start_idx = idx[row_of("", E)]
connections, seen = [], set()
for i in range(len(reps)):
    for a in range(6):
        p = partner(delta, i, a)
        if p is None: continue
        j,b = p
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in seen:
            seen.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})
map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print(json.dumps(map_obj, indent=2))
rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
print("Guess:", rr.status_code, rr.text)

Fetched 1 plans; queryCount: 10
Fetched 6 plans; queryCount: 17
Fetched 18 plans; queryCount: 36
{
  "rooms": [
    0,
    2,
    3,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 1,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 0,
        "door": 4
      }
    },
    {
      "from": {
        "room": 0,
        "door": 5
      },
      "to": {
        "room": 3,
        "door": 2
      }
    },
    {
      

In [ ]:
import requests, re

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ"  # <-- use EXACT id you got from /register
PROBLEM = "primus"                                    # or "probatio" etc.

def must_ok(r, label):
    print(f"{label}: {r.status_code} {r.text[:200]}")
    r.raise_for_status()

# 0) Re-select the problem to be safe (clears any stale state)
sel = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": PROBLEM}, timeout=30)
must_ok(sel, "SELECT")

# 1) Strict plan validator (digits 0-5 only; allow "" start)
digit_re = re.compile(r"^[0-5]*$")
def validate_plans(plans):
    assert isinstance(plans, list) and plans, "plans must be a non-empty list"
    for p in plans:
        assert isinstance(p, str), "each plan must be a string"
        assert digit_re.match(p), f"bad plan '{p}': only digits 0-5 allowed"
        # (optional) cap length if you like; API usually allows longish routes
        # assert len(p) <= 2000, "plan too long"

# 2) Safe explore wrapper that prints server errors
def explore_safe(plans):
    validate_plans(plans)
    try:
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": plans}, timeout=120)
        must_ok(r, "EXPLORE")
        data = r.json()
        print("queryCount:", data.get("queryCount"))
        return data["results"]
    except requests.HTTPError as e:
        # Print full server response to see the real reason for 400
        print("EXPLORE failed. Body:", getattr(e.response, "text", "(no body)"))
        raise

# 3) Tiny probe to confirm everything works before your learner runs
probe = ["", "0", "1", "2", "3", "4", "5"]
print("Probing …")
res = explore_safe(probe)
for p, seq in zip(probe, res):
    print(f"{p!r} -> {seq}")

SELECT: 200 {"problemName":"primus"}
Probing …
EXPLORE: 200 {"results":[[0],[0,2],[0,3],[0,2],[0,3],[0,0],[0,1]],"queryCount":8}
queryCount: 8
'' -> [0]
'0' -> [0, 2]
'1' -> [0, 3]
'2' -> [0, 2]
'3' -> [0, 3]
'4' -> [0, 0]
'5' -> [0, 1]


In [ ]:
map_obj

{'rooms': [0, 2, 3, 1],
 'startingRoom': 0,
 'connections': [{'from': {'room': 0, 'door': 0},
   'to': {'room': 1, 'door': 1}},
  {'from': {'room': 0, 'door': 1}, 'to': {'room': 2, 'door': 1}},
  {'from': {'room': 0, 'door': 2}, 'to': {'room': 1, 'door': 1}},
  {'from': {'room': 0, 'door': 3}, 'to': {'room': 2, 'door': 1}},
  {'from': {'room': 0, 'door': 4}, 'to': {'room': 0, 'door': 4}},
  {'from': {'room': 0, 'door': 5}, 'to': {'room': 3, 'door': 2}},
  {'from': {'room': 1, 'door': 0}, 'to': {'room': 1, 'door': 0}},
  {'from': {'room': 1, 'door': 2}, 'to': {'room': 3, 'door': 1}},
  {'from': {'room': 1, 'door': 3}, 'to': {'room': 3, 'door': 1}},
  {'from': {'room': 1, 'door': 4}, 'to': {'room': 1, 'door': 0}},
  {'from': {'room': 1, 'door': 5}, 'to': {'room': 0, 'door': 0}},
  {'from': {'room': 2, 'door': 0}, 'to': {'room': 3, 'door': 3}},
  {'from': {'room': 2, 'door': 2}, 'to': {'room': 0, 'door': 1}},
  {'from': {'room': 2, 'door': 3}, 'to': {'room': 0, 'door': 1}},
  {'from': {'r

In [ ]:
import json, requests

# -------------------------------------------------------------------------
# Configuration – fill in your secret TEAM_ID from /register:
BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com X87FWxcYFUh0pd0m2aopZQ"
# Select the 'primus' problem; the call resets any previous state.
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
r.raise_for_status()
print("select:", r.json())

# -------------------------------------------------------------------------
# Helper to batch /explore calls and cache results.  We mask each label to
# its lowest two bits (mod 4) so that states with the same two‑bit label
# are not falsely distinguished.
alphabet = "012345"
results  = {}  # plan → masked label sequence

def explore(plans, chunk=200):
    """Send any unexplored plans to /explore in batches and cache the results."""
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        resp.raise_for_status()
        data = resp.json()
        for p, seq in zip(batch, data["results"]):
            # clip each observed label to the lowest 2 bits
            results[p] = [label & 3 for label in seq]
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def label_at(plan, pos):
    """Return the masked label at the given position for a plan."""
    return results[plan][pos]

# We use a set of distinguishing suffixes E to differentiate rooms that share
# the same immediate label.  For six doors, taking each door once is enough.
E = [""] + list(alphabet)  # ['', '0', '1', '2', '3', '4', '5']

def row_of(prefix):
    """Return the signature of a prefix: the labels after following each suffix in E."""
    return tuple(label_at(prefix + e, len(prefix + e)) for e in E)

# -------------------------------------------------------------------------
# Discover all rooms for the 'primus' instance.
# reps holds representative access strings (prefixes) to each distinct state.
reps = [""]     # start with the empty prefix
rows = {}       # row(tuple) → representative prefix

# measure the empty plan first
explore([""])
rows[row_of("")] = ""

# Expand reps until we see six distinct rows (primus has 6 rooms)
while True:
    # Explore all one‑step extensions of known reps
    succs = [rep + a for rep in reps for a in alphabet]
    explore(succs)
    new_reps = []
    for rep in reps:
        for a in alphabet:
            sp   = rep + a
            r    = row_of(sp)
            if r not in rows:
                rows[r] = sp
                new_reps.append(sp)
    if not new_reps or len(rows) >= 6:
        break
    reps.extend(new_reps)

print("States discovered:", len(rows))

# Sort representatives by length (shortest first)
reps = [rows[r] for r in rows]
reps.sort(key=len)

# Build a mapping from row signature to state index
idx_of_row = {row_of(rep): i for i, rep in enumerate(reps)}

# Ensure all rep+a are explored, then build the transition table
needs = [rep + a for rep in reps for a in alphabet]
explore(needs)

n     = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps):
    for a in range(6):
        sp = rep + str(a)
        delta[i][a] = idx_of_row[row_of(sp)]

# Helper to find the partner door j,b such that from room i via door a we
# reach room j and come back via door b
def partner(i, a):
    j = delta[i][a]
    for b in range(6):
        if delta[j][b] == i:
            return j, b
    return None

# Build the final map object
rooms      = [row_of(rep)[0] for rep in reps]  # two‑bit labels for each room
start_idx  = idx_of_row[row_of("")]            # index of the starting room
connections = []
seen_edges  = set()
for i in range(n):
    for a in range(6):
        pb = partner(i, a)
        if pb is None:
            continue
        j, b  = pb
        edge  = tuple(sorted([(i, a), (j, b)]))
        if edge not in seen_edges:
            seen_edges.add(edge)
            connections.append({
                "from": {"room": i, "door": a},
                "to":   {"room": j, "door": b}
            })

# Print out the map.  It will have 6 room labels, as required.
map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print(json.dumps(map_obj, indent=2))

select: {'problemName': 'primus'}
Fetched 1 plans; queryCount: 2


KeyError: '0'

In [ ]:
import json, requests

# --- Config (uses your earlier printed ID) ---
BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ"  # update if you re-register
alphabet = "012345"

# Select primus
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
r.raise_for_status()
print("select:", r.json())

# --- Explore cache (mask to 2 bits) ---
results = {}  # plan -> list[int]

def explore(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        resp.raise_for_status()
        data = resp.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # keep only lowest 2 bits
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

# Distinguishing suffixes: empty + each single door
E = [""] + list(alphabet)

def ensure_rows(prefixes):
    plans = []
    for p in prefixes:
        for e in E:
            plans.append(p + e)
    explore(plans)

def row_of(prefix):
    # Make sure all prefix+E are measured before reading
    ensure_rows([prefix])
    return tuple(results[prefix + e][len(prefix + e)] for e in E)

# --- Learn primus (6 rooms) ---
reps = [""]   # representative access strings
rows = {}     # row(tuple) -> representative prefix

# seed
ensure_rows([""])
rows[row_of("")] = ""

# discover up to 6 states
TARGET = 6
while True:
    succs = [rep + a for rep in reps for a in alphabet]
    ensure_rows(succs)
    new_reps = []
    for rep in reps:
        for a in alphabet:
            sp = rep + a
            sig = row_of(sp)
            if sig not in rows:
                rows[sig] = sp
                new_reps.append(sp)
    if not new_reps or len(rows) >= TARGET:
        break
    reps.extend(new_reps)

print("States discovered:", len(rows))

# Freeze reps and index
reps = sorted([rows[sig] for sig in rows], key=len)
idx_of = {row_of(rep): i for i, rep in enumerate(reps)}

# Ensure rep+door measured
ensure_rows([rep + a for rep in reps for a in alphabet])

# Build transitions
n = len(reps)
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps):
    for a in range(6):
        sp = rep + str(a)
        delta[i][a] = idx_of[row_of(sp)]

# Find partner doors and build connections
def partner(i, a):
    j = delta[i][a]
    for b in range(6):
        if delta[j][b] == i:
            return j, b
    return None

rooms = [row_of(rep)[0] for rep in reps]  # six 2-bit labels
start_idx = idx_of[row_of("")]
connections, seen = [], set()
for i in range(n):
    for a in range(6):
        pb = partner(i, a)
        if not pb:
            continue
        j, b = pb
        edge = tuple(sorted([(i, a), (j, b)]))
        if edge in seen:
            continue
        seen.add(edge)
        connections.append({"from": {"room": i, "door": a},
                            "to":   {"room": j, "door": b}})

map_obj = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print(json.dumps(map_obj, indent=2))


select: {'problemName': 'primus'}
Fetched 7 plans; queryCount: 8
Fetched 36 plans; queryCount: 45
Fetched 72 plans; queryCount: 118
States discovered: 6
Fetched 108 plans; queryCount: 227
{
  "rooms": [
    0,
    0,
    3,
    1,
    2,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 2,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 2,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,


In [ ]:
import json, requests, itertools

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ"  # keep as is unless you re-register
alphabet = "012345"

# Reselect primus to reset state
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
r.raise_for_status()
print("select:", r.json())

# ------------ Explore cache (clip to 2 bits) ------------
results = {}  # plan -> [labels]

def explore(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        resp.raise_for_status()
        data = resp.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def build_E(depth=1):
    if depth == 1:
        return [""] + list(alphabet)
    if depth == 2:
        pairs = ["".join(p) for p in itertools.product(alphabet, repeat=2)]
        return [""] + list(alphabet) + pairs
    raise ValueError("depth must be 1 or 2")

def ensure(prefixes, E):
    plans = []
    for p in prefixes:
        for e in E:
            plans.append(p + e)
    explore(plans)

def row_of(prefix, E):
    return tuple(results[prefix + e][len(prefix + e)] for e in E)

def learn(E, target_states=6):
    reps = [""]      # representative access strings
    rows = {}        # signature -> rep

    ensure([""], E)
    rows[row_of("", E)] = ""

    while True:
        succs = [rep + a for rep in reps for a in alphabet]
        ensure(succs, E)
        new_reps = []
        for rep in reps:
            for a in alphabet:
                sp = rep + a
                sig = row_of(sp, E)
                if sig not in rows:
                    rows[sig] = sp
                    new_reps.append(sp)
        if not new_reps or len(rows) >= target_states:
            break
        reps.extend(new_reps)

    reps = sorted([rows[s] for s in rows], key=len)
    idx_of = {row_of(rep, E): i for i, rep in enumerate(reps)}
    ensure([rep + a for rep in reps for a in alphabet], E)

    n = len(reps)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps):
        for a in range(6):
            sp = rep + str(a)
            delta[i][a] = idx_of[row_of(sp, E)]

    # partner search
    def partner(i, a):
        j = delta[i][a]
        for b in range(6):
            if delta[j][b] == i:
                return j, b
        return None

    # Build connections and also validate door uniqueness
    rooms = [row_of(rep, E)[0] for rep in reps]
    start_idx = idx_of[row_of("", E)]
    connections, used = [], set()
    ok = True
    for i in range(n):
        for a in range(6):
            pb = partner(i, a)
            if not pb:
                ok = False
                continue
            j, b = pb
            if (i, a) in used or (j, b) in used:
                # door used twice -> invalid under undirected model
                ok = False
            used.add((i, a)); used.add((j, b))
            edge = tuple(sorted([(i, a), (j, b)]))
            if not any(
                (c["from"]["room"], c["from"]["door"], c["to"]["room"], c["to"]["door"]) == (edge[0][0], edge[0][1], edge[1][0], edge[1][1]) or
                (c["from"]["room"], c["from"]["door"], c["to"]["room"], c["to"]["door"]) == (edge[1][0], edge[1][1], edge[0][0], edge[0][1])
                for c in connections
            ):
                connections.append({
                    "from": {"room": edge[0][0], "door": edge[0][1]},
                    "to":   {"room": edge[1][0], "door": edge[1][1]},
                })

    return {
        "rooms": rooms,
        "startingRoom": start_idx,
        "connections": connections,
        "ok": ok,
        "reps": reps,
        "delta": delta
    }

# -------- First pass with 1-step suffixes --------
E1 = build_E(depth=1)
m1 = learn(E1)
print("\nPASS1 ok:", m1["ok"])
print(json.dumps({"rooms": m1["rooms"], "startingRoom": m1["startingRoom"]}, indent=2))

# If door pairing is inconsistent, strengthen signatures with 2-step suffixes and relearn
if not m1["ok"]:
    print("\nRefining with 2-step suffixes…")
    # Re-use cached results; just ask for any missing 2-step plans
    E2 = build_E(depth=2)
    m2 = learn(E2)
    print("PASS2 ok:", m2["ok"])
    print(json.dumps(m2["rooms"], indent=2))
    map_obj = {"rooms": m2["rooms"], "startingRoom": m2["startingRoom"], "connections": m2["connections"]}
else:
    map_obj = {"rooms": m1["rooms"], "startingRoom": m1["startingRoom"], "connections": m1["connections"]}

print("\n=== Candidate map ===")
print(json.dumps(map_obj, indent=2))

# Uncomment to submit:
# rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
# print("guess:", rr.status_code, rr.text)

select: {'problemName': 'primus'}
Fetched 7 plans; queryCount: 8
Fetched 36 plans; queryCount: 45
Fetched 108 plans; queryCount: 154
Fetched 72 plans; queryCount: 227

PASS1 ok: False
{
  "rooms": [
    0,
    2,
    3,
    0,
    1,
    1
  ],
  "startingRoom": 0
}

Refining with 2-step suffixes…
Fetched 108 plans; queryCount: 336
Fetched 200 plans; queryCount: 537
Fetched 200 plans; queryCount: 738
Fetched 176 plans; queryCount: 915
Fetched 200 plans; queryCount: 1116
Fetched 200 plans; queryCount: 1317
Fetched 32 plans; queryCount: 1350
PASS2 ok: False
[
  0,
  2,
  3,
  0,
  1,
  1
]

=== Candidate map ===
{
  "rooms": [
    0,
    2,
    3,
    0,
    1,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 4
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "

In [ ]:
import json, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ"  # same ID you used

# If you already have map_obj from the previous cell, keep it.
# Otherwise, paste your printed values here:
# map_obj = {
#   "rooms": [0,2,3,0,1,1],
#   "startingRoom": 0,
#   "connections": [ ... your list ... ]
# }

def validate_map(m):
    ok = True
    issues = []

    rooms = m.get("rooms", [])
    start = m.get("startingRoom", None)
    conns = m.get("connections", [])

    # 1) basic checks
    if len(rooms) != 6:
        ok = False; issues.append(f"rooms length is {len(rooms)}, expected 6")
    if not isinstance(start, int) or not (0 <= start < len(rooms)):
        ok = False; issues.append(f"startingRoom {start} invalid")

    # 2) door usage: each (room,door) must appear exactly once
    used = {}
    for c in conns:
        for side in ("from","to"):
            try:
                r = c[side]["room"]; d = c[side]["door"]
            except Exception:
                ok = False; issues.append(f"bad connection entry: {c}"); continue
            if not (0 <= r < len(rooms)) or not (0 <= d <= 5):
                ok = False; issues.append(f"out of range endpoint {(r,d)}")
            used[(r,d)] = used.get((r,d), 0) + 1

    # each door should be used exactly once
    for r in range(len(rooms)):
        for d in range(6):
            cnt = used.get((r,d), 0)
            if cnt != 1:
                ok = False; issues.append(f"(room {r}, door {d}) used {cnt} times (need 1)")

    # 3) undirected consistency: partner doors must mutually point back
    # Build quick adjacency by (room,door) -> (room,door)
    adj = {}
    for c in conns:
        a = (c["from"]["room"], c["from"]["door"])
        b = (c["to"]["room"],   c["to"]["door"])
        adj[a] = b
        adj[b] = a
    for a, b in adj.items():
        if adj.get(b) != a:
            ok = False; issues.append(f"not mutually paired: {a} <-> {b}")

    return ok, issues

ok, issues = validate_map(map_obj)
print("VALID:", ok)
if not ok:
    print("\n".join(issues))
else:
    rr = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": map_obj}, timeout=60)
    print("guess:", rr.status_code, rr.text)

VALID: False
(room 0, door 1) used 2 times (need 1)
(room 0, door 2) used 3 times (need 1)
(room 1, door 0) used 2 times (need 1)
(room 1, door 1) used 2 times (need 1)
(room 2, door 1) used 2 times (need 1)
(room 2, door 2) used 2 times (need 1)
(room 3, door 2) used 2 times (need 1)
(room 4, door 0) used 2 times (need 1)
(room 4, door 3) used 2 times (need 1)
(room 5, door 3) used 3 times (need 1)


In [ ]:
import json, requests, itertools

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ"  # same ID you used
alphabet = "012345"

# -- reset to primus each run --
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
r.raise_for_status()
print("select:", r.json())

# ---------- explore cache (clip labels to 2 bits) ----------
results = {}  # plan -> [labels]

def explore(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp  = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": batch}, timeout=120)
        resp.raise_for_status()
        data  = resp.json()
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]   # keep only lowest 2 bits
        print(f"Fetched {len(batch)} plans; queryCount:", data["queryCount"])

def build_E(depth):
    if depth == 1:
        return [""] + list(alphabet)
    elif depth == 2:
        pairs = ["".join(p) for p in itertools.product(alphabet, repeat=2)]
        return [""] + list(alphabet) + pairs
    elif depth == 3:
        pairs = ["".join(p) for p in itertools.product(alphabet, repeat=2)]
        trips = ["".join(p) for p in itertools.product(alphabet, repeat=3)]
        return [""] + list(alphabet) + pairs + trips
    else:
        raise ValueError("depth must be 1..3")

def ensure(prefixes, E):
    plans = []
    for p in prefixes:
        for e in E:
            plans.append(p + e)
    explore(plans)

def row_of(prefix, E):
    return tuple(results[prefix + e][len(prefix + e)] for e in E)

def learn_with_E(E, target_states=6):
    # Discover states
    reps = [""]      # representative access strings
    rows = {}        # signature -> rep

    ensure([""], E)
    rows[row_of("", E)] = ""

    while True:
        succs = [rep + a for rep in reps for a in alphabet]
        ensure(succs, E)
        new_reps = []
        for rep in reps:
            for a in alphabet:
                sp  = rep + a
                sig = row_of(sp, E)
                if sig not in rows:
                    rows[sig] = sp
                    new_reps.append(sp)
        if not new_reps or len(rows) >= target_states:
            break
        reps.extend(new_reps)

    reps = sorted([rows[s] for s in rows], key=len)
    idx  = {row_of(rep, E): i for i, rep in enumerate(reps)}

    # Ensure all one-step successors measured
    ensure([rep + a for rep in reps for a in alphabet], E)

    # Transitions
    n = len(reps)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps):
        for a in range(6):
            sp = rep + str(a)
            delta[i][a] = idx[row_of(sp, E)]

    # Partner door finder
    def partner(i, a):
        j = delta[i][a]
        # Unique door b s.t. back to i
        cands = [b for b in range(6) if delta[j][b] == i]
        return (j, cands[0]) if len(cands) == 1 else None

    # Build connections; also validate uniqueness
    rooms = [row_of(rep, E)[0] for rep in reps]
    start = idx[row_of("", E)]
    used  = {}
    conns = []
    ok = True

    for i in range(n):
        for a in range(6):
            pb = partner(i, a)
            if not pb:
                ok = False
                continue
            j, b = pb
            used[(i,a)] = used.get((i,a), 0) + 1
            used[(j,b)] = used.get((j,b), 0) + 1
            # add undirected edge once
            A, B = (i,a), (j,b)
            if A > B: A, B = B, A
            if not any((c["from"]["room"], c["from"]["door"], c["to"]["room"], c["to"]["door"])==(A[0],A[1],B[0],B[1]) or
                       (c["from"]["room"], c["from"]["door"], c["to"]["room"], c["to"]["door"])==(B[0],B[1],A[0],A[1]) for c in conns):
                conns.append({"from":{"room":A[0], "door":A[1]},
                              "to":  {"room":B[0], "door":B[1]}})

    # door usage must be exactly once
    for i in range(n):
        for a in range(6):
            if used.get((i,a), 0) != 1:
                ok = False

    return {"rooms": rooms, "startingRoom": start, "connections": conns, "ok": ok}

# Try depths 1 -> 2 -> 3
best = None
for depth in (1, 2, 3):
    print(f"\n=== Learning with suffix depth {depth} ===")
    E = build_E(depth)
    m = learn_with_E(E)
    print("ok:", m["ok"], "rooms:", m["rooms"])
    best = m
    if m["ok"]:
        break

map_obj = {"rooms": best["rooms"], "startingRoom": best["startingRoom"], "connections": best["connections"]}
print("\n=== Candidate map ===")
print(json.dumps(map_obj, indent=2))

# Optional: local validator (no submit)
def validate_map(m):
    ok = True
    issues = []
    n = len(m["rooms"])
    used = {}
    for c in m["connections"]:
        for side in ("from","to"):
            r = c[side]["room"]; d = c[side]["door"]
            used[(r,d)] = used.get((r,d), 0) + 1
    for r in range(n):
        for d in range(6):
            cnt = used.get((r,d), 0)
            if cnt != 1:
                ok = False; issues.append(f"(room {r}, door {d}) used {cnt} times")
    return ok, issues

ok, issues = validate_map(map_obj)
print("VALID:", ok)
if not ok:
    print("\n".join(issues))

select: {'problemName': 'primus'}

=== Learning with suffix depth 1 ===
Fetched 7 plans; queryCount: 8
Fetched 36 plans; queryCount: 45
Fetched 36 plans; queryCount: 82
Fetched 108 plans; queryCount: 191
Fetched 36 plans; queryCount: 228
ok: False rooms: [0, 1, 0, 2, 1, 3]

=== Learning with suffix depth 2 ===
Fetched 180 plans; queryCount: 409
Fetched 108 plans; queryCount: 518
Fetched 200 plans; queryCount: 719
Fetched 200 plans; queryCount: 920
Fetched 200 plans; queryCount: 1121
Fetched 12 plans; queryCount: 1134
Fetched 200 plans; queryCount: 1335
Fetched 16 plans; queryCount: 1352
ok: False rooms: [0, 1, 0, 2, 1, 3]

=== Learning with suffix depth 3 ===
Fetched 200 plans; queryCount: 1553
Fetched 200 plans; queryCount: 1754
Fetched 200 plans; queryCount: 1955
Fetched 200 plans; queryCount: 2156
Fetched 200 plans; queryCount: 2357
Fetched 80 plans; queryCount: 2438
Fetched 200 plans; queryCount: 2639
Fetched 200 plans; queryCount: 2840
Fetched 200 plans; queryCount: 3041
Fetched 4

In [ ]:
import json, os, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = os.getenv("ICFPC_ID", "YOUR_SECRET_ID_HERE")  # FIX: prefer env var
alphabet = "012345"

# ---- API helpers -----------------------------------------------------------
def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def select(problem):
    return api("/select", {"id": TEAM_ID, "problemName": problem}, timeout=30)

def explore_plans(plans):
    """Batch /explore calls, dedup, and return (labels, queryCount)."""
    todo = [p for p in plans if p not in results]
    qcount = None
    # keep batches modest to avoid timeouts; server allows batching (spec)
    for i in range(0, len(todo), 200):
        batch = todo[i:i+200]
        resp = api("/explore", {"id": TEAM_ID, "plans": batch})
        qcount = resp["queryCount"]
        for p, seq in zip(batch, resp["results"]):
            results[p] = [val & 3 for val in seq]  # labels are 2-bit, mask for safety
    return qcount

def guess(map_obj):
    return api("/guess", {"id": TEAM_ID, "map": map_obj})

# ---- Observation-table machinery ------------------------------------------
results = {}                # plan -> label sequence (masked to 2 bits)
E = [""] + list(alphabet)   # FIX: distinguishing suffixes = empty + each door

def ensure_row_available(prefix):
    """FIX: make sure all plans needed to compute row_of(prefix) are explored."""
    needed = [prefix + s for s in E]
    explore_plans(needed)

def label_at(plan, pos):
    return results[plan][pos]

def row_of(prefix):
    """Signature across all suffixes in E (labels at end of each plan)."""
    return tuple(label_at(prefix + suffix, len(prefix + suffix)) for suffix in E)

# ---- Discover the automaton (rooms & transitions) -------------------------
def discover():
    # Seed: select problem and explore empty plan
    select("primus")  # this resets previous selection (per spec)
    explore_plans([""])
    ensure_row_available("")  # FIX: ensure all suffix probes for "" exist

    reps = [""]      # representative prefixes for discovered states
    rows = {row_of(""): ""}

    # FIX: grow until closed — no more new rows after exploring successors
    changed = True
    while changed:
        changed = False
        succs = []
        for rep in reps:
            for a in alphabet:
                succs.append(rep + a)
        # We will need row_of(sp) for each succ, so ensure their suffix probes exist
        # in one go to reduce /explore penalties (spec encourages batching).
        bulk_needed = []
        for sp in succs:
            bulk_needed.extend(sp + s for s in E)
        explore_plans(bulk_needed)

        for rep in list(reps):
            for a in alphabet:
                sp = rep + a
                r  = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    reps.append(sp)
                    changed = True

    # Build row->index map
    reps_by_len = sorted((rows[r] for r in rows), key=len)
    idx_of_row  = {row_of(rep): i for i, rep in enumerate(reps_by_len)}

    # Ensure successors are explored (should already be, but cheap to assert)
    explore_plans([rep + a for rep in reps_by_len for a in alphabet])

    # Build transition table
    n = len(reps_by_len)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_by_len):
        for a in range(6):
            sp = rep + str(a)
            delta[i][a] = idx_of_row[row_of(sp)]

    # Room labels = label for empty suffix for each representative
    rooms = [row_of(rep)[0] for rep in reps_by_len]
    start_idx = idx_of_row[row_of("")]

    # Undirected connections via “mutual partner” pairing
    connections, seen = [], set()
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            # find b such that going j --b--> i (mutual door)
            for b in range(6):
                if delta[j][b] == i:
                    edge = tuple(sorted([(i, a), (j, b)]))
                    if edge not in seen:
                        seen.add(edge)
                        connections.append({
                            "from": {"room": i, "door": a},
                            "to":   {"room": j, "door": b}
                        })
                    break

    return {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

if __name__ == "__main__":
    # Build the map
    map_obj = discover()
    print(json.dumps(map_obj, indent=2))

    # Optional: submit the guess (uncomment to actually grade & clear selection)
    # NOTE: You should only call /guess once you’re satisfied; it clears the problem.
    # resp = guess(map_obj)
    # print("Correct?", resp["correct"])

HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/select

In [ ]:
# === PRIMUS data dumper & analyzer (Colab-ready) ============================
import json, collections, requests, math, os

BASE     = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID  = "tarstars@gmail.com 9QSCzqm4wQp0qkXXQ9nLtQ"  # keep inline for tablet use
TEAM_ID  = TEAM_ID.split()[-1]  # only the token part if user pasted "email token"
ALPHA    = "012345"
E        = [""] + list(ALPHA)   # distinguishing suffix set we’ll use
CHUNK    = 200

# ---------- tiny API wrapper with readable errors ----------
def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    if not r.ok:
        raise RuntimeError(f"{path} {r.status_code}: {r.text}")
    return r.json()

def select(problem):  return api("/select",  {"id": TEAM_ID, "problemName": problem}, timeout=30)
def explore(plans):   return api("/explore", {"id": TEAM_ID, "plans": plans})

# ---------- cache of observations: plan -> list[int] ----------
results = {}  # e.g. "01203" -> [l0,l1,l2,l3,l4]

def explore_plans(plans):
    """Batch unexplored plans and cache label sequences (masked to 2 bits)."""
    todo = [p for p in plans if p not in results]
    total_new, last_qc = 0, None
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        resp = explore(batch)
        last_qc = resp.get("queryCount")
        for p, seq in zip(batch, resp["results"]):
            results[p] = [v & 3 for v in seq]   # 2-bit labels
        total_new += len(batch)
        print(f"  fetched {len(batch)}   (queryCount={last_qc})")
    return total_new, last_qc

def ensure_row_available(prefix):
    """Ensure we can compute row_of(prefix) by probing prefix+each suffix in E."""
    need = [prefix + s for s in E]
    explore_plans(need)

def label_at(plan, pos):
    return results[plan][pos]

def row_of(prefix):
    """Row signature across E: labels observed at end of each (prefix+suffix)."""
    return tuple(label_at(prefix + s, len(prefix + s)) for s in E)

# ---------- crawl: BFS to a given depth just to COLLECT data ----------
def bfs_collect(depth=7):
    """
    Explore all plans up to given depth (inclusive), and for each visited prefix
    also explore (prefix + each suffix in E) so rows are computable.
    """
    q = collections.deque([""])
    seen = set([""])
    level = { "": 0 }

    # seed empty plan + its suffix probes
    explore_plans([""])
    ensure_row_available("")

    while q:
        p = q.popleft()
        d = level[p]
        if d >= depth:
            continue
        # expand successors
        succs = [p + a for a in ALPHA]
        # explore their raw sequences first (cheap to batch)
        explore_plans(succs)
        # ensure rows for successors
        bulk = []
        for sp in succs:
            bulk += [sp + s for s in E]
        explore_plans(bulk)
        # enqueue
        for sp in succs:
            if sp not in seen:
                seen.add(sp)
                level[sp] = d + 1
                q.append(sp)
    return seen

# ---------- analysis helpers ----------
def analyze_rows():
    # Compute rows for all prefixes that are fully available
    rows, reps = {}, {}
    count_skipped = 0
    for p in results.keys():
        # we only treat p as a "prefix" if all p+E are known
        if all(p + s in results for s in E):
            r = row_of(p)
            rows.setdefault(r, []).append(p)
        else:
            count_skipped += 1

    # representatives: shortest prefix per row
    rep_list = []
    for r, plist in rows.items():
        rep = min(plist, key=len)
        reps[r] = rep
        rep_list.append(rep)

    rep_list.sort(key=len)
    print(f"Unique rows discovered: {len(rep_list)}")
    print("Shortest representatives (first 15):")
    for i, rep in enumerate(rep_list[:15]):
        print(f"  [{i:02d}] len={len(rep)} rep='{rep}' row={row_of(rep)}")
    if len(rep_list) > 15:
        print(f"  … (+{len(rep_list)-15} more)")
    return reps, rep_list

def build_partial_delta(reps_sorted):
    """Build delta from representatives using row signatures (may be partial)."""
    idx_of_row = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    n = len(reps_sorted)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_sorted):
        for a in range(6):
            sp = rep + str(a)
            # if row_of(sp) available, fill transition
            if all(sp + s in results for s in E):
                delta[i][a] = idx_of_row[row_of(sp)]
    return delta

def partner_edges(delta):
    """Collect mutual edges and list inconsistencies (non-mutual or missing)."""
    n = len(delta)
    edges, non_mutual, missing = set(), [], []
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if j is None:
                missing.append((i, a))
                continue
            # find a back-door b with delta[j][b] == i
            bj = None
            for b in range(6):
                if delta[j][b] == i:
                    bj = b; break
            if bj is None:
                non_mutual.append((i, a, j))
            else:
                edge = tuple(sorted([(i, a), (j, bj)]))
                edges.add(edge)
    return edges, non_mutual, missing

# ---------- run the collection + analysis on PRIMUS ----------
print("Selecting primus …")
print(select("primus"))

print("\nCollecting exploration traces (BFS) …")
seen = bfs_collect(depth=8)        # adjust depth if you want more/less data
print(f"Collected prefixes (<= depth): {len(seen)}")
print(f"Total plans in cache: {len(results)}")

# dump raw observations to a file you can re-load later
dump_path = "/content/primus_results.json"
with open(dump_path, "w") as f:
    json.dump(results, f)
print(f"Saved raw results to {dump_path}")

# quick aggregate stats
max_len = max((len(p) for p in results), default=0)
lens = collections.Counter(len(p) for p in results)
print("\nPlan length histogram (first 15 lengths):")
for L in range(min(lens.keys() or [0]), min(max_len, 15)+1):
    print(f"  len={L:2d}: {lens.get(L,0)} plans")

# analyze rows (candidate rooms)
print("\nAnalyzing rows …")
reps_map, reps_sorted = analyze_rows()

# build partial delta + check consistency
print("\nBuilding partial delta …")
delta = build_partial_delta(reps_sorted)
edges, non_mutual, missing = partner_edges(delta)
print(f"States (unique rows): {len(reps_sorted)}")
print(f"Filled transitions: {sum(1 for i in range(len(delta)) for a in range(6) if delta[i][a] is not None)} / {len(reps_sorted)*6}")
print(f"Unique undirected edges (mutual): {len(edges)}")
print(f"Non-mutual transitions: {len(non_mutual)} (showing up to 10)")
for tup in non_mutual[:10]:
    i,a,j = tup
    print(f"  i={i}, door {a} -> j={j}, but no back-door to i")

print(f"Missing transitions: {len(missing)} (first 20): {missing[:20]}")

# suggest next probes to close gaps (optional)
need_more = []
for i, rep in enumerate(reps_sorted):
    for a in range(6):
        sp = rep + str(a)
        if not all(sp + s in results for s in E):
            need_more += [sp + s for s in E if sp + s not in results]
print(f"\nNext probes to close gaps (unique): {len(set(need_more))}")

Selecting primus …


RuntimeError: /select 400: {"error":"Error: couldn't find matching record"}

In [ ]:
import json, pathlib, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com 9QSCzqm4wPQ0qkXXQ9nLtQ"

r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "primus"}, timeout=30)
print("select:", r.status_code, r.text)
r.raise_for_status()

select: 400 {"error":"Error: couldn't find matching record"}


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/select

In [ ]:
# End-to-end: re-register -> select primus -> collect -> analyze -> save dump
import requests, json, collections

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

# 1) Register to get a fresh id (token)
reg = requests.post(f"{BASE}/register", json={
    "name":  "tarstars",
    "email": "tarstars@gmail.com",
    "pl":    "python"
}, timeout=30)
reg.raise_for_status()
TEAM_ID = reg.json()["id"]
print("New TEAM_ID:", TEAM_ID)

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    if not r.ok:
        raise RuntimeError(f"{path} {r.status_code}: {r.text}")
    return r.json()

def select(problem):  return api("/select",  {"id": TEAM_ID, "problemName": problem}, timeout=30)
def explore(plans):   return api("/explore", {"id": TEAM_ID, "plans": plans})

# 2) Select primus
print("\nSelecting primus …")
print(select("primus"))

# 3) Collect data (BFS + suffix probes)
ALPHA = "012345"
E      = [""] + list(ALPHA)
CHUNK  = 200
results = {}

def explore_plans(plans):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        resp  = explore(batch)
        for p, seq in zip(batch, resp["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={resp.get('queryCount')})")

def ensure_row_available(prefix):
    need = [prefix + s for s in E]
    explore_plans(need)

def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix+s for s in E))

# Seed + BFS
print("\nCollecting …")
explore_plans([""]); ensure_row_available("")
q, seen, depth = collections.deque([""]), set([""]), 8
while q:
    p = q.popleft()
    if len(p) >= depth: continue
    succs = [p+a for a in ALPHA]
    explore_plans(succs)
    bulk = []
    for sp in succs: bulk += [sp + s for s in E]
    explore_plans(bulk)
    for sp in succs:
        if sp not in seen:
            seen.add(sp); q.append(sp)

# 4) Analyze rows + partial transitions
rows = {}
for p in list(results):
    if all(p+s in results for s in E):
        rows.setdefault(row_of(p), []).append(p)

reps = [min(pl, key=len) for pl in rows.values()]
reps.sort(key=len)
print(f"\nUnique rows discovered: {len(reps)} (showing up to 15)")
for i, rep in enumerate(reps[:15]):
    print(f"  [{i:02d}] len={len(rep)} rep='{rep}' row={row_of(rep)}")

idx_of = {row_of(rep): i for i, rep in enumerate(reps)}
delta = [[None]*6 for _ in reps]
for i, rep in enumerate(reps):
    for a in range(6):
        sp = rep + str(a)
        if all(sp+s in results for s in E):
            delta[i][a] = idx_of[row_of(sp)]

non_mutual, missing = [], []
for i in range(len(reps)):
    for a in range(6):
        j = delta[i][a]
        if j is None:
            missing.append((i, a))
        else:
            if all(delta[j][b] != i for b in range(6)):
                non_mutual.append((i, a, j))
print(f"\nFilled transitions: {sum(v is not None for row in delta for v in row)} / {len(reps)*6}")
print("Non-mutual:", len(non_mutual), " Missing:", len(missing))

# 5) Save dump for later reuse
dump_path = "/content/primus_results.json"
with open(dump_path, "w") as f:
    json.dump(results, f)
print("Saved raw results to", dump_path)

New TEAM_ID: tarstars@gmail.com yFD_tf-9rd1LJxZsYIBOeg

Selecting primus …
{'problemName': 'primus'}

  fetched 1 (queryCount=2)
  fetched 6 (queryCount=9)
  fetched 36 (queryCount=46)
  fetched 36 (queryCount=83)
  fetched 36 (queryCount=120)
  fetched 36 (queryCount=157)
  fetched 36 (queryCount=194)
  fetched 36 (queryCount=231)
  fetched 36 (queryCount=268)
  fetched 36 (queryCount=305)
  fetched 36 (queryCount=342)
  fetched 36 (queryCount=379)
  fetched 36 (queryCount=416)
  fetched 36 (queryCount=453)
  fetched 36 (queryCount=490)
  fetched 36 (queryCount=527)
  fetched 36 (queryCount=564)
  fetched 36 (queryCount=601)
  fetched 36 (queryCount=638)
  fetched 36 (queryCount=675)
  fetched 36 (queryCount=712)
  fetched 36 (queryCount=749)
  fetched 36 (queryCount=786)
  fetched 36 (queryCount=823)
  fetched 36 (queryCount=860)
  fetched 36 (queryCount=897)
  fetched 36 (queryCount=934)
  fetched 36 (queryCount=971)
  fetched 36 (queryCount=1008)
  fetched 36 (queryCount=1045)
  fe

KeyboardInterrupt: 

In [ ]:
# Close table with dynamic suffix growth + progress guard; build final map.
import json, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
# Paste TEAM_ID printed by the collector cell:
TEAM_ID = "tarstars@gmail.com yFD_tf-9rd1LJxZsYIBOeg"

with open("/content/primus_results.json") as f:
    results = json.load(f)

ALPHA = "012345"

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    if not r.ok:
        raise RuntimeError(f"{path} {r.status_code}: {r.text}")
    return r.json()

def explore(plans):   return api("/explore", {"id": TEAM_ID, "plans": plans})

def explore_plans(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp  = explore(batch)
        for p, seq in zip(batch, resp["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={resp.get('queryCount')})")

# ---- Suffix-set management -------------------------------------------------
def make_suffixes(level):
    if level <= 1:
        return [""] + list(ALPHA)
    if level == 2:
        return [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]
    # level >= 3 (rarely needed; capped)
    return ([""] + list(ALPHA)
            + [a+b for a in ALPHA for b in ALPHA]
            + [a+b+c for a in ALPHA for b in ALPHA for c in ALPHA])

E_level = 1
E = make_suffixes(E_level)

# Utility with current E
def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

def ensure_rows_for(prefixes):
    need = []
    for p in prefixes:
        need += [p + s for s in E if p + s not in results]
    if need:
        explore_plans(need)

def rows_and_reps():
    rows = {}
    for p in list(results):
        if all(p+s in results for s in E):
            rows.setdefault(row_of(p), []).append(p)
    reps = [min(pl, key=len) for pl in rows.values()]
    reps.sort(key=len)
    return rows, reps

def build_delta(reps_sorted):
    idx_of = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    n = len(reps_sorted)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_sorted):
        for a in range(6):
            sp = rep + str(a)
            if all(sp+s in results for s in E):
                delta[i][a] = idx_of[row_of(sp)]
    return delta

def partner_issues(delta):
    n = len(delta)
    non_mutual, missing = [], []
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if j is None:
                missing.append((i, a))
            else:
                if all(delta[j][b] != i for b in range(6)):
                    non_mutual.append((i, a, j))
    return non_mutual, missing

def probes_for_gaps(reps_sorted, delta, non_mutual, missing):
    need = set()
    for (i, a) in missing:
        sp = reps_sorted[i] + str(a)
        for s in E:
            if sp + s not in results:
                need.add(sp + s)
    for (i, a, j) in non_mutual:
        for b in range(6):
            sp = reps_sorted[j] + str(b)
            for s in E:
                if sp + s not in results:
                    need.add(sp + s)
    return list(need)

def build_map(reps_sorted, delta):
    idx_of = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    rooms = [row_of(rep)[0] for rep in reps_sorted]
    start_idx = idx_of[row_of("")]
    seen = set(); connections = []
    for i in range(len(reps_sorted)):
        for a in range(6):
            j = delta[i][a]
            if j is None: continue
            for b in range(6):
                if delta[j][b] == i:
                    edge = tuple(sorted([(i, a), (j, b)]))
                    if edge not in seen:
                        seen.add(edge)
                        connections.append({
                            "from": {"room": i, "door": a},
                            "to":   {"room": j, "door": b},
                        })
                    break
    return {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

# ---- Closure with progress guard + dynamic E -------------------------------
# Prime the table for current E
rows, reps = rows_and_reps()
ensure_rows_for([r for r in reps] + [r + d for r in reps for d in ALPHA])

def score(delta):
    n = len(delta)
    filled = sum(v is not None for row in delta for v in row)
    miss = n*6 - filled
    return (n, filled, miss)

rounds = 0
stagnant = 0
prev = (-1,-1,10**9)  # (states, filled, missing)
MAX_ROUNDS = 25
MAX_E_LEVEL = 3

while rounds < MAX_ROUNDS:
    rows, reps = rows_and_reps()
    delta = build_delta(reps)
    non_mutual, missing = partner_issues(delta)
    cur = score(delta)
    print(f"Round {rounds}: states={cur[0]} filled={cur[1]} missing={cur[2]} non_mutual={len(non_mutual)} E_level={E_level}")

    # closed?
    if not non_mutual and not missing:
        final_map = build_map(reps, delta)
        print("\n=== MAP (primus) ===")
        print(json.dumps(final_map, indent=2))
        # Optionally submit:
        # resp = api("/guess", {"id": TEAM_ID, "map": final_map}); print("Correct?", resp["correct"])
        break

    # compute probes and send
    probes = probes_for_gaps(reps, delta, non_mutual, missing)
    if probes:
        explore_plans(probes)
    else:
        stagnant += 1

    # progress check
    made_progress = (cur[0] > prev[0]) or (cur[1] > prev[1]) or (cur[2] < prev[2])
    if not made_progress:
        stagnant += 1
    else:
        stagnant = 0
    prev = cur
    rounds += 1

    # If we’re stagnating, expand E (at most to Σ², then Σ³)
    if stagnant >= 2 and E_level < MAX_E_LEVEL:
        E_level += 1
        E = make_suffixes(E_level)
        stagnant = 0
        print(f"\n>>> Expanding suffix set to level {E_level} (|E|={len(E)}) and re-probing reps and exits …")
        # Ensure all row probes for reps and their one-step successors exist under new E
        need = []
        for rep in reps:
            need += [rep + s for s in E if rep + s not in results]
            for a in ALPHA:
                sp = rep + a
                need += [sp + s for s in E if sp + s not in results]
        explore_plans(list(set(need)))

else:
    print("\nStopped due to round cap / lack of progress.")
    print("Try increasing MAX_ROUNDS or allow E_level 3 if still needed.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/primus_results.json'

In [ ]:
results

NameError: name 'results' is not defined

In [ ]:
# Close table with dynamic suffix growth + progress guard; build final map.
import json, requests

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
# Paste TEAM_ID printed by the collector cell:
TEAM_ID = "tarstars@gmail.com yFD_tf-9rd1LJxZsYIBOeg"

with open("/content/primus_results.json") as f:
    results = json.load(f)

ALPHA = "012345"

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    if not r.ok:
        raise RuntimeError(f"{path} {r.status_code}: {r.text}")
    return r.json()

def explore(plans):   return api("/explore", {"id": TEAM_ID, "plans": plans})

def explore_plans(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp  = explore(batch)
        for p, seq in zip(batch, resp["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={resp.get('queryCount')})")

# ---- Suffix-set management -------------------------------------------------
def make_suffixes(level):
    if level <= 1:
        return [""] + list(ALPHA)
    if level == 2:
        return [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]
    # level >= 3 (rarely needed; capped)
    return ([""] + list(ALPHA)
            + [a+b for a in ALPHA for b in ALPHA]
            + [a+b+c for a in ALPHA for b in ALPHA for c in ALPHA])

E_level = 1
E = make_suffixes(E_level)

# Utility with current E
def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

def ensure_rows_for(prefixes):
    need = []
    for p in prefixes:
        need += [p + s for s in E if p + s not in results]
    if need:
        explore_plans(need)

def rows_and_reps():
    rows = {}
    for p in list(results):
        if all(p+s in results for s in E):
            rows.setdefault(row_of(p), []).append(p)
    reps = [min(pl, key=len) for pl in rows.values()]
    reps.sort(key=len)
    return rows, reps

def build_delta(reps_sorted):
    idx_of = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    n = len(reps_sorted)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_sorted):
        for a in range(6):
            sp = rep + str(a)
            if all(sp+s in results for s in E):
                delta[i][a] = idx_of[row_of(sp)]
    return delta

def partner_issues(delta):
    n = len(delta)
    non_mutual, missing = [], []
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if j is None:
                missing.append((i, a))
            else:
                if all(delta[j][b] != i for b in range(6)):
                    non_mutual.append((i, a, j))
    return non_mutual, missing

def probes_for_gaps(reps_sorted, delta, non_mutual, missing):
    need = set()
    for (i, a) in missing:
        sp = reps_sorted[i] + str(a)
        for s in E:
            if sp + s not in results:
                need.add(sp + s)
    for (i, a, j) in non_mutual:
        for b in range(6):
            sp = reps_sorted[j] + str(b)
            for s in E:
                if sp + s not in results:
                    need.add(sp + s)
    return list(need)

def build_map(reps_sorted, delta):
    idx_of = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    rooms = [row_of(rep)[0] for rep in reps_sorted]
    start_idx = idx_of[row_of("")]
    seen = set(); connections = []
    for i in range(len(reps_sorted)):
        for a in range(6):
            j = delta[i][a]
            if j is None: continue
            for b in range(6):
                if delta[j][b] == i:
                    edge = tuple(sorted([(i, a), (j, b)]))
                    if edge not in seen:
                        seen.add(edge)
                        connections.append({
                            "from": {"room": i, "door": a},
                            "to":   {"room": j, "door": b},
                        })
                    break
    return {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

# ---- Closure with progress guard + dynamic E -------------------------------
# Prime the table for current E
rows, reps = rows_and_reps()
ensure_rows_for([r for r in reps] + [r + d for r in reps for d in ALPHA])

def score(delta):
    n = len(delta)
    filled = sum(v is not None for row in delta for v in row)
    miss = n*6 - filled
    return (n, filled, miss)

rounds = 0
stagnant = 0
prev = (-1,-1,10**9)  # (states, filled, missing)
MAX_ROUNDS = 25
MAX_E_LEVEL = 3

while rounds < MAX_ROUNDS:
    rows, reps = rows_and_reps()
    delta = build_delta(reps)
    non_mutual, missing = partner_issues(delta)
    cur = score(delta)
    print(f"Round {rounds}: states={cur[0]} filled={cur[1]} missing={cur[2]} non_mutual={len(non_mutual)} E_level={E_level}")

    # closed?
    if not non_mutual and not missing:
        final_map = build_map(reps, delta)
        print("\n=== MAP (primus) ===")
        print(json.dumps(final_map, indent=2))
        # Optionally submit:
        # resp = api("/guess", {"id": TEAM_ID, "map": final_map}); print("Correct?", resp["correct"])
        break

    # compute probes and send
    probes = probes_for_gaps(reps, delta, non_mutual, missing)
    if probes:
        explore_plans(probes)
    else:
        stagnant += 1

    # progress check
    made_progress = (cur[0] > prev[0]) or (cur[1] > prev[1]) or (cur[2] < prev[2])
    if not made_progress:
        stagnant += 1
    else:
        stagnant = 0
    prev = cur
    rounds += 1

    # If we’re stagnating, expand E (at most to Σ², then Σ³)
    if stagnant >= 2 and E_level < MAX_E_LEVEL:
        E_level += 1
        E = make_suffixes(E_level)
        stagnant = 0
        print(f"\n>>> Expanding suffix set to level {E_level} (|E|={len(E)}) and re-probing reps and exits …")
        # Ensure all row probes for reps and their one-step successors exist under new E
        need = []
        for rep in reps:
            need += [rep + s for s in E if rep + s not in results]
            for a in ALPHA:
                sp = rep + a
                need += [sp + s for s in E if sp + s not in results]
        explore_plans(list(set(need)))

else:
    print("\nStopped due to round cap / lack of progress.")
    print("Try increasing MAX_ROUNDS or allow E_level 3 if still needed.")


In [ ]:
# End-to-end PRIMUS solver (no infinite loops): select→collect (capped)→close table→build map.
import json, collections, requests, math

# ---- Config you can tweak --------------------------------------------------
BASE      = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID   = "tarstars@gmail.com yFD_tf-9rd1LJxZsYIBOeg"   # keep inline; we keep last token only
ALPHA     = "012345"
DEPTH     = 7          # BFS depth cap (safe & fast). Raise to 8–9 if needed.
CHUNK     = 200        # /explore batch size (server-friendly)
MAX_PLANS = 8000       # hard cap on total plan probes (prevents hangs)
MAX_ROUNDS= 25         # closure rounds cap
MAX_E_LVL = 3          # Σ -> Σ² -> Σ³ at most

# ---- Helpers ---------------------------------------------------------------
TEAM_ID = TEAM_ID.split()[-1]  # keep only the token if "email token" provided

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    if not r.ok:
        raise RuntimeError(f"{path} {r.status_code}: {r.text}")
    return r.json()

def try_select(problem):
    # Try with provided id; if fails, auto-register and retry once.
    try:
        return api("/select", {"id": TEAM_ID, "problemName": problem}, timeout=30)
    except Exception as e:
        msg = str(e)
        if "/select 400" in msg and "couldn't find matching record" in msg:
            print("ID invalid; auto-registering…")
            reg = requests.post(f"{BASE}/register",
                                json={"name":"tarstars","email":"tarstars@gmail.com","pl":"python"},
                                timeout=30)
            reg.raise_for_status()
            globals()["TEAM_ID"] = reg.json()["id"]
            print("New TEAM_ID:", TEAM_ID)
            return api("/select", {"id": TEAM_ID, "problemName": problem}, timeout=30)
        raise

def explore(plans):
    return api("/explore", {"id": TEAM_ID, "plans": plans})

# ---- Observation store -----------------------------------------------------
results = {}           # plan -> [labels], masked to 2 bits
total_probed = 0       # probe counter to enforce MAX_PLANS

def explore_plans(plans, chunk=CHUNK):
    global total_probed
    todo = [p for p in plans if p not in results]
    if not todo: return
    if total_probed >= MAX_PLANS:
        return
    # respect MAX_PLANS cap
    room = MAX_PLANS - total_probed
    if len(todo) > room:
        todo = todo[:room]
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        resp  = explore(batch)
        for p, seq in zip(batch, resp["results"]):
            results[p] = [v & 3 for v in seq]
        total_probed += len(batch)
        print(f"  fetched {len(batch)} (queryCount={resp.get('queryCount')}) | total={total_probed}/{MAX_PLANS}")
        if total_probed >= MAX_PLANS:
            break

# ---- Suffix set (dynamic growth) ------------------------------------------
def suffixes(level):
    if level == 1:
        return [""] + list(ALPHA)
    if level == 2:
        return [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]
    return ([""] + list(ALPHA)
            + [a+b for a in ALPHA for b in ALPHA]
            + [a+b+c for a in ALPHA for b in ALPHA for c in ALPHA])

E_level = 1
E = suffixes(E_level)

def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

def ensure_rows_for(prefixes):
    need = []
    for p in prefixes:
        for s in E:
            q = p + s
            if q not in results:
                need.append(q)
    explore_plans(need)

def rows_and_reps():
    rows = {}
    for p in list(results):
        ok = True
        for s in E:
            if p + s not in results:
                ok = False; break
        if ok:
            rows.setdefault(row_of(p), []).append(p)
    reps = [min(pl, key=len) for pl in rows.values()]
    reps.sort(key=len)
    return rows, reps

def build_delta(reps_sorted):
    idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    n = len(reps_sorted)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_sorted):
        for a in range(6):
            sp = rep + str(a)
            # need all sp+E present
            missing = [sp+s for s in E if sp+s not in results]
            if missing:
                continue
            delta[i][a] = idx[row_of(sp)]
    return delta

def partner_issues(delta):
    n = len(delta)
    non_mutual, missing = [], []
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if j is None:
                missing.append((i, a))
            else:
                if all(delta[j][b] != i for b in range(6)):
                    non_mutual.append((i, a, j))
    return non_mutual, missing

def probes_for_gaps(reps_sorted, delta, non_mutual, missing):
    need = set()
    for (i, a) in missing:
        sp = reps_sorted[i] + str(a)
        for s in E:
            q = sp + s
            if q not in results:
                need.add(q)
    for (i, a, j) in non_mutual:
        # probe the whole exit set of j
        for b in range(6):
            sp = reps_sorted[j] + str(b)
            for s in E:
                q = sp + s
                if q not in results:
                    need.add(q)
    return list(need)

def build_map(reps_sorted, delta):
    idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    rooms = [row_of(rep)[0] for rep in reps_sorted]
    start_idx = idx[row_of("")]
    seen = set(); connections = []
    for i in range(len(reps_sorted)):
        for a in range(6):
            j = delta[i][a]
            if j is None: continue
            for b in range(6):
                if delta[j][b] == i:
                    edge = tuple(sorted([(i, a), (j, b)]))
                    if edge not in seen:
                        seen.add(edge)
                        connections.append({
                            "from": {"room": i, "door": a},
                            "to":   {"room": j, "door": b},
                        })
                    break
    return {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

def score(delta):
    n = len(delta)
    filled = sum(v is not None for row in delta for v in row)
    miss = n*6 - filled
    return (n, filled, miss)

# ---- Run: select → collect (capped) ---------------------------------------
print("Selecting primus…")
print(try_select("primus"))

# seed empty plan and its row
explore_plans([""])
ensure_rows_for([""])

# modest BFS with caps
q = collections.deque([""])
seen = set([""])
while q:
    p = q.popleft()
    if len(p) >= DEPTH or total_probed >= MAX_PLANS:
        continue
    # expand successors and their rows in batches
    succs = [p + a for a in ALPHA]
    explore_plans(succs)
    ensure_rows_for(succs)
    for sp in succs:
        if sp not in seen:
            seen.add(sp); q.append(sp)
    if total_probed >= MAX_PLANS:
        break

print(f"\nCollected: {len(results)} plans (cap {MAX_PLANS})")

# ---- Close table with progress guard + dynamic E --------------------------
rows, reps = rows_and_reps()
ensure_rows_for([r for r in reps] + [r + d for r in reps for d in ALPHA])

prev = (-1, -1, 10**9)
stagnant = 0
rounds = 0

while rounds < MAX_ROUNDS and total_probed < MAX_PLANS:
    rows, reps = rows_and_reps()
    delta = build_delta(reps)
    non_mutual, missing = partner_issues(delta)
    cur = score(delta)
    print(f"Round {rounds}: states={cur[0]} filled={cur[1]} missing={cur[2]} non_mutual={len(non_mutual)} E_level={E_level} total={total_probed}")

    # done?
    if not non_mutual and not missing and cur[0] > 0:
        final_map = build_map(reps, delta)
        print("\n=== MAP (primus) ===")
        print(json.dumps(final_map, indent=2))
        # Optional submit:
        # resp = api("/guess", {"id": TEAM_ID, "map": final_map}); print("Correct?", resp["correct"])
        break

    # probe what’s needed
    probes = probes_for_gaps(reps, delta, non_mutual, missing)
    if probes:
        explore_plans(probes)
    else:
        stagnant += 1

    # progress check
    made_progress = (cur[0] > prev[0]) or (cur[1] > prev[1]) or (cur[2] < prev[2])
    stagnant = 0 if made_progress else (stagnant + 1)
    prev = cur
    rounds += 1

    # expand E if stagnating
    if stagnant >= 2 and E_level < MAX_E_LVL:
        E_level += 1
        E = suffixes(E_level)
        stagnant = 0
        print(f">>> Expanding E to level {E_level} (|E|={len(E)}) & re-probing reps/exits")
        need = []
        for rep in reps:
            need += [rep + s for s in E if rep + s not in results]
            for a in ALPHA:
                sp = rep + a
                need += [sp + s for s in E if sp + s not in results]
        explore_plans(list(set(need)))

else:
    print("\nStopped (cap hit or no progress).")
    rows, reps = rows_and_reps()
    delta = build_delta(reps)
    non_mutual, missing = partner_issues(delta)
    print(f"Final status: states={len(reps)} filled={sum(v is not None for row in delta for v in row)} "
          f"missing={len(missing)} non_mutual={len(non_mutual)} total={total_probed}")


Selecting primus…
ID invalid; auto-registering…
New TEAM_ID: tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g
{'problemName': 'primus'}
  fetched 1 (queryCount=2) | total=1/8000
  fetched 6 (queryCount=9) | total=7/8000
  fetched 36 (queryCount=46) | total=43/8000
  fetched 36 (queryCount=83) | total=79/8000
  fetched 36 (queryCount=120) | total=115/8000
  fetched 36 (queryCount=157) | total=151/8000
  fetched 36 (queryCount=194) | total=187/8000
  fetched 36 (queryCount=231) | total=223/8000
  fetched 36 (queryCount=268) | total=259/8000
  fetched 36 (queryCount=305) | total=295/8000
  fetched 36 (queryCount=342) | total=331/8000
  fetched 36 (queryCount=379) | total=367/8000
  fetched 36 (queryCount=416) | total=403/8000
  fetched 36 (queryCount=453) | total=439/8000
  fetched 36 (queryCount=490) | total=475/8000
  fetched 36 (queryCount=527) | total=511/8000
  fetched 36 (queryCount=564) | total=547/8000
  fetched 36 (queryCount=601) | total=583/8000
  fetched 36 (queryCount=638) | total=6

In [ ]:
# Build & (optionally) submit the final map from the data already in memory.
import json, requests

# Reuse suffix set E if it exists; otherwise fall back to the minimal one.
try:
    E  # if defined in your previous cell
except NameError:
    ALPHA = "012345"
    E = [""] + list(ALPHA)

def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

def rows_and_reps():
    rows = {}
    for p in list(results):
        if all(p + s in results for s in E):
            rows.setdefault(row_of(p), []).append(p)
    reps = [min(pl, key=len) for pl in rows.values()]
    reps.sort(key=len)
    return rows, reps

def build_delta(reps_sorted):
    idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    n = len(reps_sorted)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_sorted):
        for a in range(6):
            sp = rep + str(a)
            if all(sp + s in results for s in E):
                delta[i][a] = idx[row_of(sp)]
    return delta

def build_map(reps_sorted, delta):
    idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    rooms = [row_of(rep)[0] for rep in reps_sorted]
    start_idx = idx[row_of("")]
    seen = set(); connections = []
    for i in range(len(reps_sorted)):
        for a in range(6):
            j = delta[i][a]
            if j is None:
                continue
            # pair only mutual edges
            for b in range(6):
                if delta[j][b] == i:
                    edge = tuple(sorted([(i, a), (j, b)]))
                    if edge not in seen:
                        seen.add(edge)
                        connections.append({
                            "from": {"room": i, "door": a},
                            "to":   {"room": j, "door": b},
                        })
                    break
    return {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

rows, reps_sorted = rows_and_reps()
delta = build_delta(reps_sorted)
final_map = build_map(reps_sorted, delta)

print("States:", len(reps_sorted))
print("\n=== MAP (primus) ===")
print(json.dumps(final_map, indent=2))

# Optional: submit the guess
# resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID.split()[-1], "map": final_map}, timeout=60)
# print("\nCorrect?", resp.json().get("correct"))


States: 6

=== MAP (primus) ===
{
  "rooms": [
    0,
    3,
    1,
    0,
    1,
    2
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 3,
        "door": 3
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 4,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 5
      },
      "to": {
        "room": 3,
        "door": 3
      }
    },
    {
      "from": {
        "room": 1,
        "door": 0
    

In [ ]:
# Validate & submit
import requests, json

# If your map is in `final_map`, keep using it; otherwise paste it here:
# final_map = {...}

def validate_map(m):
    # 36 ports total (6 rooms × 6 doors), must be paired into 18 edges
    conns = m["connections"]
    assert len(conns) == 18, f"Expected 18 connections, got {len(conns)}"

    ports = set()
    for e in conns:
        a = (e["from"]["room"], e["from"]["door"])
        b = (e["to"]["room"],   e["to"]["door"])
        # doors in 0..5
        assert 0 <= a[1] < 6 and 0 <= b[1] < 6, "Door out of range"
        # no self-pairing of the same port
        assert a != b, "Edge pairs a port with itself"
        # each port appears exactly once
        assert a not in ports and b not in ports, "Port appears more than once"
        ports.add(a); ports.add(b)

    # every room has 6 distinct doors
    by_room = {i: set() for i in range(len(m["rooms"]))}
    for (r,d) in ports:
        by_room[r].add(d)
    for r, ds in by_room.items():
        assert ds == set(range(6)), f"Room {r} missing doors: {set(range(6)) - ds}"
    print("Validation OK ✅")

validate_map(final_map)

# Submit
resp = requests.post(
    f"{BASE}/guess",
    json={"id": TEAM_ID.split()[-1], "map": final_map},
    timeout=60
)
print("Server response:", resp.status_code, resp.text)


AssertionError: Expected 18 connections, got 26

In [ ]:
# Build a valid connection set: each port used exactly once.
def build_connections_strict(reps_sorted, delta):
    n = len(reps_sorted)
    used = set()  # set of (room, door) already assigned
    conns = []

    # Precompute candidate back-doors for each (i,a)
    # forward: (i,a) -> (j, [b's s.t. delta[j][b]==i])
    candidates = {}
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if j is None:
                continue
            backs = [b for b in range(6) if delta[j][b] == i]
            candidates[(i, a)] = (j, backs)

    # Greedy pass: if a port has exactly one available partner, lock it in first
    changed = True
    while changed:
        changed = False
        for (i, a), (j, backs) in list(candidates.items()):
            if (i, a) in used:
                continue
            # filter out already-used partner ports
            avail = [b for b in backs if (j, b) not in used]
            if not avail:
                continue
            if len(avail) == 1:
                b = avail[0]
                conns.append({
                    "from": {"room": i, "door": a},
                    "to":   {"room": j, "door": b},
                })
                used.add((i, a)); used.add((j, b))
                changed = True

    # Second pass: assign any remaining unassigned ports arbitrarily among their available partners
    for (i, a), (j, backs) in candidates.items():
        if (i, a) in used:
            continue
        avail = [b for b in backs if (j, b) not in used]
        if not avail:
            # No free back-port: this *shouldn’t* happen for a closed table;
            # if it does, your E wasn’t strong enough — expand E and rebuild delta.
            continue
        b = avail[0]
        conns.append({
            "from": {"room": i, "door": a},
            "to":   {"room": j, "door": b},
        })
        used.add((i, a)); used.add((j, b))

    return conns

# Rebuild map with strict matching
rows, reps_sorted = rows_and_reps()
delta = build_delta(reps_sorted)

final_map = {
    "rooms": [row_of(rep)[0] for rep in reps_sorted],
    "startingRoom": {row_of(rep): i for i, rep in enumerate(reps_sorted)}[row_of("")],
    "connections": build_connections_strict(reps_sorted, delta),
}

print("Connections:", len(final_map["connections"]))
print(json.dumps(final_map, indent=2))


Connections: 21
{
  "rooms": [
    0,
    3,
    1,
    0,
    1,
    2
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 4,
        "door": 2
      }
    },
    {
      "from": {
        "room": 1,
        "door": 1
      },
      "to": {
        "room": 3,
        "door": 1
      }
    },
    {
      "from": {
        "room": 1,
        "door": 5
      },
      "to": {
        "room": 5,
        "door": 2
      }
    },
    {
      "from": {
        "room": 2,
        "door": 0
      },
      "to": {
        "room": 3,
        "door": 2
      }
    },
    {
      "from": {
        "room": 2,
        "door": 3
      },
      "to": {
        "room": 2,
        "door": 3
      }
    },
    {
      "from": {
        "room": 4,
        "door": 3
      },
      "to":

In [ ]:
# --- Perfect matching construction of connections (allows self-loops) ---

# 1) Reuse helpers/vars from earlier cell (reps_sorted, delta, row_of, etc.)
# If needed, recompute:
def rows_and_reps():
    rows = {}
    for p in list(results):
        if all(p + s in results for s in E):
            rows.setdefault(tuple(results[p+s][len(p+s)] for s in E), []).append(p)
    reps = [min(pl, key=len) for pl in rows.values()]
    reps.sort(key=len)
    return rows, reps

def build_delta(reps_sorted):
    idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
    n = len(reps_sorted)
    d = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_sorted):
        for a in range(6):
            sp = rep + str(a)
            if all(sp+s in results for s in E):
                d[i][a] = idx[row_of(sp)]
    return d

# Ensure we have reps_sorted & delta
rows, reps_sorted = rows_and_reps()
delta = build_delta(reps_sorted)

# 2) Build bipartite graph between "left" ports and "right" ports.
# Left nodes are tuples (i,a); right nodes are tuples (j,b).
left_ports  = [(i,a) for i in range(len(reps_sorted)) for a in range(6)]
right_ports = [(i,a) for i in range(len(reps_sorted)) for a in range(6)]
right_index = {p: k for k, p in enumerate(right_ports)}

# Adjacency: for each left port (i,a), which right ports (j,b) are valid partners?
adj = [[] for _ in left_ports]
for li, (i,a) in enumerate(left_ports):
    j = delta[i][a]
    if j is None:
        continue
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        rnode = right_index[(j,b)]
        adj[li].append(rnode)

# 3) Hopcroft–Karp to find a perfect matching on 36 ports
from collections import deque

N = len(left_ports)
M = len(right_ports)
INF = 10**9

pairU = [-1]*N  # left -> right
pairV = [-1]*M  # right -> left
dist  = [0]*N

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u] = 0
            q.append(u)
        else:
            dist[u] = INF
    found_free = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu != -1 and dist[pu] == INF:
                dist[pu] = dist[u] + 1
                q.append(pu)
            if pu == -1:
                found_free = True
    return found_free

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu] == dist[u] + 1 and dfs(pu)):
            pairU[u] = v
            pairV[v] = u
            return True
    dist[u] = INF
    return False

matching = 0
while bfs():
    for u in range(N):
        if pairU[u] == -1 and dfs(u):
            matching += 1

assert matching == N, f"Matching incomplete: matched {matching} of {N} ports"

# 4) Build the 18 undirected connections from the matching.
# Each edge is determined by a left port u matched to right port v.
# To avoid duplicating edges, canonicalize with sorted((i,a),(j,b)).
edges = set()
connections = []
for u in range(N):
    v = pairU[u]
    i,a = left_ports[u]
    j,b = right_ports[v]
    # sanity: must be mutual
    assert delta[i][a] == j and delta[j][b] == i
    edge = tuple(sorted([(i,a),(j,b)]))
    if edge not in edges:
        edges.add(edge)
        connections.append({
            "from": {"room": i, "door": a},
            "to":   {"room": j, "door": b},
        })

print("Connections built:", len(connections))  # should be 18

final_map = {
    "rooms": [row_of(rep)[0] for rep in reps_sorted],
    "startingRoom": {row_of(rep): i for i, rep in enumerate(reps_sorted)}[row_of("")],
    "connections": connections,
}

import json
print(json.dumps(final_map, indent=2))

# 5) Validator that accepts self-loops (spec allows same door).
def validate_map(m):
    # must be exactly 18 edges for 6 rooms
    assert len(m["connections"]) == 18, f"Expected 18 connections, got {len(m['connections'])}"
    seen = set()  # each port should appear in exactly one connection
    for e in m["connections"]:
        a = (e["from"]["room"], e["from"]["door"])
        b = (e["to"]["room"],   e["to"]["door"])
        assert 0 <= a[1] < 6 and 0 <= b[1] < 6
        # Each port must participate in exactly one edge (loops use the same port twice, but still one edge)
        assert a not in seen and b not in seen, "Port used in multiple edges"
        seen.add(a); seen.add(b)
    # every room has all 6 doors present
    from collections import defaultdict
    per = defaultdict(set)
    for (r,d) in seen:
        per[r].add(d)
    for r in range(len(m["rooms"])):
        assert per[r] == set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print("Validation OK ✅")

validate_map(final_map)

# Optional: submit
# import requests
# resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID.split()[-1], "map": final_map}, timeout=60)
# print("Server response:", resp.status_code, resp.text)


Connections built: 21
{
  "rooms": [
    0,
    3,
    1,
    0,
    1,
    2
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 3,
        "door": 3
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 4,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 2,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 5
      },
      "to": {
        "room": 3,
        "door": 5
      }
    },
    {
      "from": {
        "room": 1,
        "door": 0
      },
     

AssertionError: Expected 18 connections, got 21

In [ ]:
# ==== OFFLINE ANALYZER for PRIMUS (no HTTP) =================================
# Assumes you have `results` in memory (plan -> list[int]) from your last run.
# If not, change LOAD_PATH to point to your saved dump.

import json
from collections import deque, defaultdict

# 0) Snapshot current results (if present) then reload purely offline
SAVE_PATH = "/content/primus_results.json"
try:
    results  # from earlier online run
    with open(SAVE_PATH, "w") as f:
        json.dump(results, f)
    print(f"Saved snapshot to {SAVE_PATH} (plans: {len(results)})")
except NameError:
    print("No in-memory `results` found; will try to load from disk.")

with open(SAVE_PATH) as f:
    results = json.load(f)
print("Loaded results (offline):", len(results), "plans")

ALPHA = "012345"

def make_suffixes(level=1):
    if level == 1:
        return [""] + list(ALPHA)                          # Σ^0 ∪ Σ^1
    if level == 2:
        return [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]  # Σ^0..Σ^2
    # feel free to add Σ^3 if needed
    return ([""] + list(ALPHA)
            + [a+b for a in ALPHA for b in ALPHA]
            + [a+b+c for a in ALPHA for b in ALPHA for c in ALPHA])

# Choose how strong you want E:
E = make_suffixes(level=2)   # try Σ^2 first; bump to 3 if ambiguities remain

def row_of(prefix):
    # row across E at the *end* positions of each (prefix + suffix)
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

def has_row(prefix):
    # do we know all (prefix + E) plans?
    return all(prefix + s in results for s in E)

# 1) Build rows → representatives
rows = defaultdict(list)
for p in results.keys():
    if has_row(p):
        rows[row_of(p)].append(p)

reps_sorted = [min(plist, key=len) for plist in rows.values()]
reps_sorted.sort(key=len)
idx_of_row = {row_of(r): i for i, r in enumerate(reps_sorted)}

print(f"States (unique rows): {len(reps_sorted)}")
print("Shortest reps:", [f"'{r}'" for r in reps_sorted[:6]])

# 2) Build delta where possible
n = len(reps_sorted)
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps_sorted):
    for a in range(6):
        sp = rep + str(a)
        if has_row(sp):
            delta[i][a] = idx_of_row[row_of(sp)]

filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# 3) Partner candidates and ambiguity report
ambiguous = []     # (i,a,j, [b candidates])
missing   = []     # (i,a)
for i in range(n):
    for a in range(6):
        j = delta[i][a]
        if j is None:
            missing.append((i,a)); continue
        backs = [b for b in range(6) if delta[j][b] == i]
        if len(backs) == 0:
            # shouldn't happen if non_mutual=0 online; indicates E isn't aligned with that run
            print(f"WARNING: non-mutual at (i={i},a={a}) -> j={j}")
        elif len(backs) > 1:
            ambiguous.append((i,a,j,backs))

print(f"Missing: {len(missing)}  |  Ambiguous ports: {len(ambiguous)}")
if ambiguous:
    print("Examples (up to 10):")
    for tup in ambiguous[:10]:
        i,a,j,backs = tup
        print(f"  (room {i}, door {a}) -> room {j} has back-doors {backs}")

# 4) Perfect matching on ports (works even if some ports have multiple candidates)
left_ports  = [(i,a) for i in range(n) for a in range(6)]
right_ports = [(i,a) for i in range(n) for a in range(6)]
right_idx   = {p:k for k,p in enumerate(right_ports)}

adj = [[] for _ in left_ports]
for u,(i,a) in enumerate(left_ports):
    j = delta[i][a]
    if j is None: continue
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        adj[u].append(right_idx[(j,b)])

# Hopcroft–Karp
INF = 10**9
pairU = [-1]*len(left_ports)
pairV = [-1]*len(right_ports)
dist  = [0]*len(left_ports)

from collections import deque
def bfs():
    q = deque()
    for u in range(len(left_ports)):
        if pairU[u] == -1:
            dist[u]=0; q.append(u)
        else:
            dist[u]=INF
    reachable = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1:
                reachable = True
            elif dist[pu]==INF:
                dist[pu]=dist[u]+1; q.append(pu)
    return reachable

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu]==dist[u]+1 and dfs(pu)):
            pairU[u]=v; pairV[v]=u
            return True
    dist[u]=INF
    return False

matching=0
while bfs():
    for u in range(len(left_ports)):
        if pairU[u]==-1 and dfs(u):
            matching += 1

print(f"Port matching size: {matching} (expect {6*n})")
assert matching == 6*n, "No perfect matching — E likely too weak or table incomplete."

# 5) Emit each undirected edge exactly once (self-loops allowed)
edges=set(); connections=[]
for u,v in enumerate(pairU):
    i,a = left_ports[u]
    j,b = right_ports[v]
    # sanity
    assert delta[i][a]==j and delta[j][b]==i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({
            "from":{"room":i,"door":a},
            "to":{"room":j,"door":b},
        })

print("Connections built:", len(connections))  # should be 18

# 6) Build final map
def room_label(rep): return row_of(rep)[0]  # 2-bit label
rooms = [room_label(rep) for rep in reps_sorted]
start_idx = idx_of_row[row_of("")]

final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print(json.dumps(final_map, indent=2))

# 7) Strict validation (accepts self-loops; ensures ports used once)
def validate_map(m):
    assert len(m["connections"]) == 18, f"Need 18 connections, got {len(m['connections'])}"
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used and b not in used, "Port used more than once"
        used.add(a); used.add(b)
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(len(m["rooms"])):
        assert per[r]==set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print("Validation OK ✅")

validate_map(final_map)

# 8) If you want to see which ports were ambiguous and how to disambiguate:
def suggest_probe_suffixes(limit_per_amb=6, max_len=3):
    # purely offline suggestion: for each ambiguous (i,a)->j with back-doors b1,b2,
    # propose short t in Σ^k (k<=max_len) that *we haven't probed yet* for j+b.
    # Running those online later tends to split the ambiguity.
    cand_suffixes = [""]
    for L in range(1, max_len+1):
        new = []
        for s in cand_suffixes:
            if len(s)==L: continue
            for d in ALPHA:
                ns = s + d
                if len(ns)==L: new.append(ns)
        cand_suffixes += new
    cand_suffixes = [s for s in cand_suffixes if s!=""]  # non-empty
    plans=set()
    for (i,a,j,backs) in ambiguous:
        repj = reps_sorted[j]
        for b in backs:
            base = repj + str(b)
            for t in cand_suffixes:
                q = base + t
                if q not in results:
                    plans.add(q)
                if len(plans) >= limit_per_amb*len(ambiguous):
                    break
            if len(plans) >= limit_per_amb*len(ambiguous):
                break
    return sorted(plans)

if ambiguous:
    print("\nSuggested extra probes to disambiguate (strings only, do later):")
    print(suggest_probe_suffixes(limit_per_amb=4, max_len=3)[:40])


Saved snapshot to /content/primus_results.json (plans: 8000)
Loaded results (offline): 8000 plans
States (unique rows): 6
Shortest reps: ["''", "'0'", "'1'", "'2'", "'3'", "'05'"]
Filled transitions: 36/36
Missing: 0  |  Ambiguous ports: 22
Examples (up to 10):
  (room 0, door 1) -> room 2 has back-doors [1, 2]
  (room 0, door 2) -> room 3 has back-doors [3, 5]
  (room 0, door 4) -> room 2 has back-doors [1, 2]
  (room 0, door 5) -> room 3 has back-doors [3, 5]
  (room 1, door 0) -> room 4 has back-doors [0, 1, 5]
  (room 1, door 3) -> room 4 has back-doors [0, 1, 5]
  (room 1, door 4) -> room 4 has back-doors [0, 1, 5]
  (room 2, door 1) -> room 0 has back-doors [1, 4]
  (room 2, door 2) -> room 0 has back-doors [1, 4]
  (room 2, door 4) -> room 5 has back-doors [0, 4]
Port matching size: 36 (expect 36)
Connections built: 21
{
  "rooms": [
    0,
    3,
    1,
    0,
    1,
    2
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
  

AssertionError: Need 18 connections, got 21

In [ ]:
from google.colab import files
files.download("/content/primus_results.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# PRIMUS: offline→focused probes→perfect pairing→final map (+optional submit)
import json, requests
from collections import defaultdict, deque
from pathlib import Path

# ================== CONFIG ==================
BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
DO_SUBMIT = False                     # set True to POST /guess after building map
MAX_NEW_PLANS = 1200                  # safety cap for extra probes
CHUNK = 200                           # /explore batch size
# ===========================================

# TEAM_ID = TEAM_ID.split()[-1] if TEAM_ID.strip() else TEAM_ID

# ---- Load snapshot (offline baseline) ----
snap_candidates = [Path("/content/primus_results.json"), Path("/mnt/data/primus_results.json")]
snap_path = next((p for p in snap_candidates if p.exists()), None)
assert snap_path is not None, "No snapshot found at /content/primus_results.json or /mnt/data/primus_results.json"
with open(snap_path, "r") as f:
    results = json.load(f)
print(f"Loaded snapshot: {len(results)} plans from {snap_path}")

ALPHA = "012345"

def make_suffixes(level=2):
    if level == 1:
        return [""] + list(ALPHA)
    if level == 2:
        return [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]
    return ([""] + list(ALPHA)
            + [a+b for a in ALPHA for b in ALPHA]
            + [a+b+c for a in ALPHA for b in ALPHA for c in ALPHA])

# Use *one* E consistently. Start with Σ^2; we can bump to Σ^3 if still ambiguous.
E = make_suffixes(level=2)

def has_row(p): return all((p + s) in results for s in E)
def row_of(prefix): return tuple(results[p][len(p)] for p in (prefix + s for s in E))

def build_states_and_delta():
    rows = defaultdict(list)
    for p in results.keys():
        if has_row(p):
            rows[row_of(p)].append(p)
    reps_sorted = [min(pl, key=len) for pl in rows.values()]
    reps_sorted.sort(key=len)
    idx = {row_of(r): i for i, r in enumerate(reps_sorted)}
    n = len(reps_sorted)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_sorted):
        for a in range(6):
            sp = rep + str(a)
            if has_row(sp):
                delta[i][a] = idx[row_of(sp)]
    return reps_sorted, idx, delta

def analyze_ambiguity(delta):
    n = len(delta)
    ambiguous, missing = [], []
    for i in range(n):
        for a in range(6):
            j = delta[i][a]
            if j is None:
                missing.append((i,a)); continue
            backs = [b for b in range(6) if delta[j][b] == i]
            if len(backs) > 1:
                ambiguous.append((i,a,j,backs))
    return ambiguous, missing

# ---- API helpers ----
def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    if not r.ok:
        raise RuntimeError(f"{path} {r.status_code}: {r.text}")
    return r.json()

def try_select_primus():
    global TEAM_ID
    try:
        return api("/select", {"id": TEAM_ID, "problemName": "primus"}, timeout=30)
    except Exception as e:
        msg = str(e)
        if "/select 400" in msg and "couldn't find matching record" in msg:
            print("TEAM_ID invalid; registering a new one …")
            reg = requests.post(f"{BASE}/register", json={"name":"tarstars","email":"tarstars@gmail.com","pl":"python"}, timeout=30)
            reg.raise_for_status()
            TEAM_ID = reg.json()["id"]
            print("New TEAM_ID:", TEAM_ID)
            return api("/select", {"id": TEAM_ID, "problemName": "primus"}, timeout=30)
        raise

def explore_plans(plans):
    todo = [p for p in plans if p not in results]
    if not todo:
        return 0
    fetched = 0
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        resp = api("/explore", {"id": TEAM_ID, "plans": batch})
        for p, seq in zip(batch, resp["results"]):
            results[p] = [v & 3 for v in seq]  # mask to 2 bits (server already 0..3; harmless)
        fetched += len(batch)
        print(f"  fetched {len(batch)} (queryCount={resp.get('queryCount')})")
    return fetched

# ---- Step 1: Build delta and see ambiguity offline ----
reps_sorted, idx, delta = build_states_and_delta()
n = len(reps_sorted)
filled = sum(v is not None for row in delta for v in row)
print(f"States: {n}, filled transitions: {filled}/{6*n}")
ambiguous, missing = analyze_ambiguity(delta)
print(f"Ambiguous ports: {len(ambiguous)}, missing: {len(missing)}")
if ambiguous[:6]:
    print("Examples:", ambiguous[:6])

# ---- Step 2: Generate *focused* probe plans for just the ambiguous targets ----
# For each ambiguous (i,a)->j with back-doors b's, probe rep_j + b + t for t in small suffixes.
small_T = [""] + list(ALPHA)    # lengths 0..1
probe = set()
for (i,a,j,backs) in ambiguous:
    repj = reps_sorted[j]
    for b in backs:
        base = repj + str(b)
        for t in small_T:
            q = base + t
            if q not in results:
                probe.add(q)
probe = list(probe)
probe.sort()
if not probe:
    print("No extra probes needed from snapshot (nice!).")
else:
    print(f"Planned extra probes: {len(probe)} (cap {MAX_NEW_PLANS})")

# ---- Step 3: Hit /explore with a small capped set of probes ----
if probe:
    try_select_primus()
    if len(probe) > MAX_NEW_PLANS:
        probe = probe[:MAX_NEW_PLANS]
        print(f"Trimmed probe list to {len(probe)}")
    fetched = explore_plans(probe)
    print("Fetched new plans:", fetched)
    # persist snapshot
    with open("/content/primus_results.json", "w") as f:
        json.dump(results, f)
    print("Saved updated snapshot to /content/primus_results.json")

# ---- Step 4: Rebuild delta (same E) and try to pair ports perfectly ----
reps_sorted, idx, delta = build_states_and_delta()
n = len(reps_sorted)
filled = sum(v is not None for row in delta for v in row)
print(f"After probes → states={n}, filled={filled}/{6*n}")

# Build mutual candidate graph between ports (only mutual pairs)
ports = [(i,a) for i in range(n) for a in range(6)]
right_index = {p:k for k,p in enumerate(ports)}
adj = [[] for _ in ports]  # bipartite-style: left index u -> right index v

for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    if j is None:
        continue
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        v = right_index[(j,b)]
        adj[u].append(v)

# Hopcroft–Karp (left=N=36, right=N=36)
N = len(ports)
pairU = [-1]*N
pairV = [-1]*N
dist = [0]*N
INF = 10**9

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u] = 0
            q.append(u)
        else:
            dist[u] = INF
    found = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1:
                found = True
            elif dist[pu] == INF:
                dist[pu] = dist[u] + 1
                q.append(pu)
    return found

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu] == dist[u] + 1 and dfs(pu)):
            pairU[u] = v
            pairV[v] = u
            return True
    dist[u] = INF
    return False

matching = 0
while bfs():
    for u in range(N):
        if pairU[u] == -1 and dfs(u):
            matching += 1

print(f"Port matching size: {matching} (expect {N})")

if matching != N:
    # If still not perfect, try strengthening E to Σ^3 *offline* and retry matching,
    # without making more HTTP calls. (Sometimes longer rows separate candidates.)
    print("Matching not perfect with Σ^2 rows — retrying with Σ^3 rows offline …")
    E = make_suffixes(level=3)
    reps_sorted, idx, delta = build_states_and_delta()
    ports = [(i,a) for i in range(n) for a in range(6)]
    right_index = {p:k for k,p in enumerate(ports)}
    adj = [[] for _ in ports]
    for u,(i,a) in enumerate(ports):
        j = delta[i][a]
        if j is None: continue
        backs = [b for b in range(6) if delta[j][b] == i]
        for b in backs:
            adj[u].append(right_index[(j,b)])
    pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N
    matching = 0
    while bfs():
        for u in range(N):
            if pairU[u] == -1 and dfs(u):
                matching += 1
    print(f"Port matching (Σ^3 rows): {matching} (expect {N})")

assert matching == N, "Still ambiguous after focused probes; increase MAX_NEW_PLANS or allow Σ^3 online."

# Emit each undirected edge once (unique) from the bijection
edges = set()
connections = []
for u in range(N):
    v = pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    # must be mutual by construction
    assert delta[i][a] == j and delta[j][b] == i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({
            "from": {"room": i, "door": a},
            "to":   {"room": j, "door": b},
        })

print("Connections:", len(connections))  # should be 18

# Build rooms + startingRoom from consistent E/rows
def room_label(rep): return row_of(rep)[0]
rooms = [room_label(rep) for rep in reps_sorted]
start_idx = idx[row_of("")]

final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== MAP (primus) ===")
print(json.dumps(final_map, indent=2))

# Validate (self-loops allowed; each port used once)
def validate_map(m):
    assert len(m["connections"]) == 18, f"Need 18 connections, got {len(m['connections'])}"
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0 <= a[1] < 6 and 0 <= b[1] < 6
        assert a not in used and b not in used, "Port used more than once"
        used.add(a); used.add(b)
    per=defaultdict(set)
    for (r,d) in used:
        per[r].add(d)
    for r in range(len(m["rooms"])):
        assert per[r] == set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print("Validation OK ✅")

validate_map(final_map)

# Optional submission
if DO_SUBMIT:
    resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
    print("Guess response:", resp.status_code, resp.text)


Loaded snapshot: 8000 plans from /content/primus_results.json
States: 6, filled transitions: 36/36
Ambiguous ports: 22, missing: 0
Examples: [(0, 1, 2, [1, 2]), (0, 2, 3, [3, 5]), (0, 4, 2, [1, 2]), (0, 5, 3, [3, 5]), (1, 0, 4, [0, 1, 5]), (1, 3, 4, [0, 1, 5])]
No extra probes needed from snapshot (nice!).
After probes → states=6, filled=36/36
Port matching size: 36 (expect 36)
Connections: 21

=== MAP (primus) ===
{
  "rooms": [
    0,
    3,
    1,
    0,
    1,
    2
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 3,
        "door": 3
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to"

AssertionError: Need 18 connections, got 21

In [ ]:
# --- Correct validation & optional submit for primus map ---

from collections import defaultdict
import requests, json

def validate_map_general(m):
    used = set()
    for e in m["connections"]:
        a = (e["from"]["room"], e["from"]["door"])
        b = (e["to"]["room"],   e["to"]["door"])
        assert 0 <= a[1] < 6 and 0 <= b[1] < 6, "Door out of range"
        assert a not in used, f"Port reused: {a}"
        assert b not in used, f"Port reused: {b}"
        used.add(a); used.add(b)
    # must cover all 36 ports
    n_rooms = len(m["rooms"])
    assert n_rooms == 6, f"Expected 6 rooms, got {n_rooms}"
    assert len(used) == 36, f"Used {len(used)} ports, expected 36"
    # each room has all 6 doors covered
    per = defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n_rooms):
        assert per[r] == set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅  (connections={len(m['connections'])})")

# Run validation on the final map you already built
validate_map_general(final_map)

# --- Optional: submit to server ---
# Make sure TEAM_ID is set to your active ID string
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
print("Server response:", resp.status_code, resp.text)


Validation OK ✅  (connections=21)
Server response: 200 {"correct":true}


In [ ]:
TEAM_ID

'tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g'

In [ ]:
# === Full Probatio Submit ===

import json, requests
from collections import defaultdict

BASE = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID string

# 1) Select probatio
r = requests.post(f"{BASE}/select", json={"id": TEAM_ID, "problemName": "probatio"}, timeout=30)
r.raise_for_status()
print("Selected probatio:", r.json())

# 2) Explore the empty plan just to get labels
resp = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": [""]}, timeout=30)
resp.raise_for_status()
data = resp.json()
print("QueryCount:", data.get("queryCount"))
print("Results:", data["results"])

# 3) The map for probatio is trivial (1 room, 6 self-loops)
rooms = [data["results"][0][0] & 3]   # 2-bit label of the single room
final_map = {
    "rooms": rooms,
    "startingRoom": 0,
    "connections": [
        {"from": {"room": 0, "door": d}, "to": {"room": 0, "door": d}}
        for d in range(6)
    ]
}

print(json.dumps(final_map, indent=2))

# 4) Submit
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
print("Server response:", resp.status_code, resp.text)


Selected probatio: {'problemName': 'probatio'}
QueryCount: 2
Results: [[0]]
{
  "rooms": [
    0
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 0,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 0,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 0,
        "door": 3
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 0,
        "door": 4
      }
    },
    {
      "from": {
        "room": 0,
        "door": 5
      },
      "to": {
        "room": 0,
        "door": 5
      }
    }
  ]
}
Server response: 200 {"correct":false}


In [ ]:
# === Discover & Submit PROBATIO (3-room) ===

import json, requests
from collections import defaultdict, deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID

ALPHA = "012345"
E     = [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]  # Σ^2 rows

def api(path, payload, timeout=60):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

print("Selecting probatio…")
api("/select", {"id": TEAM_ID, "problemName": "probatio"}, timeout=30)

# --- Explore helper with caching ---
results = {}  # plan -> list of ints (server already 0..3; &3 harmless)
def explore(plans, chunk=200):
    todo = [p for p in plans if p not in results]
    if not todo: return 0
    fetched = 0
    for i in range(0, len(todo), chunk):
        batch = todo[i:i+chunk]
        data  = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=90)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        fetched += len(batch)
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")
    return fetched

def has_row(p): return all(p + s in results for s in E)
def row_of(prefix): return tuple(results[p][len(p)] for p in (prefix + s for s in E))

# --- Seed + small frontier expansion until we see 3 distinct rows and close δ ---
print("Bootstrapping…")
explore([""])  # seed

rows = {}
rows[row_of("")] = ""
reps = [""]

# keep expanding short prefixes until we have 3 states (probatio) and all 18 transitions filled
while True:
    succs = [rep + a for rep in reps for a in ALPHA]
    explore(succs)
    new = False
    for rep in list(reps):
        for a in ALPHA:
            sp = rep + a
            if has_row(sp):
                r = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    reps.append(sp)
                    new = True
    if len(rows) >= 3 and not new:
        break

# Representatives (shortest per row), sorted
reps_sorted = [rows[r] for r in rows]
reps_sorted.sort(key=len)
idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
print("States discovered:", len(reps_sorted), "Reps:", reps_sorted)

# Ensure all rep+a rows exist
needs = [rep + a for rep in reps_sorted for a in ALPHA]
explore(needs)

# Build delta
n = len(reps_sorted)
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps_sorted):
    for a in range(6):
        sp = rep + str(a)
        assert has_row(sp), f"missing row for {sp}"
        delta[i][a] = idx[row_of(sp)]

filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# Build mutual candidate graph between ports (i,a) ↔ (j,b) where δ[i][a]=j and δ[j][b]=i
ports = [(i,a) for i in range(n) for a in range(6)]
pos   = {p:k for k,p in enumerate(ports)}
adj   = [[] for _ in ports]  # bipartite formulation (left->right)

for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        adj[u].append(pos[(j,b)])

# Hopcroft–Karp perfect matching on 36×36 (enforces each port used once)
from collections import deque
N = len(ports)
pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N; INF = 10**9

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u] = 0; q.append(u)
        else:
            dist[u] = INF
    found = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1:
                found = True
            elif dist[pu] == INF:
                dist[pu] = dist[u] + 1; q.append(pu)
    return found

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu] == dist[u]+1 and dfs(pu)):
            pairU[u] = v; pairV[v] = u
            return True
    dist[u] = INF
    return False

matching = 0
while bfs():
    for u in range(N):
        if pairU[u] == -1 and dfs(u):
            matching += 1

print(f"Port matching size: {matching}/{N}")
assert matching == N, "Could not find a full one-to-one pairing of ports."

# Emit each undirected edge once (self-loops allowed)
edges = set(); connections = []
for u in range(N):
    v = pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    # sanity: mutual
    assert delta[i][a] == j and delta[j][b] == i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from": {"room": i, "door": a}, "to": {"room": j, "door": b}})

print("Connections:", len(connections))  # could be >18 if there are self-loops

# Build rooms (2-bit labels) and startingRoom
def room_label(rep): return row_of(rep)[0]
rooms = [room_label(rep) for rep in reps_sorted]
start_idx = idx[row_of("")]
final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== MAP (probatio) ===")
print(json.dumps(final_map, indent=2))

# Correct validation: cover all 36 ports once; allow self-loops
def validate_map_general(m):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],  e["to"]["door"])
        assert 0 <= a[1] < 6 and 0 <= b[1] < 6
        assert a not in used, f"Port reused: {a}"
        assert b not in used, f"Port reused: {b}"
        used.add(a); used.add(b)
    assert len(used) == 6*n, f"Covered {len(used)} ports, expected {6*n}"
    per = defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        assert per[r] == set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅  (rooms={n}, connections={len(m['connections'])})")

validate_map_general(final_map)

# Submit
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
print("Server response:", resp.status_code, resp.text)


Selecting probatio…
Bootstrapping…
  fetched 1 (queryCount=2) | total=1


KeyError: '0'

In [ ]:
# === PROBATIO: discover → build → validate → submit (fixed row seeding) ===
import json, requests
from collections import defaultdict, deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID string

ALPHA = "012345"
E     = [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]  # Σ^2 rows (plenty for probatio)
CHUNK = 200

def api(path, payload, timeout=60):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return 0
    fetched = 0
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data  = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=90)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # mask to 2 bits (harmless)
        fetched += len(batch)
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")
    return fetched

def ensure_rows_for(prefixes):
    need = []
    for p in prefixes:
        for s in E:
            q = p + s
            if q not in results:
                need.append(q)
    if need:
        explore(need)

def has_row(p):  # after ensure_rows_for, this should be true
    return all(p + s in results for s in E)

def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

print("Selecting probatio…")
api("/select", {"id": TEAM_ID, "problemName": "probatio"}, timeout=30)

# --- Seed & guarantee base row exists ---
results = {}
explore([""])
ensure_rows_for([""])           # make "" + E exist
assert has_row(""), "Failed to seed base row"

# --- Discover states (probatio has 3) ---
rows = {}
rows[row_of("")] = ""
reps = [""]

while True:
    succs = [rep + a for rep in reps for a in ALPHA]
    explore(succs)
    ensure_rows_for(succs)
    new_found = False
    for rep in list(reps):
        for a in ALPHA:
            sp = rep + a
            if has_row(sp):
                r = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    reps.append(sp)
                    new_found = True
    if len(rows) >= 3 and not new_found:
        break

# Representatives (shortest per row)
reps_sorted = [rows[r] for r in rows]
reps_sorted.sort(key=len)
idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
n = len(reps_sorted)
print("States discovered:", n, "Reps:", reps_sorted)

# Ensure all rep+a rows exist then build delta
needs = [rep + a for rep in reps_sorted for a in ALPHA]
ensure_rows_for(needs)

delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps_sorted):
    for a in range(6):
        sp = rep + str(a)
        assert has_row(sp), f"missing row for {sp}"
        delta[i][a] = idx[row_of(sp)]

filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# --- Pair ports with a one-to-one matching (self-loops allowed) ---
ports = [(i,a) for i in range(n) for a in range(6)]
right_index = {p:k for k,p in enumerate(ports)}
adj = [[] for _ in ports]  # left (u) -> right (v) where mutual

for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        adj[u].append(right_index[(j,b)])

# Hopcroft–Karp on 36×36
from collections import deque
N = len(ports)
pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N; INF = 10**9

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u] = 0; q.append(u)
        else:
            dist[u] = INF
    found = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1:
                found = True
            elif dist[pu] == INF:
                dist[pu] = dist[u] + 1; q.append(pu)
    return found

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu] == dist[u] + 1 and dfs(pu)):
            pairU[u] = v; pairV[v] = u
            return True
    dist[u] = INF
    return False

matching = 0
while bfs():
    for u in range(N):
        if pairU[u] == -1 and dfs(u):
            matching += 1
print(f"Port matching size: {matching}/{N}")
assert matching == N, "Could not find a full pairing of ports."

# Emit each undirected edge once (loops allowed)
edges=set(); connections=[]
for u in range(N):
    v = pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    # sanity: mutual
    assert delta[i][a] == j and delta[j][b] == i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

print("Connections:", len(connections))

# Build rooms + starting
rooms = [row_of(rep)[0] for rep in reps_sorted]
start_idx = idx[row_of("")]
final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== MAP (probatio) ===")
print(json.dumps(final_map, indent=2))

# --- Validate (cover all 6*n ports; allow self-loops) ---
def validate_map_general(m):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used, f"Port reused: {a}"
        assert b not in used, f"Port reused: {b}"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        assert per[r]==set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

validate_map_general(final_map)

# --- Submit ---
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
print("Server response:", resp.status_code, resp.text)


Selecting probatio…
  fetched 1 (queryCount=2) | total=1
  fetched 42 (queryCount=45) | total=43
  fetched 200 (queryCount=246) | total=243
  fetched 16 (queryCount=263) | total=259
  fetched 200 (queryCount=464) | total=459
  fetched 200 (queryCount=665) | total=659
  fetched 32 (queryCount=698) | total=691
States discovered: 3 Reps: ['', '0', '2']
Filled transitions: 18/18
Port matching size: 18/18
Connections: 11

=== MAP (probatio) ===
{
  "rooms": [
    0,
    1,
    2
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 3
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      

In [ ]:
# === PROBATIO: discover → build → validate → submit (fixed row seeding) ===
import json, requests
from collections import defaultdict, deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID string

ALPHA = "012345"
E     = [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]  # Σ^2 rows (plenty for probatio)
CHUNK = 200

def api(path, payload, timeout=60):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return 0
    fetched = 0
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data  = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=90)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # mask to 2 bits (harmless)
        fetched += len(batch)
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")
    return fetched

def ensure_rows_for(prefixes):
    need = []
    for p in prefixes:
        for s in E:
            q = p + s
            if q not in results:
                need.append(q)
    if need:
        explore(need)

def has_row(p):  # after ensure_rows_for, this should be true
    return all(p + s in results for s in E)

def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

print("Selecting secundus")
api("/select", {"id": TEAM_ID, "problemName": "secundus"}, timeout=30)

# --- Seed & guarantee base row exists ---
results = {}
explore([""])
ensure_rows_for([""])           # make "" + E exist
assert has_row(""), "Failed to seed base row"

# --- Discover states (probatio has 3) ---
rows = {}
rows[row_of("")] = ""
reps = [""]

while True:
    succs = [rep + a for rep in reps for a in ALPHA]
    explore(succs)
    ensure_rows_for(succs)
    new_found = False
    for rep in list(reps):
        for a in ALPHA:
            sp = rep + a
            if has_row(sp):
                r = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    reps.append(sp)
                    new_found = True
    if len(rows) >= 3 and not new_found:
        break

# Representatives (shortest per row)
reps_sorted = [rows[r] for r in rows]
reps_sorted.sort(key=len)
idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
n = len(reps_sorted)
print("States discovered:", n, "Reps:", reps_sorted)

# Ensure all rep+a rows exist then build delta
needs = [rep + a for rep in reps_sorted for a in ALPHA]
ensure_rows_for(needs)

delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps_sorted):
    for a in range(6):
        sp = rep + str(a)
        assert has_row(sp), f"missing row for {sp}"
        delta[i][a] = idx[row_of(sp)]

filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# --- Pair ports with a one-to-one matching (self-loops allowed) ---
ports = [(i,a) for i in range(n) for a in range(6)]
right_index = {p:k for k,p in enumerate(ports)}
adj = [[] for _ in ports]  # left (u) -> right (v) where mutual

for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        adj[u].append(right_index[(j,b)])

# Hopcroft–Karp on 36×36
from collections import deque
N = len(ports)
pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N; INF = 10**9

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u] = 0; q.append(u)
        else:
            dist[u] = INF
    found = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1:
                found = True
            elif dist[pu] == INF:
                dist[pu] = dist[u] + 1; q.append(pu)
    return found

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu] == dist[u] + 1 and dfs(pu)):
            pairU[u] = v; pairV[v] = u
            return True
    dist[u] = INF
    return False

matching = 0
while bfs():
    for u in range(N):
        if pairU[u] == -1 and dfs(u):
            matching += 1
print(f"Port matching size: {matching}/{N}")
assert matching == N, "Could not find a full pairing of ports."

# Emit each undirected edge once (loops allowed)
edges=set(); connections=[]
for u in range(N):
    v = pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    # sanity: mutual
    assert delta[i][a] == j and delta[j][b] == i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

print("Connections:", len(connections))

# Build rooms + starting
rooms = [row_of(rep)[0] for rep in reps_sorted]
start_idx = idx[row_of("")]
final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== MAP (probatio) ===")
print(json.dumps(final_map, indent=2))

# --- Validate (cover all 6*n ports; allow self-loops) ---
def validate_map_general(m):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used, f"Port reused: {a}"
        assert b not in used, f"Port reused: {b}"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        assert per[r]==set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

validate_map_general(final_map)

# --- Submit ---
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
print("Server response:", resp.status_code, resp.text)


Selecting secundus
  fetched 1 (queryCount=2) | total=1
  fetched 42 (queryCount=45) | total=43
  fetched 200 (queryCount=246) | total=243
  fetched 16 (queryCount=263) | total=259
  fetched 200 (queryCount=464) | total=459
  fetched 200 (queryCount=665) | total=659
  fetched 200 (queryCount=866) | total=859
  fetched 200 (queryCount=1067) | total=1059
  fetched 64 (queryCount=1132) | total=1123
  fetched 200 (queryCount=1333) | total=1323
  fetched 200 (queryCount=1534) | total=1523
  fetched 200 (queryCount=1735) | total=1723
  fetched 200 (queryCount=1936) | total=1923
  fetched 200 (queryCount=2137) | total=2123
  fetched 200 (queryCount=2338) | total=2323
  fetched 96 (queryCount=2435) | total=2419
  fetched 200 (queryCount=2636) | total=2619
  fetched 16 (queryCount=2653) | total=2635
States discovered: 12 Reps: ['', '0', '1', '3', '5', '00', '02', '10', '11', '12', '13', '023']
Filled transitions: 72/72
Port matching size: 72/72
Connections: 41

=== MAP (probatio) ===
{
  "rooms

In [ ]:
# === PROBATIO: discover → build → validate → submit (fixed row seeding) ===
import json, requests
from collections import defaultdict, deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID string

ALPHA = "012345"
E     = [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]  # Σ^2 rows (plenty for probatio)
CHUNK = 200

def api(path, payload, timeout=60):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return 0
    fetched = 0
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data  = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=90)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # mask to 2 bits (harmless)
        fetched += len(batch)
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")
    return fetched

def ensure_rows_for(prefixes):
    need = []
    for p in prefixes:
        for s in E:
            q = p + s
            if q not in results:
                need.append(q)
    if need:
        explore(need)

def has_row(p):  # after ensure_rows_for, this should be true
    return all(p + s in results for s in E)

def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

print("Selecting tertius")
api("/select", {"id": TEAM_ID, "problemName": "tertius"}, timeout=30)

# --- Seed & guarantee base row exists ---
results = {}
explore([""])
ensure_rows_for([""])           # make "" + E exist
assert has_row(""), "Failed to seed base row"

# --- Discover states (probatio has 3) ---
rows = {}
rows[row_of("")] = ""
reps = [""]

while True:
    succs = [rep + a for rep in reps for a in ALPHA]
    explore(succs)
    ensure_rows_for(succs)
    new_found = False
    for rep in list(reps):
        for a in ALPHA:
            sp = rep + a
            if has_row(sp):
                r = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    reps.append(sp)
                    new_found = True
    if len(rows) >= 3 and not new_found:
        break

# Representatives (shortest per row)
reps_sorted = [rows[r] for r in rows]
reps_sorted.sort(key=len)
idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
n = len(reps_sorted)
print("States discovered:", n, "Reps:", reps_sorted)

# Ensure all rep+a rows exist then build delta
needs = [rep + a for rep in reps_sorted for a in ALPHA]
ensure_rows_for(needs)

delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps_sorted):
    for a in range(6):
        sp = rep + str(a)
        assert has_row(sp), f"missing row for {sp}"
        delta[i][a] = idx[row_of(sp)]

filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# --- Pair ports with a one-to-one matching (self-loops allowed) ---
ports = [(i,a) for i in range(n) for a in range(6)]
right_index = {p:k for k,p in enumerate(ports)}
adj = [[] for _ in ports]  # left (u) -> right (v) where mutual

for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        adj[u].append(right_index[(j,b)])

# Hopcroft–Karp on 36×36
from collections import deque
N = len(ports)
pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N; INF = 10**9

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u] = 0; q.append(u)
        else:
            dist[u] = INF
    found = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1:
                found = True
            elif dist[pu] == INF:
                dist[pu] = dist[u] + 1; q.append(pu)
    return found

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu] == dist[u] + 1 and dfs(pu)):
            pairU[u] = v; pairV[v] = u
            return True
    dist[u] = INF
    return False

matching = 0
while bfs():
    for u in range(N):
        if pairU[u] == -1 and dfs(u):
            matching += 1
print(f"Port matching size: {matching}/{N}")
assert matching == N, "Could not find a full pairing of ports."

# Emit each undirected edge once (loops allowed)
edges=set(); connections=[]
for u in range(N):
    v = pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    # sanity: mutual
    assert delta[i][a] == j and delta[j][b] == i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

print("Connections:", len(connections))

# Build rooms + starting
rooms = [row_of(rep)[0] for rep in reps_sorted]
start_idx = idx[row_of("")]
final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== MAP (probatio) ===")
print(json.dumps(final_map, indent=2))

# --- Validate (cover all 6*n ports; allow self-loops) ---
def validate_map_general(m):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used, f"Port reused: {a}"
        assert b not in used, f"Port reused: {b}"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        assert per[r]==set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

validate_map_general(final_map)

# --- Submit ---
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
print("Server response:", resp.status_code, resp.text)


Selecting tertius
  fetched 1 (queryCount=2) | total=1
  fetched 42 (queryCount=45) | total=43
  fetched 200 (queryCount=246) | total=243
  fetched 16 (queryCount=263) | total=259
  fetched 200 (queryCount=464) | total=459
  fetched 200 (queryCount=665) | total=659
  fetched 200 (queryCount=866) | total=859
  fetched 200 (queryCount=1067) | total=1059
  fetched 200 (queryCount=1268) | total=1259
  fetched 200 (queryCount=1469) | total=1459
  fetched 96 (queryCount=1566) | total=1555
  fetched 200 (queryCount=1767) | total=1755
  fetched 200 (queryCount=1968) | total=1955
  fetched 200 (queryCount=2169) | total=2155
  fetched 200 (queryCount=2370) | total=2355
  fetched 200 (queryCount=2571) | total=2555
  fetched 200 (queryCount=2772) | total=2755
  fetched 200 (queryCount=2973) | total=2955
  fetched 200 (queryCount=3174) | total=3155
  fetched 200 (queryCount=3375) | total=3355
  fetched 200 (queryCount=3576) | total=3555
  fetched 200 (queryCount=3777) | total=3755
  fetched 176 (qu

In [ ]:
# === PROBATIO: discover → build → validate → submit (fixed row seeding) ===
import json, requests
from collections import defaultdict, deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID string

ALPHA = "012345"
E     = [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]  # Σ^2 rows (plenty for probatio)
CHUNK = 200

def api(path, payload, timeout=60):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return 0
    fetched = 0
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data  = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=90)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # mask to 2 bits (harmless)
        fetched += len(batch)
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")
    return fetched

def ensure_rows_for(prefixes):
    need = []
    for p in prefixes:
        for s in E:
            q = p + s
            if q not in results:
                need.append(q)
    if need:
        explore(need)

def has_row(p):  # after ensure_rows_for, this should be true
    return all(p + s in results for s in E)

def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

print("Selecting quartus")
api("/select", {"id": TEAM_ID, "problemName": "quartus"}, timeout=30)

# --- Seed & guarantee base row exists ---
results = {}
explore([""])
ensure_rows_for([""])           # make "" + E exist
assert has_row(""), "Failed to seed base row"

# --- Discover states (probatio has 3) ---
rows = {}
rows[row_of("")] = ""
reps = [""]

while True:
    succs = [rep + a for rep in reps for a in ALPHA]
    explore(succs)
    ensure_rows_for(succs)
    new_found = False
    for rep in list(reps):
        for a in ALPHA:
            sp = rep + a
            if has_row(sp):
                r = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    reps.append(sp)
                    new_found = True
    if len(rows) >= 3 and not new_found:
        break

# Representatives (shortest per row)
reps_sorted = [rows[r] for r in rows]
reps_sorted.sort(key=len)
idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
n = len(reps_sorted)
print("States discovered:", n, "Reps:", reps_sorted)

# Ensure all rep+a rows exist then build delta
needs = [rep + a for rep in reps_sorted for a in ALPHA]
ensure_rows_for(needs)

delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps_sorted):
    for a in range(6):
        sp = rep + str(a)
        assert has_row(sp), f"missing row for {sp}"
        delta[i][a] = idx[row_of(sp)]

filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# --- Pair ports with a one-to-one matching (self-loops allowed) ---
ports = [(i,a) for i in range(n) for a in range(6)]
right_index = {p:k for k,p in enumerate(ports)}
adj = [[] for _ in ports]  # left (u) -> right (v) where mutual

for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        adj[u].append(right_index[(j,b)])

# Hopcroft–Karp on 36×36
from collections import deque
N = len(ports)
pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N; INF = 10**9

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u] = 0; q.append(u)
        else:
            dist[u] = INF
    found = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1:
                found = True
            elif dist[pu] == INF:
                dist[pu] = dist[u] + 1; q.append(pu)
    return found

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu] == dist[u] + 1 and dfs(pu)):
            pairU[u] = v; pairV[v] = u
            return True
    dist[u] = INF
    return False

matching = 0
while bfs():
    for u in range(N):
        if pairU[u] == -1 and dfs(u):
            matching += 1
print(f"Port matching size: {matching}/{N}")
assert matching == N, "Could not find a full pairing of ports."

# Emit each undirected edge once (loops allowed)
edges=set(); connections=[]
for u in range(N):
    v = pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    # sanity: mutual
    assert delta[i][a] == j and delta[j][b] == i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

print("Connections:", len(connections))

# Build rooms + starting
rooms = [row_of(rep)[0] for rep in reps_sorted]
start_idx = idx[row_of("")]
final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== MAP (probatio) ===")
print(json.dumps(final_map, indent=2))

# --- Validate (cover all 6*n ports; allow self-loops) ---
def validate_map_general(m):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used, f"Port reused: {a}"
        assert b not in used, f"Port reused: {b}"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        assert per[r]==set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

validate_map_general(final_map)

# --- Submit ---
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
print("Server response:", resp.status_code, resp.text)


Selecting quartus
  fetched 1 (queryCount=2) | total=1
  fetched 42 (queryCount=45) | total=43
  fetched 200 (queryCount=246) | total=243
  fetched 16 (queryCount=263) | total=259
  fetched 200 (queryCount=464) | total=459
  fetched 200 (queryCount=665) | total=659
  fetched 200 (queryCount=866) | total=859
  fetched 200 (queryCount=1067) | total=1059
  fetched 200 (queryCount=1268) | total=1259
  fetched 200 (queryCount=1469) | total=1459
  fetched 96 (queryCount=1566) | total=1555
  fetched 200 (queryCount=1767) | total=1755
  fetched 200 (queryCount=1968) | total=1955
  fetched 200 (queryCount=2169) | total=2155
  fetched 200 (queryCount=2370) | total=2355
  fetched 200 (queryCount=2571) | total=2555
  fetched 200 (queryCount=2772) | total=2755
  fetched 200 (queryCount=2973) | total=2955
  fetched 200 (queryCount=3174) | total=3155
  fetched 200 (queryCount=3375) | total=3355
  fetched 200 (queryCount=3576) | total=3555
  fetched 200 (queryCount=3777) | total=3755
  fetched 200 (qu

In [ ]:
# === PROBATIO: discover → build → validate → submit (fixed row seeding) ===
import json, requests
from collections import defaultdict, deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID string

ALPHA = "012345"
E     = [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]  # Σ^2 rows (plenty for probatio)
CHUNK = 200

def api(path, payload, timeout=60):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return 0
    fetched = 0
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data  = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=90)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # mask to 2 bits (harmless)
        fetched += len(batch)
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")
    return fetched

def ensure_rows_for(prefixes):
    need = []
    for p in prefixes:
        for s in E:
            q = p + s
            if q not in results:
                need.append(q)
    if need:
        explore(need)

def has_row(p):  # after ensure_rows_for, this should be true
    return all(p + s in results for s in E)

def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

print("Selecting quintus")
api("/select", {"id": TEAM_ID, "problemName": "quintus"}, timeout=30)

# --- Seed & guarantee base row exists ---
results = {}
explore([""])
ensure_rows_for([""])           # make "" + E exist
assert has_row(""), "Failed to seed base row"

# --- Discover states (probatio has 3) ---
rows = {}
rows[row_of("")] = ""
reps = [""]

while True:
    succs = [rep + a for rep in reps for a in ALPHA]
    explore(succs)
    ensure_rows_for(succs)
    new_found = False
    for rep in list(reps):
        for a in ALPHA:
            sp = rep + a
            if has_row(sp):
                r = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    reps.append(sp)
                    new_found = True
    if len(rows) >= 3 and not new_found:
        break

# Representatives (shortest per row)
reps_sorted = [rows[r] for r in rows]
reps_sorted.sort(key=len)
idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
n = len(reps_sorted)
print("States discovered:", n, "Reps:", reps_sorted)

# Ensure all rep+a rows exist then build delta
needs = [rep + a for rep in reps_sorted for a in ALPHA]
ensure_rows_for(needs)

delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps_sorted):
    for a in range(6):
        sp = rep + str(a)
        assert has_row(sp), f"missing row for {sp}"
        delta[i][a] = idx[row_of(sp)]

filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# --- Pair ports with a one-to-one matching (self-loops allowed) ---
ports = [(i,a) for i in range(n) for a in range(6)]
right_index = {p:k for k,p in enumerate(ports)}
adj = [[] for _ in ports]  # left (u) -> right (v) where mutual

for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        adj[u].append(right_index[(j,b)])

# Hopcroft–Karp on 36×36
from collections import deque
N = len(ports)
pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N; INF = 10**9

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u] = 0; q.append(u)
        else:
            dist[u] = INF
    found = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1:
                found = True
            elif dist[pu] == INF:
                dist[pu] = dist[u] + 1; q.append(pu)
    return found

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu] == dist[u] + 1 and dfs(pu)):
            pairU[u] = v; pairV[v] = u
            return True
    dist[u] = INF
    return False

matching = 0
while bfs():
    for u in range(N):
        if pairU[u] == -1 and dfs(u):
            matching += 1
print(f"Port matching size: {matching}/{N}")
assert matching == N, "Could not find a full pairing of ports."

# Emit each undirected edge once (loops allowed)
edges=set(); connections=[]
for u in range(N):
    v = pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    # sanity: mutual
    assert delta[i][a] == j and delta[j][b] == i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

print("Connections:", len(connections))

# Build rooms + starting
rooms = [row_of(rep)[0] for rep in reps_sorted]
start_idx = idx[row_of("")]
final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== MAP (probatio) ===")
print(json.dumps(final_map, indent=2))

# --- Validate (cover all 6*n ports; allow self-loops) ---
def validate_map_general(m):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used, f"Port reused: {a}"
        assert b not in used, f"Port reused: {b}"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        assert per[r]==set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

validate_map_general(final_map)

# --- Submit ---
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
print("Server response:", resp.status_code, resp.text)


Selecting quintus
  fetched 1 (queryCount=2) | total=1
  fetched 42 (queryCount=45) | total=43
  fetched 200 (queryCount=246) | total=243
  fetched 16 (queryCount=263) | total=259
  fetched 200 (queryCount=464) | total=459
  fetched 200 (queryCount=665) | total=659
  fetched 200 (queryCount=866) | total=859
  fetched 200 (queryCount=1067) | total=1059
  fetched 200 (queryCount=1268) | total=1259
  fetched 80 (queryCount=1349) | total=1339
  fetched 200 (queryCount=1550) | total=1539
  fetched 200 (queryCount=1751) | total=1739
  fetched 200 (queryCount=1952) | total=1939
  fetched 200 (queryCount=2153) | total=2139
  fetched 200 (queryCount=2354) | total=2339
  fetched 200 (queryCount=2555) | total=2539
  fetched 200 (queryCount=2756) | total=2739
  fetched 200 (queryCount=2957) | total=2939
  fetched 200 (queryCount=3158) | total=3139
  fetched 200 (queryCount=3359) | total=3339
  fetched 200 (queryCount=3560) | total=3539
  fetched 200 (queryCount=3761) | total=3739
  fetched 200 (qu

In [ ]:
# === PROBATIO: discover → build → validate → submit (fixed row seeding) ===
import json, requests
from collections import defaultdict, deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID string

ALPHA = "012345"
E     = [""] + list(ALPHA) + [a+b for a in ALPHA for b in ALPHA]  # Σ^2 rows (plenty for probatio)
CHUNK = 200

def api(path, payload, timeout=60):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    if not todo: return 0
    fetched = 0
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data  = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=90)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # mask to 2 bits (harmless)
        fetched += len(batch)
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")
    return fetched

def ensure_rows_for(prefixes):
    need = []
    for p in prefixes:
        for s in E:
            q = p + s
            if q not in results:
                need.append(q)
    if need:
        explore(need)

def has_row(p):  # after ensure_rows_for, this should be true
    return all(p + s in results for s in E)

def row_of(prefix):
    return tuple(results[p][len(p)] for p in (prefix + s for s in E))

print("Selecting aleph")
api("/select", {"id": TEAM_ID, "problemName": "aleph"}, timeout=30)

# --- Seed & guarantee base row exists ---
results = {}
explore([""])
ensure_rows_for([""])           # make "" + E exist
assert has_row(""), "Failed to seed base row"

# --- Discover states (probatio has 3) ---
rows = {}
rows[row_of("")] = ""
reps = [""]

while True:
    succs = [rep + a for rep in reps for a in ALPHA]
    explore(succs)
    ensure_rows_for(succs)
    new_found = False
    for rep in list(reps):
        for a in ALPHA:
            sp = rep + a
            if has_row(sp):
                r = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    reps.append(sp)
                    new_found = True
    if len(rows) >= 3 and not new_found:
        break

# Representatives (shortest per row)
reps_sorted = [rows[r] for r in rows]
reps_sorted.sort(key=len)
idx = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
n = len(reps_sorted)
print("States discovered:", n, "Reps:", reps_sorted)

# Ensure all rep+a rows exist then build delta
needs = [rep + a for rep in reps_sorted for a in ALPHA]
ensure_rows_for(needs)

delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps_sorted):
    for a in range(6):
        sp = rep + str(a)
        assert has_row(sp), f"missing row for {sp}"
        delta[i][a] = idx[row_of(sp)]

filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# --- Pair ports with a one-to-one matching (self-loops allowed) ---
ports = [(i,a) for i in range(n) for a in range(6)]
right_index = {p:k for k,p in enumerate(ports)}
adj = [[] for _ in ports]  # left (u) -> right (v) where mutual

for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        adj[u].append(right_index[(j,b)])

# Hopcroft–Karp on 36×36
from collections import deque
N = len(ports)
pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N; INF = 10**9

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u] = 0; q.append(u)
        else:
            dist[u] = INF
    found = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1:
                found = True
            elif dist[pu] == INF:
                dist[pu] = dist[u] + 1; q.append(pu)
    return found

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu] == dist[u] + 1 and dfs(pu)):
            pairU[u] = v; pairV[v] = u
            return True
    dist[u] = INF
    return False

matching = 0
while bfs():
    for u in range(N):
        if pairU[u] == -1 and dfs(u):
            matching += 1
print(f"Port matching size: {matching}/{N}")
assert matching == N, "Could not find a full pairing of ports."

# Emit each undirected edge once (loops allowed)
edges=set(); connections=[]
for u in range(N):
    v = pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    # sanity: mutual
    assert delta[i][a] == j and delta[j][b] == i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

print("Connections:", len(connections))

# Build rooms + starting
rooms = [row_of(rep)[0] for rep in reps_sorted]
start_idx = idx[row_of("")]
final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== MAP (probatio) ===")
print(json.dumps(final_map, indent=2))

# --- Validate (cover all 6*n ports; allow self-loops) ---
def validate_map_general(m):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used, f"Port reused: {a}"
        assert b not in used, f"Port reused: {b}"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        assert per[r]==set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

validate_map_general(final_map)

# --- Submit ---
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
print("Server response:", resp.status_code, resp.text)


Selecting aleph
  fetched 1 (queryCount=2) | total=1
  fetched 42 (queryCount=45) | total=43
  fetched 200 (queryCount=246) | total=243
  fetched 16 (queryCount=263) | total=259
  fetched 200 (queryCount=464) | total=459
  fetched 16 (queryCount=481) | total=475
  fetched 200 (queryCount=682) | total=675
  fetched 16 (queryCount=699) | total=691
  fetched 200 (queryCount=900) | total=891
  fetched 200 (queryCount=1101) | total=1091
  fetched 32 (queryCount=1134) | total=1123
  fetched 200 (queryCount=1335) | total=1323
  fetched 16 (queryCount=1352) | total=1339
States discovered: 6 Reps: ['', '0', '02', '021', '022', '0220']
Filled transitions: 36/36
Port matching size: 36/36
Connections: 26

=== MAP (probatio) ===
{
  "rooms": [
    0,
    2,
    1,
    0,
    3,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0

In [ ]:
# === ALEPH solver with charcoal marks: discover → build → validate → submit ===
import json, requests
from collections import defaultdict, deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID string

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']  # charcoal actions
CHUNK = 200

# ----- token helpers -----
def serialize(tokens):  # tokens -> string for API
    return ''.join(tokens)

def is_door(tok): return tok in DOORS
def is_mark(tok): return tok in MARKS

def steps(tokens):
    """Door-steps + mark-steps count. Every token is one step."""
    return len(tokens)

# Distinguishing suffix set E: lightweight, includes marks
def make_E():
    E = [
        [],                         # empty
        *[[m] for m in MARKS],      # single mark
        *[[d] for d in DOORS],      # single door
    ]
    # short mark+door and door+mark combos help separate copies
    E += [[m,d] for m in MARKS for d in DOORS]
    E += [[d,m] for d in DOORS for m in MARKS]
    return E

E = make_E()

# ----- API -----
def api(path, payload, timeout=90):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def select(problem):
    print(f"Selecting {problem}…")
    return api("/select", {"id": TEAM_ID, "problemName": problem}, timeout=30)

# Explore cache: plan_str -> list[int]
results = {}

def explore_plans(plan_tokens_list):
    todo = []
    for toks in plan_tokens_list:
        s = serialize(toks)
        if s not in results:
            todo.append(s)
    if not todo: return 0
    fetched = 0
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # labels are 0..3; &3 is harmless
        fetched += len(batch)
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")
    return fetched

def ensure_rows_for(prefixes):
    need = []
    for pref in prefixes:
        for suf in E:
            plan = pref + suf
            if serialize(plan) not in results:
                need.append(plan)
    if need:
        explore_plans(need)

# Row signature with marks-aware indexing
def row_of(prefix):
    sig = []
    for suf in E:
        plan = prefix + suf
        s = serialize(plan)
        sig.append(results[s][steps(plan)])
    return tuple(sig)

def has_row(prefix):
    for suf in E:
        if serialize(prefix + suf) not in results:
            return False
    return True

# ----- 1) Select ALEPH -----
select("aleph")

# ----- 2) Seed: ensure base row "" exists with E -----
root = []  # empty token list
explore_plans([root])
ensure_rows_for([root])
assert has_row(root), "Failed to seed base row with E"

# ----- 3) Discover states (expect 12 for aleph; could be 18 for harder ones) -----
rows = {}
rows[row_of(root)] = root
frontier = [root]

# bounded breadth expansion (short prefixes with/without occasional marks)
def successors(toks):
    succ = []
    # door successors
    for d in DOORS:
        succ.append(toks + [d])
    # also try a local mark then a door occasionally (helps split copies)
    for m in MARKS:
        succ.append(toks + [m])
        for d in DOORS:
            succ.append(toks + [m, d])
    return succ

rounds = 0
while True:
    rounds += 1
    succs = []
    for rep in frontier:
        succs.extend(successors(rep))
    explore_plans(succs)
    ensure_rows_for(succs)

    new = []
    for rep in list(frontier):
        # inspect pure-door successors first for transitions; marks help only signatures
        for d in DOORS:
            sp = rep + [d]
            if has_row(sp):
                r = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    new.append(sp)
        # include marked successors as potential new reps (if they create new signatures)
        for m in MARKS:
            sp = rep + [m]
            if has_row(sp):
                r = row_of(sp)
                if r not in rows:
                    rows[r] = sp
                    new.append(sp)

    if not new:
        break
    frontier = new
    # safety: stop if we already have many rooms (12 or 18 are expected for duplicated copies)
    if len(rows) >= 18:
        break

reps_sorted = sorted(rows.values(), key=len)
idx_of = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
n = len(reps_sorted)
print(f"States discovered: {n}  Reps (shortest first): {reps_sorted[:8]}{' …' if n>8 else ''}")

# ----- 4) Build transition table δ using *door steps only* -----
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps_sorted):
    for a, d in enumerate(DOORS):
        sp = rep + [d]
        assert has_row(sp), f"missing row for {sp}"
        delta[i][a] = idx_of[row_of(sp)]

filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# ----- 5) Build mutual candidate graph between ports and do perfect matching -----
ports = [(i,a) for i in range(n) for a in range(6)]
pos   = {p:k for k,p in enumerate(ports)}
adj   = [[] for _ in ports]   # left (u) -> right (v) if δ[i][a]=j and δ[j][b]=i

for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        adj[u].append(pos[(j,b)])

# Hopcroft–Karp (36*n_left == 6*n)
from collections import deque
N = len(ports)
pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N; INF = 10**9

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u] = 0; q.append(u)
        else:
            dist[u] = INF
    found = False
    while q:
        u = q.popleft()
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1:
                found = True
            elif dist[pu] == INF:
                dist[pu] = dist[u] + 1; q.append(pu)
    return found

def dfs(u):
    for v in adj[u]:
        pu = pairV[v]
        if pu == -1 or (dist[pu] == dist[u] + 1 and dfs(pu)):
            pairU[u] = v; pairV[v] = u
            return True
    dist[u] = INF
    return False

matching = 0
while bfs():
    for u in range(N):
        if pairU[u] == -1 and dfs(u):
            matching += 1
print(f"Port matching size: {matching}/{N}")
assert matching == N, "Could not find a full one-to-one pairing of ports."

# ----- 6) Emit each undirected edge once (self-loops allowed) -----
edges=set(); connections=[]
for u in range(N):
    v = pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    # sanity: mutual
    assert delta[i][a] == j and delta[j][b] == i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

print(f"Connections: {len(connections)}")

# ----- 7) Room labels and startingRoom -----
# For consistency with the judge, the "rooms" array uses the label observed at the empty suffix.
def label_of(rep):
    s = serialize(rep)
    return results[s][steps(rep)]  # label *after* executing rep
rooms = [label_of(rep) for rep in reps_sorted]

# The starting room is the index of the row for the empty plan
start_idx = idx_of[row_of(root)]

final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== MAP (aleph) ===")
print(json.dumps(final_map, indent=2))

# ----- 8) Validate: cover all 6*n ports once; allow self-loops -----
def validate_map_general(m):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6, "Door out of range"
        assert a not in used, f"Port reused: {a}"
        assert b not in used, f"Port reused: {b}"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        assert per[r]==set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

validate_map_general(final_map)

# ----- 9) Submit -----
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=60)
print("Server response:", resp.status_code, resp.text)


Selecting aleph…
  fetched 1 (queryCount=2) | total=1
  fetched 58 (queryCount=61) | total=59
  fetched 200 (queryCount=262) | total=259
  fetched 200 (queryCount=463) | total=455
  fetched 200 (queryCount=664) | total=643
  fetched 200 (queryCount=865) | total=831
  fetched 200 (queryCount=1066) | total=1019
  fetched 200 (queryCount=1267) | total=1207
  fetched 200 (queryCount=1468) | total=1395
  fetched 200 (queryCount=1669) | total=1583
  fetched 200 (queryCount=1870) | total=1771
  fetched 124 (queryCount=1995) | total=1887
  fetched 200 (queryCount=2196) | total=2087
  fetched 200 (queryCount=2397) | total=2283
  fetched 200 (queryCount=2598) | total=2471
  fetched 200 (queryCount=2799) | total=2659
  fetched 200 (queryCount=3000) | total=2847
  fetched 200 (queryCount=3201) | total=3035
  fetched 200 (queryCount=3402) | total=3223
  fetched 200 (queryCount=3603) | total=3411
  fetched 200 (queryCount=3804) | total=3599
  fetched 200 (queryCount=4005) | total=3791
  fetched 200 

AssertionError: missing row for ['0', '0', '0']

In [ ]:
# === ALEPH solver (lazy row fill) ===
import json, requests
from collections import defaultdict, deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']
CHUNK = 200

# Distinguishing suffixes: small but mark-aware
def make_E():
    E = [[]]                                  # empty
    E += [[d] for d in DOORS]                 # 1 door
    E += [[m] for m in MARKS]                 # 1 mark
    E += [[m,d] for m in MARKS for d in DOORS]  # mark then door
    E += [[d,m] for d in DOORS for m in MARKS]  # door then mark
    return E

E = make_E()

def serialize(tokens): return ''.join(tokens)
def steps(tokens):     return len(tokens)

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore_plans(token_lists):
    todo = [serialize(t) for t in token_lists if serialize(t) not in results]
    fetched = 0
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        if not batch: break
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        fetched += len(batch)
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")
    return fetched

def has_row(prefix):
    sp = serialize(prefix)
    for suf in E:
        if serialize(prefix + suf) not in results:
            return False
    return True

def ensure_rows_for(prefixes):
    need = []
    for pref in prefixes:
        for suf in E:
            plan = pref + suf
            s = serialize(plan)
            if s not in results:
                need.append(plan)
    if need:
        explore_plans(need)

def row_of(prefix):
    # lazily ensure rows for 'prefix'
    if not has_row(prefix):
        ensure_rows_for([prefix])
    # now read signature at the end of each (prefix + suf)
    out = []
    for suf in E:
        plan = prefix + suf
        s = serialize(plan)
        out.append(results[s][steps(plan)])
    return tuple(out)

def successors(toks):
    succ = []
    # doors
    for d in DOORS:
        succ.append(toks + [d])
    # one mark, and mark+door to split copies
    for m in MARKS:
        succ.append(toks + [m])
        for d in DOORS:
            succ.append(toks + [m,d])
    return succ

# ---- Select aleph
print("Selecting aleph…")
api("/select", {"id": TEAM_ID, "problemName": "aleph"}, timeout=30)

# ---- Seed
results = {}
root = []
explore_plans([root])
ensure_rows_for([root])

# ---- Discover states with marks
rows = {}
rows[row_of(root)] = root
frontier = [root]

rounds = 0
MAX_ROUNDS = 8   # usually enough; increase if needed
while rounds < MAX_ROUNDS:
    rounds += 1
    succs = []
    for rep in frontier:
        succs.extend(successors(rep))
    explore_plans(succs)       # fetch what we touched
    # We don't mass-ensure here; lazy fill happens in row_of()
    new = []
    for rep in list(frontier):
        # doors first
        for d in DOORS:
            sp = rep + [d]
            r = row_of(sp)
            if r not in rows:
                rows[r] = sp
                new.append(sp)
        # marks can also produce new rows
        for m in MARKS:
            sp = rep + [m]
            r = row_of(sp)
            if r not in rows:
                rows[r] = sp
                new.append(sp)
    if not new:
        break
    frontier = new

reps_sorted = sorted(rows.values(), key=len)
idx_of = {row_of(rep): i for i, rep in enumerate(reps_sorted)}
n = len(reps_sorted)
print(f"States discovered: {n} | first reps: {reps_sorted[:8]}{' …' if n>8 else ''}")

# ---- Build delta (lazily ensure each rep+[door])
delta = [[None]*6 for _ in range(n)]
for i, rep in enumerate(reps_sorted):
    for a, d in enumerate(DOORS):
        sp = rep + [d]
        r = row_of(sp)          # this will ensure rows if missing
        delta[i][a] = idx_of[r]

filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# ---- Perfect matching on mutual pairs (self-loops allowed)
ports = [(i,a) for i in range(n) for a in range(6)]
pos   = {p:k for k,p in enumerate(ports)}
adj   = [[] for _ in ports]
for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b] == i]
    for b in backs:
        adj[u].append(pos[(j,b)])

from collections import deque
N = len(ports)
pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N; INF = 10**9

def bfs():
    q = deque()
    for u in range(N):
        if pairU[u] == -1:
            dist[u]=0; q.append(u)
        else:
            dist[u]=INF
    found=False
    while q:
        u=q.popleft()
        for v in adj[u]:
            pu=pairV[v]
            if pu==-1:
                found=True
            elif dist[pu]==INF:
                dist[pu]=dist[u]+1; q.append(pu)
    return found

def dfs(u):
    for v in adj[u]:
        pu=pairV[v]
        if pu==-1 or (dist[pu]==dist[u]+1 and dfs(pu)):
            pairU[u]=v; pairV[v]=u
            return True
    dist[u]=INF
    return False

matching=0
while bfs():
    for u in range(N):
        if pairU[u]==-1 and dfs(u):
            matching+=1
print(f"Port matching size: {matching}/{N}")
assert matching==N, "Could not pair all ports once."

# ---- Emit connections (one per undirected pair)
edges=set(); connections=[]
for u in range(N):
    v=pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    # sanity
    assert delta[i][a]==j and delta[j][b]==i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})

print(f"Connections: {len(connections)}")

# ---- Rooms & startingRoom
def label_after(rep):
    s = serialize(rep)
    return results[s][steps(rep)]
rooms = [label_after(rep) for rep in reps_sorted]
start_idx = idx_of[row_of(root)]

final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== MAP (aleph) ===")
print(json.dumps(final_map, indent=2))

# ---- Validate: cover all 6*n ports, allow loops
def validate_map_general(m):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used, f"Port reused: {a}"
        assert b not in used, f"Port reused: {b}"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        assert per[r]==set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

validate_map_general(final_map)

# ---- Submit
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=90)
print("Server response:", resp.status_code, resp.text)


Selecting aleph…
  fetched 1 (queryCount=2) | total=1
  fetched 58 (queryCount=61) | total=59
  fetched 54 (queryCount=116) | total=113
  fetched 54 (queryCount=171) | total=167
  fetched 54 (queryCount=226) | total=221
  fetched 54 (queryCount=281) | total=275
  fetched 54 (queryCount=336) | total=329
  fetched 54 (queryCount=391) | total=383
  fetched 52 (queryCount=444) | total=435
  fetched 52 (queryCount=497) | total=487
  fetched 52 (queryCount=550) | total=539
  fetched 52 (queryCount=603) | total=591
  fetched 54 (queryCount=658) | total=645
  fetched 54 (queryCount=713) | total=699
  fetched 54 (queryCount=768) | total=753
  fetched 54 (queryCount=823) | total=807
  fetched 54 (queryCount=878) | total=861
  fetched 54 (queryCount=933) | total=915
  fetched 52 (queryCount=986) | total=967
  fetched 52 (queryCount=1039) | total=1019
  fetched 52 (queryCount=1092) | total=1071
  fetched 52 (queryCount=1145) | total=1123
  fetched 54 (queryCount=1200) | total=1177
  fetched 54 (qu

KeyboardInterrupt: 

In [ ]:
# === ALEPH (efficient) — doors-only frontier, marks only in suffixes ===
import json, requests
from collections import defaultdict, deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # full ID

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']
CHUNK = 200

def serialize(tokens): return ''.join(tokens)
def steps(tokens):     return len(tokens)

def E_light():
    # ε + 4 marks + 6 doors
    E = [[]]
    E += [[m] for m in MARKS]
    E += [[d] for d in DOORS]
    return E

def E_stronger():
    E = E_light()
    E += [[m,d] for m in MARKS for d in DOORS]   # mark then door
    E += [[d,m] for d in DOORS for m in MARKS]   # door then mark
    return E

def api(path, payload, timeout=90):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore_plans(token_lists, results):
    todo = [serialize(t) for t in token_lists if serialize(t) not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        if not batch: break
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

def ensure_rows_for(prefixes, E, results):
    need = []
    seen = set()
    for pref in prefixes:
        for suf in E:
            plan = pref + suf
            s = serialize(plan)
            if s not in results and s not in seen:
                seen.add(s)
                need.append(plan)
    if need:
        explore_plans(need, results)

def row_of(prefix, E, results):
    # assumes ensure_rows_for already called for this prefix with this E
    return tuple(results[serialize(prefix + suf)][steps(prefix + suf)] for suf in E)

def discover_states_aleph(E):
    # doors-only frontier; marks only in suffix E
    results = {}
    api("/select", {"id": TEAM_ID, "problemName": "aleph"}, timeout=30)

    root = []
    explore_plans([root], results)
    ensure_rows_for([root], E, results)

    rows = {}
    rows[row_of(root, E, results)] = root
    frontier = [root]

    # expand only with doors
    while True:
        succs = [rep + [d] for rep in frontier for d in DOORS]
        # fetch only rep, rep+door signatures
        ensure_rows_for(frontier, E, results)
        ensure_rows_for(succs,   E, results)

        new = []
        for rep in list(frontier):
            for d in DOORS:
                sp = rep + [d]
                r = row_of(sp, E, results)
                if r not in rows:
                    rows[r] = sp
                    new.append(sp)
        if not new:
            break
        frontier = new
        # aleph typically ~12 states; stop if plenty
        if len(rows) >= 20:
            break

    reps_sorted = sorted(rows.values(), key=len)
    idx_of = {row_of(rep, E, results): i for i, rep in enumerate(reps_sorted)}
    n = len(reps_sorted)
    print(f"States discovered: {n} | reps (first): {reps_sorted[:8]}{' …' if n>8 else ''}")

    # Build delta on rep+door only (ensure rows for those)
    ensure_rows_for(reps_sorted, E, results)
    ensure_rows_for([rep+[d] for rep in reps_sorted for d in DOORS], E, results)

    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps_sorted):
        for a, d in enumerate(DOORS):
            sp = rep + [d]
            r = row_of(sp, E, results)
            delta[i][a] = idx_of[r]

    filled = sum(v is not None for row in delta for v in row)
    print(f"Filled transitions: {filled}/{6*n}")
    return results, E, reps_sorted, idx_of, delta

def pair_ports(delta):
    n = len(delta)
    ports = [(i,a) for i in range(n) for a in range(6)]
    pos   = {p:k for k,p in enumerate(ports)}
    adj   = [[] for _ in ports]  # u->v mutual candidates

    for u,(i,a) in enumerate(ports):
        j = delta[i][a]
        backs = [b for b in range(6) if delta[j][b] == i]
        for b in backs:
            adj[u].append(pos[(j,b)])

    # Hopcroft–Karp
    N = len(ports)
    pairU = [-1]*N; pairV = [-1]*N; dist = [0]*N; INF = 10**9
    from collections import deque
    def bfs():
        q = deque()
        for u in range(N):
            if pairU[u] == -1:
                dist[u]=0; q.append(u)
            else:
                dist[u]=INF
        found=False
        while q:
            u=q.popleft()
            for v in adj[u]:
                pu = pairV[v]
                if pu == -1:
                    found = True
                elif dist[pu] == INF:
                    dist[pu] = dist[u] + 1; q.append(pu)
        return found
    def dfs(u):
        for v in adj[u]:
            pu = pairV[v]
            if pu == -1 or (dist[pu]==dist[u]+1 and dfs(pu)):
                pairU[u]=v; pairV[v]=u
                return True
        dist[u]=INF
        return False
    matching=0
    while bfs():
        for u in range(N):
            if pairU[u]==-1 and dfs(u):
                matching+=1
    return matching, ports, pairU

def build_map(results, E, reps_sorted, idx_of, delta, pairU):
    n = len(reps_sorted)
    ports = [(i,a) for i in range(n) for a in range(6)]
    # connections
    edges=set(); connections=[]
    for u in range(len(ports)):
        v = pairU[u]
        i,a = ports[u]
        j,b = ports[v]
        # sanity
        assert delta[i][a]==j and delta[j][b]==i
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in edges:
            edges.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})
    # rooms labels (label after executing rep)
    def label_after(rep): return results[serialize(rep)][steps(rep)]
    rooms = [label_after(rep) for rep in reps_sorted]
    start_idx = idx_of[row_of([], E, results)]
    return {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

def validate_map_general(m, n):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used and b not in used, "Port reused"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        assert per[r]==set(range(6)), f"Room {r} missing doors: {set(range(6)) - per[r]}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

# ---- Run with light E; if pairing not full, retry with stronger E only on needed prefixes
results, E, reps_sorted, idx_of, delta = None, None, None, None, None

print("Selecting & discovering (light E)…")
results, E1, reps_sorted, idx_of, delta = discover_states_aleph(E_light())
match, ports, pairU = pair_ports(delta)
print(f"Port matching size (E1): {match}/{len(ports)}")

if match != len(ports):
    print("Pairing not perfect with E1; retrying with stronger E2 on minimal prefixes…")
    # Recompute rows only for reps and rep+door under E2
    results2 = dict(results)  # reuse already fetched stuff
    E2 = E_stronger()
    ensure_rows_for(reps_sorted, E2, results2)
    ensure_rows_for([rep+[d] for rep in reps_sorted for d in DOORS], E2, results2)
    # rebuild index & delta with E2
    idx2 = {row_of(rep, E2, results2): i for i, rep in enumerate(reps_sorted)}
    delta2 = [[None]*6 for _ in range(len(reps_sorted))]
    for i, rep in enumerate(reps_sorted):
        for a,d in enumerate(DOORS):
            delta2[i][a] = idx2[row_of(rep+[d], E2, results2)]
    match2, ports2, pairU2 = pair_ports(delta2)
    print(f"Port matching size (E2): {match2}/{len(ports2)}")
    assert match2 == len(ports2), "Still ambiguous after E2; increase suffixes slightly."
    final_map = build_map(results2, E2, reps_sorted, idx2, delta2, pairU2)
    n = len(reps_sorted)
else:
    final_map = build_map(results, E1, reps_sorted, idx_of, delta, pairU)
    n = len(reps_sorted)

print("\n=== MAP (aleph) ===")
print(json.dumps(final_map, indent=2))
validate_map_general(final_map, n)

# Submit
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=90)
print("Server response:", resp.status_code, resp.text)


Selecting & discovering (light E)…
  fetched 1 (queryCount=2) | total=1
  fetched 10 (queryCount=13) | total=11
  fetched 60 (queryCount=74) | total=71
  fetched 200 (queryCount=275) | total=271
  fetched 40 (queryCount=316) | total=311
  fetched 60 (queryCount=377) | total=371
States discovered: 6 | reps (first): [[], ['0'], ['2'], ['3'], ['5'], ['0', '1']]
Filled transitions: 36/36
Port matching size (E1): 36/36

=== MAP (aleph) ===
{
  "rooms": [
    0,
    1,
    3,
    1,
    2,
    0
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 1,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 0,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 2,
        "door": 4
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3

In [ ]:
# === ALEPH data collector: door-frontier + targeted mark probes → JSON snapshot ===
import json, requests
from collections import deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"  # use full string exactly
OUTPATH = "/content/aleph_results.json"

DOORS = "012345"
MARKS = ["[0]","[1]","[2]","[3]"]

# Tuning knobs — safe defaults
MAX_PLANS = 12000        # hard cap on total unique plans to fetch
MAX_DEPTH = 8            # door-depth for BFS frontier (no marks in frontier)
CHUNK     = 200          # /explore batch size

# ---------------- Helpers ----------------
def api(path, payload, timeout=90):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def is_mark_token_at(s, i):
    # plan strings encode marks as literal "[k]"; detect token boundaries
    return s.startswith("[0]", i) or s.startswith("[1]", i) or s.startswith("[2]", i) or s.startswith("[3]", i)

def door_depth(plan_str):
    """Count only door steps (digits 0..5) in the plan string."""
    c = 0
    i = 0
    n = len(plan_str)
    while i < n:
        ch = plan_str[i]
        if ch in DOORS:
            c += 1
            i += 1
        elif ch == '[':
            # skip "[k]"
            i += 3
        else:
            i += 1
    return c

# ---------------- Select aleph ----------------
print("Selecting aleph …")
api("/select", {"id": TEAM_ID, "problemName": "aleph"}, timeout=30)

# ---------------- Collector state ----------------
results = {}          # plan_str -> list[int] (labels per step)
pending = set()       # plans staged but not yet fetched
seen    = set()       # in results or pending, to avoid duplicates

def stage(plan):
    """Stage a plan for exploration if not seen and not over cap."""
    if plan in seen:
        return False
    if len(results) + len(pending) >= MAX_PLANS:
        return False
    pending.add(plan)
    seen.add(plan)
    return True

def flush():
    """Send pending plans to /explore in CHUNK batches."""
    if not pending:
        return 0
    queued = 0
    pending_list = list(pending)
    pending.clear()
    for i in range(0, len(pending_list), CHUNK):
        batch = pending_list[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # store 2-bit labels
        queued += len(batch)
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")
    return queued

# ---------------- Seed & BFS (doors-only) ----------------
q = deque()
stage("")   # root
flush()
q.append("")  # door frontier uses only door-strings

while q and len(results) < MAX_PLANS:
    p = q.popleft()
    ddepth = door_depth(p)

    # 1) Stage door successors (and enqueue them if under depth)
    for d in DOORS:
        s = p + d
        if stage(s) and ddepth + 1 < MAX_DEPTH:
            q.append(s)

    # 2) Stage targeted marked probes for this prefix (but do NOT enqueue them)
    #    - p + [k]
    #    - p + [k] + d   (local effect only; helps separate copies)
    for m in MARKS:
        stage(p + m)
        for d in DOORS:
            stage(p + m + d)

    # 3) Periodically flush to keep memory bounded
    if len(pending) >= CHUNK * 5 or len(results) + len(pending) >= MAX_PLANS:
        flush()

# Final flush
flush()

# ---------------- Save snapshot ----------------
with open(OUTPATH, "w") as f:
    json.dump(results, f)
print(f"\nSaved snapshot with {len(results)} plans → {OUTPATH}")

# (Optional) quick stats
door_only = sum(1 for k in results if all(ch in DOORS for ch in k))
mark_present = len(results) - door_only
print(f"Door-only plans: {door_only} | Plans with marks: {mark_present}")


Selecting aleph …
  fetched 1 (queryCount=2) | total=1
  fetched 200 (queryCount=203) | total=201
  fetched 200 (queryCount=404) | total=401
  fetched 200 (queryCount=605) | total=601
  fetched 200 (queryCount=806) | total=801
  fetched 200 (queryCount=1007) | total=1001
  fetched 20 (queryCount=1028) | total=1021
  fetched 200 (queryCount=1229) | total=1221
  fetched 200 (queryCount=1430) | total=1421
  fetched 200 (queryCount=1631) | total=1621
  fetched 200 (queryCount=1832) | total=1821
  fetched 200 (queryCount=2033) | total=2021
  fetched 20 (queryCount=2054) | total=2041
  fetched 200 (queryCount=2255) | total=2241
  fetched 200 (queryCount=2456) | total=2441
  fetched 200 (queryCount=2657) | total=2641
  fetched 200 (queryCount=2858) | total=2841
  fetched 200 (queryCount=3059) | total=3041
  fetched 20 (queryCount=3080) | total=3061
  fetched 200 (queryCount=3281) | total=3261
  fetched 200 (queryCount=3482) | total=3461
  fetched 200 (queryCount=3683) | total=3661
  fetched 2

In [ ]:
from google.colab import files
files.download("/content/aleph_results.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# === ALEPH top-up → rebuild → validate → submit (single cell) ===
import json, requests, os
from collections import defaultdict, deque
from pathlib import Path

# ---------- Config ----------
BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"   # full ID
SNAP    = "/content/aleph_results.json"                 # snapshot location
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

# ---------- Helpers ----------
def serialize(tokens): return ''.join(tokens)
def steps(tokens):     return len(tokens)

def steps_from_str(plan):
    i=0; n=len(plan); c=0
    while i<n:
        ch = plan[i]
        if ch in '012345':
            c+=1; i+=1
        elif ch=='[' and i+2<n and plan[i+2]==']':
            c+=1; i+=3
        else:
            i+=1
    return c

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore_plans(token_lists, results):
    todo = []
    seen = set()
    for t in token_lists:
        s = serialize(t)
        if s not in results and s not in seen:
            seen.add(s); todo.append(s)
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        if not batch: break
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # store 2-bit labels
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

def get_label_from_results(results, plan_str):
    seq = results[plan_str]
    idx = steps_from_str(plan_str)
    if idx >= len(seq): idx = len(seq)-1
    return seq[idx] & 3

def E_light():
    # ε + 4 marks + 6 doors
    E=[[]]
    E += [[m] for m in MARKS]
    E += [[d] for d in DOORS]
    return E

def E_strong():
    E = E_light()
    E += [[m,d] for m in MARKS for d in DOORS]   # mark→door
    E += [[d,m] for d in DOORS for m in MARKS]   # door→mark
    return E

def ensure_rows_for(prefixes, E, results):
    need=[]
    seen=set()
    for pref in prefixes:
        for suf in E:
            plan = pref + suf
            s = serialize(plan)
            if s not in results and s not in seen:
                seen.add(s); need.append(plan)
    if need:
        explore_plans(need, results)

def row_of(prefix, E, results):
    # assumes ensure_rows_for called for this (prefix, E)
    vals=[]
    for suf in E:
        s = serialize(prefix + suf)
        vals.append(get_label_from_results(results, s))
    return tuple(vals)

def build_reps_via_doors(E, results, max_depth=3):
    # small doors-only frontier to get ~6 states
    frontier = [[]]
    ensure_rows_for(frontier, E, results)
    rows = { row_of([], E, results): [] }
    rows[row_of([], E, results)].append([])
    for depth in range(max_depth):
        succs=[]
        for rep in frontier:
            for d in DOORS:
                succs.append(rep+[d])
        ensure_rows_for(succs, E, results)
        new=[]
        for pref in succs:
            r = row_of(pref, E, results)
            if r not in rows:
                rows[r]=[pref]; new.append(pref)
            else:
                rows[r].append(pref)
        if not new:
            break
        frontier = new
        if len(rows) >= 6:   # aleph will exceed 6 only after mark-prefix top-up
            # keep exploring up to max_depth just to stabilize
            pass
    reps = [min(pls, key=len) for pls in rows.values()]
    reps.sort(key=len)
    idx = {row_of(r, E, results): i for i, r in enumerate(reps)}
    return reps, idx

def build_delta(reps, idx, E, results):
    n=len(reps)
    delta=[[None]*6 for _ in range(n)]
    # ensure rows for all rep+door with this E
    ensure_rows_for(reps, E, results)
    ensure_rows_for([rep+[d] for rep in reps for d in DOORS], E, results)
    for i, rep in enumerate(reps):
        for a,d in enumerate(DOORS):
            sp = rep+[d]
            delta[i][a] = idx[row_of(sp, E, results)]
    return delta

def pair_ports(delta):
    n=len(delta)
    ports=[(i,a) for i in range(n) for a in range(6)]
    pos={p:k for k,p in enumerate(ports)}
    adj=[[] for _ in ports]
    for u,(i,a) in enumerate(ports):
        j=delta[i][a]
        backs=[b for b in range(6) if delta[j][b]==i]
        for b in backs:
            adj[u].append(pos[(j,b)])
    # Hopcroft–Karp
    N=len(ports)
    pairU=[-1]*N; pairV=[-1]*N; dist=[0]*N; INF=10**9
    from collections import deque
    def bfs():
        q=deque()
        for u in range(N):
            if pairU[u]==-1:
                dist[u]=0; q.append(u)
            else:
                dist[u]=INF
        found=False
        while q:
            u=q.popleft()
            for v in adj[u]:
                pu=pairV[v]
                if pu==-1: found=True
                elif dist[pu]==INF:
                    dist[pu]=dist[u]+1; q.append(pu)
        return found
    def dfs(u):
        for v in adj[u]:
            pu=pairV[v]
            if pu==-1 or (dist[pu]==dist[u]+1 and dfs(pu)):
                pairU[u]=v; pairV[v]=u
                return True
        dist[u]=INF
        return False
    matching=0
    while bfs():
        for u in range(N):
            if pairU[u]==-1 and dfs(u):
                matching+=1
    return matching, ports, pairU

def build_map(results, E, reps, idx, delta, pairU):
    n=len(reps)
    ports=[(i,a) for i in range(n) for a in range(6)]
    edges=set(); connections=[]
    for u in range(len(ports)):
        v = pairU[u]
        i,a = ports[u]
        j,b = ports[v]
        assert delta[i][a]==j and delta[j][b]==i
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in edges:
            edges.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})
    # room labels after executing rep
    rooms = [ get_label_from_results(results, serialize(rep)) for rep in reps ]
    start_idx = idx[row_of([], E, results)]
    return {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

def validate_map_general(m, n):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used and b not in used, "Port reused"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    per=defaultdict(set)
    for (r,d) in used: per[r].add(d)
    for r in range(n):
        missing = set(range(6)) - per[r]
        assert not missing, f"Room {r} missing doors: {missing}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

# ---------- Load snapshot or start blank ----------
snap_candidates = [Path(SNAP), Path("/mnt/data/aleph_results.json")]
loaded_path = next((p for p in snap_candidates if p.exists()), None)
if loaded_path:
    with open(loaded_path, "r") as f:
        results = json.load(f)
    print(f"Loaded snapshot: {len(results)} plans from {loaded_path}")
else:
    results = {}
    print("No existing snapshot. Starting fresh.")

# ---------- Select aleph ----------
print("Selecting aleph …")
api("/select", {"id": TEAM_ID, "problemName": "aleph"}, timeout=30)

# ---------- Step 1: get 6 representatives under light E (doors-only small frontier) ----------
E1 = E_light()
# Ensure base rows for "", and small door frontier (depth 2–3)
explore_plans([[]], results)
reps1, idx1 = build_reps_via_doors(E1, results, max_depth=3)
print("Light E reps:", reps1)

# ---------- Step 2: targeted marked-prefix top-up on just the reps ----------
# Fetch rows for: rep+[k], rep+[k]+d, rep+d+[k] (k=0..3, d=0..5)
probe = []
for rep in reps1:
    for m in MARKS:
        probe.append(rep + [m])
        for d in DOORS:
            probe.append(rep + [m, d])
    for d in DOORS:
        for m in MARKS:
            probe.append(rep + [d, m])
explore_plans(probe, results)

# ---------- Step 3: rebuild with strong E on reps and rep+door only ----------
E2 = E_strong()
# Ensure rows for reps and reps+door under E2 (no BFS)
ensure_rows_for(reps1, E2, results)
ensure_rows_for([rep+[d] for rep in reps1 for d in DOORS], E2, results)

# Recompute reps under E2 (signatures may split); keep shortest per row
rows2 = defaultdict(list)
for rep in reps1:
    r = row_of(rep, E2, results)
    rows2[r].append(rep)
reps2 = [min(pl, key=len) for pl in rows2.values()]
reps2.sort(key=len)
idx2 = {row_of(r, E2, results): i for i, r in enumerate(reps2)}
print("Strong E reps:", reps2)

# Build delta on reps2 (ensuring only reps2 and reps2+door)
delta2 = build_delta(reps2, idx2, E2, results)
n = len(reps2)
filled = sum(v is not None for row in delta2 for v in row)
print(f"Filled transitions: {filled}/{6*n}")

# ---------- Step 4: pair ports ----------
matching, ports, pairU = pair_ports(delta2)
print(f"Port matching size: {matching}/{len(ports)}")
assert matching == len(ports), "Pairing incomplete; add a few more targeted probes if this hits."

# ---------- Step 5: build map, validate, submit ----------
final_map = build_map(results, E2, reps2, idx2, delta2, pairU)
print("\n=== MAP (aleph) ===")
print(json.dumps(final_map, indent=2))
validate_map_general(final_map, len(reps2))

resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=90)
print("Server response:", resp.status_code, resp.text)

# ---------- Save updated snapshot ----------
with open(SNAP, "w") as f:
    json.dump(results, f)
print(f"Saved updated snapshot with {len(results)} plans → {SNAP}")


Loaded snapshot: 12000 plans from /content/aleph_results.json
Selecting aleph …
Light E reps: [[], ['1'], ['4'], ['1', '1'], ['4', '2'], ['4', '3']]
  fetched 200 (queryCount=201) | total=12200
  fetched 200 (queryCount=402) | total=12400
  fetched 32 (queryCount=435) | total=12432
Strong E reps: [[], ['1'], ['4'], ['1', '1'], ['4', '2'], ['4', '3']]
Filled transitions: 36/36
Port matching size: 36/36

=== MAP (aleph) ===
{
  "rooms": [
    0,
    2,
    1,
    0,
    3,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 0,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 1,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 0,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
   

In [ ]:
# === ALEPH: progressive marked-prefix top-up → rebuild → validate → submit (single cell) ===
import json, requests, os, itertools
from collections import defaultdict, deque
from pathlib import Path

# ---------- Config ----------
BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"   # full ID (email + token)
SNAP    = "/content/aleph_results.json"
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

# ---------- Helpers ----------
def serialize(tokens): return ''.join(tokens)

def steps_from_str(plan):
    i=0; n=len(plan); c=0
    while i<n:
        ch = plan[i]
        if ch in '012345':
            c+=1; i+=1
        elif ch=='[' and i+2<n and plan[i+2]==']':
            c+=1; i+=3
        else:
            i+=1
    return c

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore_plans(token_lists, results):
    todo = []
    seen = set()
    for t in token_lists:
        s = serialize(t)
        if s not in results and s not in seen:
            seen.add(s); todo.append(s)
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        if not batch: break
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]  # 2-bit labels only
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

def get_label_from_results(results, plan_str):
    seq = results[plan_str]
    idx = steps_from_str(plan_str)
    if idx >= len(seq): idx = len(seq)-1
    return seq[idx] & 3

def ensure_rows_for(prefixes, suffixes, results):
    need=[]
    seen=set()
    for pref in prefixes:
        for suf in suffixes:
            plan = pref + suf
            s = serialize(plan)
            if s not in results and s not in seen:
                seen.add(s); need.append(plan)
    if need:
        explore_plans(need, results)

def row_of(prefix, suffixes, results):
    return tuple(get_label_from_results(results, serialize(prefix + suf)) for suf in suffixes)

def E_light():
    # ε + 4 marks + 6 doors
    E=[[]] + [[m] for m in MARKS] + [[d] for d in DOORS]
    return E

def E_strong():
    # ε, marks, doors, mark→door, door→mark, and mark→door→mark
    E = E_light()
    E += [[m,d] for m in MARKS for d in DOORS]
    E += [[d,m] for d in DOORS for m in MARKS]
    E += [[m1,d,m2] for m1 in MARKS for d in DOORS for m2 in MARKS]
    return E

def reps_via_small_door_frontier(results, depth=3):
    E = E_light()
    frontier = [[]]
    ensure_rows_for([[]], E, results)
    rows = { row_of([], E, results): [[]] }
    for _ in range(depth):
        succs = [rep+[d] for rep in frontier for d in DOORS]
        ensure_rows_for(succs, E, results)
        new=[]
        for pref in succs:
            r = row_of(pref, E, results)
            if r not in rows:
                rows[r]=[pref]; new.append(pref)
            else:
                rows[r].append(pref)
        if not new: break
        frontier=new
        if len(rows) >= 6:
            # doors-only usually stabilizes near 6; true split needs marked prefixes
            pass
    reps = [min(pls, key=len) for pls in rows.values()]
    reps.sort(key=len)
    return reps

def recompute_reps_strong(results, reps_seed):
    E = E_strong()
    ensure_rows_for(reps_seed, E, results)
    ensure_rows_for([rep+[d] for rep in reps_seed for d in DOORS], E, results)
    rows2 = defaultdict(list)
    for rep in reps_seed:
        r = row_of(rep, E, results)
        rows2[r].append(rep)
    reps2 = [min(pl, key=len) for pl in rows2.values()]
    reps2.sort(key=len)
    idx2 = {row_of(r, E, results): i for i, r in enumerate(reps2)}
    return E, reps2, idx2

def build_delta(results, E, reps, idx):
    n=len(reps)
    delta=[[None]*6 for _ in range(n)]
    ensure_rows_for(reps, E, results)
    ensure_rows_for([rep+[d] for rep in reps for d in DOORS], E, results)
    for i, rep in enumerate(reps):
        for a,d in enumerate(DOORS):
            sp = rep+[d]
            delta[i][a] = idx[row_of(sp, E, results)]
    return delta

def pair_ports(delta):
    n=len(delta)
    ports=[(i,a) for i in range(n) for a in range(6)]
    pos={p:k for k,p in enumerate(ports)}
    adj=[[] for _ in ports]
    for u,(i,a) in enumerate(ports):
        j=delta[i][a]
        backs=[b for b in range(6) if delta[j][b]==i]
        for b in backs:
            adj[u].append(pos[(j,b)])
    # Hopcroft–Karp
    N=len(ports)
    pairU=[-1]*N; pairV=[-1]*N; dist=[0]*N; INF=10**9
    from collections import deque
    def bfs():
        q=deque()
        for u in range(N):
            if pairU[u]==-1:
                dist[u]=0; q.append(u)
            else:
                dist[u]=INF
        found=False
        while q:
            u=q.popleft()
            for v in adj[u]:
                pu=pairV[v]
                if pu==-1: found=True
                elif dist[pu]==INF:
                    dist[pu]=dist[u]+1; q.append(pu)
        return found
    def dfs(u):
        for v in adj[u]:
            pu=pairV[v]
            if pu==-1 or (dist[pu]==dist[u]+1 and dfs(pu)):
                pairU[u]=v; pairV[v]=u
                return True
        dist[u]=INF
        return False
    matching=0
    while bfs():
        for u in range(N):
            if pairU[u]==-1 and dfs(u):
                matching+=1
    return matching, ports, pairU

def build_map(results, E, reps, idx, delta, pairU):
    n=len(reps)
    ports=[(i,a) for i in range(n) for a in range(6)]
    edges=set(); connections=[]
    for u in range(len(ports)):
        v = pairU[u]
        i,a = ports[u]
        j,b = ports[v]
        assert delta[i][a]==j and delta[j][b]==i
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in edges:
            edges.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})
    rooms = [ get_label_from_results(results, serialize(rep)) for rep in reps ]
    start_idx = idx[row_of([], E, results)]
    return {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

def validate_map_general(m, n):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used and b not in used, "Port reused"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

# ---------- Load snapshot or start blank ----------
snap_candidates = [Path(SNAP), Path("/mnt/data/aleph_results.json")]
loaded_path = next((p for p in snap_candidates if p.exists()), None)
if loaded_path:
    with open(loaded_path, "r") as f:
        results = json.load(f)
    print(f"Loaded snapshot: {len(results)} plans from {loaded_path}")
else:
    results = {}
    print("No existing snapshot. Starting fresh.")

# ---------- Select aleph ----------
print("Selecting aleph …")
api("/select", {"id": TEAM_ID, "problemName": "aleph"}, timeout=30)

# ---------- Step 1: base reps (doors-only) ----------
reps = reps_via_small_door_frontier(results, depth=3)
print("Base reps (light):", reps)

# ---------- Step 2: progressive marked-prefix probes ----------
rounds = []

# Round 1 (what you already tried, but keep in for idempotency):
#   rep+[k], rep+[k]+d, rep+d+[k]
r1 = []
for rep in reps:
    for m in MARKS:
        r1.append(rep+[m])
        for d in DOORS:
            r1.append(rep+[m,d])
    for d in DOORS:
        for m in MARKS:
            r1.append(rep+[d,m])
rounds.append(("R1 basic", r1))

# Round 2:
#   rep+[k1]+[k2]  (two marks)
#   rep+[k]+d+[k2] (mark→door→mark)
r2 = []
for rep in reps:
    for m1 in MARKS:
        for m2 in MARKS:
            r2.append(rep+[m1,m2])
        for d in DOORS:
            for m2 in MARKS:
                r2.append(rep+[m1,d,m2])
rounds.append(("R2 two-marks & m-d-m", r2))

# Round 3 (short bursts beyond next door):
#   (rep+door)+[k], (rep+door)+[k]+door
r3 = []
for rep in reps:
    for d in DOORS:
        rp = rep+[d]
        for m in MARKS:
            r3.append(rp+[m])
            for d2 in DOORS:
                r3.append(rp+[m,d2])
rounds.append(("R3 door→mark (+optional door)", r3))

for name, probe in rounds:
    before = len(results)
    print(f"Top-up {name} …")
    explore_plans(probe, results)
    print(f"  +{len(results)-before} new rows")

    # Recompute with strong E after each round; stop if states split and pairing perfect
    E, reps2, idx2 = recompute_reps_strong(results, reps)
    delta2 = build_delta(results, E, reps2, idx2)
    n = len(reps2)
    filled = sum(v is not None for row in delta2 for v in row)
    matching, ports, pairU = pair_ports(delta2)
    print(f"  States now: {n} | filled δ: {filled}/{6*n} | matching: {matching}/{len(ports)}")
    if n > 6 and matching == len(ports):
        print("  ✅ Sufficient split and perfect pairing achieved.")
        reps = reps2; E_final = E; idx_final = idx2; delta_final = delta2; pairU_final = pairU
        break
else:
    # If loop didn't break, use latest status anyway (may still be 6 states)
    print("Did not exceed 6 states yet; using latest strong model.")
    reps = reps2; E_final = E; idx_final = idx2; delta_final = delta2; pairU_final = pairU

# ---------- Step 3: build map, validate, submit ----------
final_map = build_map(results, E_final, reps, idx_final, delta_final, pairU_final)
print("\n=== MAP (aleph) ===")
print(json.dumps(final_map, indent=2))
validate_map_general(final_map, len(reps))

resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=90)
print("Server response:", resp.status_code, resp.text)

# ---------- Save updated snapshot ----------
with open(SNAP, "w") as f:
    json.dump(results, f)
print(f"Saved updated snapshot with {len(results)} plans → {SNAP}")


Loaded snapshot: 12432 plans from /content/aleph_results.json
Selecting aleph …
Base reps (light): [[], ['1'], ['4'], ['1', '1'], ['4', '2'], ['4', '3']]
Top-up R1 basic …
  +0 new rows
  fetched 200 (queryCount=201) | total=12632
  fetched 200 (queryCount=402) | total=12832
  fetched 176 (queryCount=579) | total=13008
  fetched 200 (queryCount=780) | total=13208
  fetched 200 (queryCount=981) | total=13408
  fetched 200 (queryCount=1182) | total=13608
  fetched 200 (queryCount=1383) | total=13808
  fetched 200 (queryCount=1584) | total=14008
  fetched 200 (queryCount=1785) | total=14208
  fetched 200 (queryCount=1986) | total=14408
  fetched 200 (queryCount=2187) | total=14608
  fetched 200 (queryCount=2388) | total=14808
  fetched 200 (queryCount=2589) | total=15008
  fetched 200 (queryCount=2790) | total=15208
  fetched 200 (queryCount=2991) | total=15408
  fetched 200 (queryCount=3192) | total=15608
  fetched 200 (queryCount=3393) | total=15808
  fetched 176 (queryCount=3570) | tot

In [ ]:
# === ALEPH: BFS with marks-in-prefix → rebuild → validate → submit (single cell) ===
import json, requests
from collections import defaultdict, deque
from pathlib import Path

# ---------- Config ----------
BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"   # full ID (email + token)
SNAP    = "/content/aleph_results.json"
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']
ALPH  = DOORS + MARKS

# ---------- Utilities ----------
def serialize(tokens): return ''.join(tokens)

def steps_from_str(plan):
    i=0; n=len(plan); c=0
    while i<n:
        ch = plan[i]
        if ch in '012345':
            c+=1; i+=1
        elif ch=='[' and i+2<n and plan[i+2]==']':
            c+=1; i+=3
        else:
            i+=1
    return c

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore_plans(token_lists, results):
    # Batch unexplored stringified plans
    todo=[]; seen=set()
    for t in token_lists:
        s=serialize(t)
        if s not in results and s not in seen:
            seen.add(s); todo.append(s)
    for i in range(0, len(todo), CHUNK):
        batch=todo[i:i+CHUNK]
        if not batch: break
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | snapshot={len(results)}")

def get_label(results, plan_str):
    seq = results[plan_str]
    idx = steps_from_str(plan_str)
    if idx >= len(seq): idx = len(seq)-1
    return seq[idx] & 3

# ---------- Suffix sets ----------
def E_strong():
    E=[[]]
    E += [[m] for m in MARKS] + [[d] for d in DOORS]
    E += [[m,d] for m in MARKS for d in DOORS]     # mark→door
    E += [[d,m] for d in DOORS for m in MARKS]     # door→mark
    E += [[m1,d,m2] for m1 in MARKS for d in DOORS for m2 in MARKS]  # m→d→m
    return E

def ensure_rows_for(prefixes, suffixes, results):
    need=[]
    have=set()
    for pref in prefixes:
        for suf in suffixes:
            s = serialize(pref + suf)
            if s not in results and s not in have:
                have.add(s); need.append(pref + suf)
    if need:
        explore_plans(need, results)

def row_of(prefix, suffixes, results):
    return tuple(get_label(results, serialize(prefix + suf)) for suf in suffixes)

# ---------- Build states with marks in PREFIX (small BFS) ----------
def discover_with_prefix_marks(results, max_depth=3, target_states=12, max_prefixes=4000):
    E = E_strong()
    frontier=[[]]
    ensure_rows_for([[]], E, results)
    rows = { row_of([], E, results): [] }
    rows[row_of([], E, results)].append([])
    visited=set([''])  # stringified prefixes we've expanded
    total_expanded=0

    for depth in range(max_depth+1):
        # generate successors over DOORS + MARKS
        succ=[]
        for pref in frontier:
            for sym in ALPH:
                child = pref + [sym]
                s = serialize(child)
                if s in visited: continue
                visited.add(s)
                succ.append(child)
        if not succ: break

        # guardrails
        if len(succ) > max_prefixes:
            succ = succ[:max_prefixes]

        ensure_rows_for(succ, E, results)

        new=[]
        for pref in succ:
            sig = row_of(pref, E, results)
            if sig not in rows:
                rows[sig]=[pref]; new.append(pref)
            else:
                rows[sig].append(pref)
        total_expanded += len(succ)
        frontier = new if new else succ  # keep moving even if no NEW rows yet

        print(f"depth={depth+1}  unique_states={len(rows)}  expanded={total_expanded}")
        if len(rows) >= target_states:
            break

    reps = [min(pl, key=len) for pl in rows.values()]
    reps.sort(key=len)
    idx  = {row_of(r, E, results): i for i,r in enumerate(reps)}
    return E, reps, idx

def build_delta(results, E, reps, idx):
    ensure_rows_for(reps, E, results)
    ensure_rows_for([rep+[d] for rep in reps for d in DOORS], E, results)
    n=len(reps)
    delta=[[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps):
        for a,d in enumerate(DOORS):
            sp = rep + [d]
            delta[i][a] = idx[row_of(sp, E, results)]
    return delta

def pair_ports(delta):
    n=len(delta)
    ports=[(i,a) for i in range(n) for a in range(6)]
    pos={p:k for k,p in enumerate(ports)}
    adj=[[] for _ in ports]
    # Build edges where door has a return path
    for u,(i,a) in enumerate(ports):
        j = delta[i][a]
        backs=[b for b in range(6) if delta[j][b]==i]
        for b in backs:
            adj[u].append(pos[(j,b)])
    # Hopcroft–Karp
    N=len(ports)
    pairU=[-1]*N; pairV=[-1]*N; dist=[0]*N; INF=10**9
    from collections import deque
    def bfs():
        q=deque()
        for u in range(N):
            if pairU[u]==-1:
                dist[u]=0; q.append(u)
            else:
                dist[u]=INF
        found=False
        while q:
            u=q.popleft()
            for v in adj[u]:
                pu=pairV[v]
                if pu==-1: found=True
                elif dist[pu]==INF:
                    dist[pu]=dist[u]+1; q.append(pu)
        return found
    def dfs(u):
        for v in adj[u]:
            pu=pairV[v]
            if pu==-1 or (dist[pu]==dist[u]+1 and dfs(pu)):
                pairU[u]=v; pairV[v]=u
                return True
        dist[u]=INF
        return False
    matching=0
    while bfs():
        for u in range(N):
            if pairU[u]==-1 and dfs(u):
                matching+=1
    return matching, ports, pairU

def build_map(results, E, reps, idx, delta, pairU):
    n=len(reps)
    ports=[(i,a) for i in range(n) for a in range(6)]
    edges=set(); connections=[]
    for u in range(len(ports)):
        v = pairU[u]
        i,a = ports[u]
        j,b = ports[v]
        assert delta[i][a]==j and delta[j][b]==i
        key = tuple(sorted([(i,a),(j,b)]))
        if key not in edges:
            edges.add(key)
            connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})
    rooms = [ get_label(results, serialize(rep)) for rep in reps ]
    start_idx = idx[row_of([], E, results)]
    return {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

def validate_map(m, n):
    used=set()
    for e in m["connections"]:
        a=(e["from"]["room"],e["from"]["door"])
        b=(e["to"]["room"],e["to"]["door"])
        assert 0<=a[1]<6 and 0<=b[1]<6
        assert a not in used and b not in used, "Port reused"
        used.add(a); used.add(b)
    assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
    print(f"Validation OK ✅ (rooms={n}, connections={len(m['connections'])})")

# ---------- Load snapshot ----------
snap_candidates = [Path(SNAP), Path("/mnt/data/aleph_results.json"), Path("/mnt/data/aleph_results (2).json")]
loaded_path = next((p for p in snap_candidates if p.exists()), None)
if loaded_path:
    with open(loaded_path, "r") as f:
        results = json.load(f)
    print(f"Loaded snapshot: {len(results)} plans from {loaded_path}")
else:
    results = {}
    print("No existing snapshot. Starting fresh.")

# ---------- Select aleph ----------
print("Selecting aleph …")
api("/select", {"id": TEAM_ID, "problemName": "aleph"}, timeout=30)

# ---------- Discover with marks in PREFIX (depth 3; if still 6, try depth 4) ----------
E, reps, idx = discover_with_prefix_marks(results, max_depth=3, target_states=12, max_prefixes=4000)
if len(reps) <= 6:
    print("Still ≤6 states after depth 3. Trying depth 4 with tighter cap …")
    E, reps, idx = discover_with_prefix_marks(results, max_depth=4, target_states=12, max_prefixes=6000)

print("Reps:", reps)
print("States:", len(reps))

# ---------- Build delta / Pair / Map ----------
delta = build_delta(results, E, reps, idx)
n = len(reps)
filled = sum(v is not None for row in delta for v in row)
print(f"Filled transitions: {filled}/{6*n}")

matching, ports, pairU = pair_ports(delta)
print(f"Port matching size: {matching}/{len(ports)}")
assert matching == len(ports), "Pairing incomplete (unexpected with strong E)."

final_map = build_map(results, E, reps, idx, delta, pairU)
print("\n=== MAP (aleph) ===")
print(json.dumps(final_map, indent=2))
validate_map(final_map, n)

# ---------- Submit ----------
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=120)
print("Server response:", resp.status_code, resp.text)

# ---------- Save snapshot ----------
with open(SNAP, "w") as f:
    json.dump(results, f)
print(f"Saved updated snapshot ({len(results)} plans) → {SNAP}")


Loaded snapshot: 16080 plans from /content/aleph_results.json
Selecting aleph …
  fetched 200 (queryCount=201) | snapshot=16280
  fetched 200 (queryCount=402) | snapshot=16480
  fetched 80 (queryCount=483) | snapshot=16560
depth=1  unique_states=7  expanded=10
  fetched 200 (queryCount=684) | snapshot=16760
  fetched 200 (queryCount=885) | snapshot=16960
  fetched 200 (queryCount=1086) | snapshot=17160
  fetched 200 (queryCount=1287) | snapshot=17360
  fetched 200 (queryCount=1488) | snapshot=17560
  fetched 200 (queryCount=1689) | snapshot=17760
  fetched 200 (queryCount=1890) | snapshot=17960
  fetched 200 (queryCount=2091) | snapshot=18160
  fetched 200 (queryCount=2292) | snapshot=18360
  fetched 200 (queryCount=2493) | snapshot=18560
  fetched 200 (queryCount=2694) | snapshot=18760
  fetched 200 (queryCount=2895) | snapshot=18960
  fetched 200 (queryCount=3096) | snapshot=19160
  fetched 200 (queryCount=3297) | snapshot=19360
  fetched 200 (queryCount=3498) | snapshot=19560
  fetc

KeyError: (0, 0, 1, 2, 3, 0, 0, 1, 2, 3, 2, 0, 0, 1, 2, 3, 2, 0, 0, 1, 2, 3, 2, 0, 0, 1, 2, 3, 2, 0, 0, 1, 2, 3, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3)

In [ ]:
# --- Recover from KeyError: make reps closed under door transitions, then finish ---
from collections import defaultdict, deque
import json, requests

DOORS = ['0','1','2','3','4','5']

def serialize(tokens): return ''.join(tokens)

def steps_from_str(plan):
    i=0; n=len(plan); c=0
    while i<n:
        ch = plan[i]
        if ch in '012345':
            c+=1; i+=1
        elif ch=='[' and i+2<n and plan[i+2]==']':
            c+=1; i+=3
        else:
            i+=1
    return c

def get_label(results, plan_str):
    seq = results[plan_str]
    idx = steps_from_str(plan_str)
    if idx >= len(seq): idx = len(seq)-1
    return seq[idx] & 3

def ensure_rows_for(prefixes, suffixes, results):
    todo=[]; seen=set()
    for pref in prefixes:
        for suf in suffixes:
            s = serialize(pref + suf)
            if s not in results and s not in seen:
                seen.add(s); todo.append(pref+suf)
    for i in range(0, len(todo), 200):
        batch = todo[i:i+200]
        if not batch: break
        r = requests.post(f"{BASE}/explore", json={"id": TEAM_ID, "plans": [serialize(t) for t in batch]}, timeout=180)
        r.raise_for_status()
        data = r.json()
        for p, seq in zip([serialize(t) for t in batch], data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | snapshot={len(results)}")

def row_of(prefix, E, results):
    return tuple(get_label(results, serialize(prefix + suf)) for suf in E)

def build_delta_closed(results, E, seed_reps, max_states=60):
    """Ensure closure: if rep+door yields a new signature, add it as a new rep and continue."""
    reps = list(seed_reps)
    idx  = {row_of(r, E, results): i for i, r in enumerate(reps)}
    changed = True
    round_no = 0
    while changed:
        round_no += 1
        changed = False
        # Ensure all rows needed for current reps and their door successors
        ensure_rows_for(reps, E, results)
        succ = [rep+[d] for rep in reps for d in DOORS]
        ensure_rows_for(succ, E, results)
        # Add any missing signatures encountered at rep+door
        for rep in list(reps):
            for d in DOORS:
                sp = rep + [d]
                sig = row_of(sp, E, results)
                if sig not in idx:
                    idx[sig] = len(reps)
                    reps.append(sp)
                    changed = True
                    if len(reps) >= max_states:
                        changed = False
                        break
            if len(reps) >= max_states:
                break
        print(f"closure round {round_no}: states={len(reps)}")
    # Build delta now that closed
    n = len(reps)
    delta = [[None]*6 for _ in range(n)]
    for i, rep in enumerate(reps):
        for a, d in enumerate(DOORS):
            delta[i][a] = idx[row_of(rep+[d], E, results)]
    return reps, idx, delta

# 1) Rebuild closed reps/idx/delta
reps_closed, idx_closed, delta = build_delta_closed(results, E, reps, max_states=80)
n = len(reps_closed)
filled = sum(v is not None for row in delta for v in row)
print(f"Closed set → states={n}, filled δ={filled}/{6*n}")

# 2) Pair ports (Hopcroft–Karp)
ports = [(i,a) for i in range(n) for a in range(6)]
pos   = {p:k for k,p in enumerate(ports)}
adj   = [[] for _ in ports]
for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b]==i]
    for b in backs: adj[u].append(pos[(j,b)])

from collections import deque
N = len(ports); pairU = [-1]*N; pairV = [-1]*N; dist=[0]*N; INF=10**9
def bfs():
    q=deque(); found=False
    for u in range(N):
        if pairU[u]==-1: dist[u]=0; q.append(u)
        else: dist[u]=INF
    while q:
        u=q.popleft()
        for v in adj[u]:
            pu=pairV[v]
            if pu==-1: found=True
            elif dist[pu]==INF: dist[pu]=dist[u]+1; q.append(pu)
    return found
def dfs(u):
    for v in adj[u]:
        pu=pairV[v]
        if pu==-1 or (dist[pu]==dist[u]+1 and dfs(pu)):
            pairU[u]=v; pairV[v]=u; return True
    dist[u]=INF; return False
matching=0
while bfs():
    for u in range(N):
        if pairU[u]==-1 and dfs(u): matching+=1
print(f"Port matching size: {matching}/{N}")
assert matching==N, "Pairing incomplete—consider raising max_states or exploring a few more marked prefixes."

# 3) Build map
def label_after(rep): return get_label(results, serialize(rep))
rooms = [label_after(rep) for rep in reps_closed]
start_idx = idx_closed[row_of([], E, results)]
edges=set(); connections=[]
for u in range(N):
    v=pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    assert delta[i][a]==j and delta[j][b]==i
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a}, "to":{"room":j,"door":b}})

final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("\n=== MAP (aleph, closed) ===")
print(json.dumps(final_map, indent=2))
print(f"connections: {len(connections)} (should be {3*n})")

# 4) Validate (each port used exactly once)
used=set()
for e in final_map["connections"]:
    a=(e["from"]["room"], e["from"]["door"])
    b=(e["to"]["room"],   e["to"]["door"])
    assert a not in used and b not in used, f"Port reused: {a if a in used else b}"
    used.add(a); used.add(b)
assert len(used)==6*n, f"Covered {len(used)} ports, expected {6*n}"
print(f"Validation OK ✅ (rooms={n}, connections={len(final_map['connections'])})")

# 5) Submit
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=120)
print("Server response:", resp.status_code, resp.text)


closure round 1: states=80


KeyError: (3, 0, 1, 2, 3, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3)

In [ ]:
# === ALEPH: targeted mark-in-prefix fetch rooted at 6 base reps → rebuild & submit ===
import json, requests
from collections import defaultdict

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

# Load snapshot
with open(SNAP, "r") as f:
    results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

def steps_from_str(plan):
    i=0; n=len(plan); c=0
    while i<n:
        ch=plan[i]
        if ch in '012345': c+=1; i+=1
        elif ch=='[' and i+2<n and plan[i+2]==']': c+=1; i+=3
        else: i+=1
    return c

def label(plan):
    seq = results[plan]
    idx = steps_from_str(plan)
    if idx >= len(seq): idx = len(seq)-1
    return seq[idx] & 3

# Strong suffix set
def E():
    lst=[""]
    lst += MARKS + DOORS
    lst += [m+d for m in MARKS for d in DOORS]
    lst += [d+m for d in DOORS for m in MARKS]
    lst += [m1+d+m2 for m1 in MARKS for d in DOORS for m2 in MARKS]
    return lst

SFX = E()
def has_row(prefix): return all(prefix + s in results for s in SFX)
def row(prefix):     return tuple(label(prefix + s) for s in SFX)

# Our 6 base reps (as you saw): "", "1", "4", "11", "42", "43"
reps = ["", "1", "4", "11", "42", "43"]

# Build a focused probe set:
#   For each rep R:
#     R+[k], R+[k]+d, R+d+[k], R+[k1]+[k2], R+[k]+d+[k2]
def gen_probes(reps):
    P=set()
    for R in reps:
        for m in MARKS:
            P.add(R + m)
            for d in DOORS:
                P.add(R + m + d)
        for d in DOORS:
            for m in MARKS:
                P.add(R + d + m)
        for m1 in MARKS:
            for m2 in MARKS:
                P.add(R + m1 + m2)
            for d in DOORS:
                for m2 in MARKS:
                    P.add(R + m1 + d + m2)
    # also ensure we can classify successors R+d under strong E
    for R in reps:
        for d in DOORS:
            for s in SFX:
                P.add(R + d + s)
    return sorted(P, key=len)

probes = gen_probes(reps)
print("Planned probe count:", len(probes))
explore(probes)

# Recompute state signatures for reps and for their door-successors (strong E)
rows = {}
for R in reps:
    if has_row(R):
        rows.setdefault(row(R), []).append(R)
    for d in DOORS:
        S = R + d
        if has_row(S):
            rows.setdefault(row(S), []).append(S)

# Representatives = one shortest prefix per signature
reps2 = [min(v, key=len) for v in rows.values()]
reps2.sort(key=len)
idx = { row(r): i for i, r in enumerate(reps2) }

# Build delta
delta = [[None]*6 for _ in range(len(reps2))]
for i, R in enumerate(reps2):
    for a,d in enumerate(DOORS):
        delta[i][a] = idx[row(R + d)]

# Pair ports (perfect mutual pairing)
ports=[(i,a) for i in range(len(reps2)) for a in range(6)]
pos={p:k for k,p in enumerate(ports)}
adj=[[] for _ in ports]
for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b]==i]
    for b in backs:
        adj[u].append(pos[(j,b)])

from collections import deque
N=len(ports); pairU=[-1]*N; pairV=[-1]*N; dist=[0]*N; INF=10**9
def bfs():
    q=deque(); found=False
    for u in range(N):
        if pairU[u]==-1: dist[u]=0; q.append(u)
        else: dist[u]=INF
    while q:
        u=q.popleft()
        for v in adj[u]:
            pu=pairV[v]
            if pu==-1: found=True
            elif dist[pu]==INF: dist[pu]=dist[u]+1; q.append(pu)
    return found
def dfs(u):
    for v in adj[u]:
        pu=pairV[v]
        if pu==-1 or (dist[pu]==dist[u]+1 and dfs(pu)):
            pairU[u]=v; pairV[v]=u; return True
    dist[u]=INF; return False

matching=0
while bfs():
    for u in range(N):
        if pairU[u]==-1 and dfs(u): matching+=1

print("States:", len(reps2), " | matching:", matching, "/", N)

# Build final map
rooms = [label(R) for R in reps2]
start_idx = idx[row("")]
edges=set(); connections=[]
for u in range(N):
    v=pairU[u]
    i,a = ports[u]
    j,b = ports[v]
    if (delta[i][a]!=j) or (delta[j][b]!=i): continue
    key = tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a}, "to":{"room":j,"door":b}})

final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}
print("Connections:", len(connections))

# Submit
api("/select", {"id": TEAM_ID, "problemName": "aleph"}, timeout=30)
resp = requests.post(f"{BASE}/guess", json={"id": TEAM_ID, "map": final_map}, timeout=120)
print("Server response:", resp.status_code, resp.text)

# Save snapshot
with open(SNAP, "w") as f:
    json.dump(results, f)
print("Saved:", SNAP, "plans:", len(results))


Loaded snapshot: 16080
Planned probe count: 5650
States: 6  | matching: 36 / 36
Connections: 22
Server response: 200 {"correct":false}
Saved: /content/aleph_results.json plans: 16080


In [ ]:
# === ALEPH: targeted mark–door–mark probes ===
import json, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

with open(SNAP) as f: results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status(); return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

# Use your 6 base reps
reps = ["", "1", "4", "11", "42", "43"]

# Generate mark–door–mark probes
probes = []
for R in reps:
    for m1 in MARKS:
        for d in DOORS:
            for m2 in MARKS:
                probes.append(R + m1 + d + m2)
print("Planned probe count:", len(probes))

explore(probes)

# Save snapshot
with open(SNAP, "w") as f: json.dump(results, f)
print("Saved:", SNAP, "plans:", len(results))


Loaded snapshot: 16080
Planned probe count: 576
Saved: /content/aleph_results.json plans: 16080


In [ ]:
# === ALEPH targeted mark–door–mark experiment ===
import json, requests
from collections import deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

# Load snapshot
with open(SNAP) as f: results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status(); return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

def steps_from_str(plan):
    i=0; n=len(plan); c=0
    while i<n:
        ch=plan[i]
        if ch in '012345': c+=1; i+=1
        elif ch=='[' and i+2<n and plan[i+2]==']': c+=1; i+=3
        else: i+=1
    return c

def label(plan):
    seq = results[plan]; idx = steps_from_str(plan)
    if idx >= len(seq): idx = len(seq)-1
    return seq[idx] & 3

# Strong suffix set
def E():
    lst=[""]
    lst += MARKS + DOORS
    lst += [m+d for m in MARKS for d in DOORS]
    lst += [d+m for d in DOORS for m in MARKS]
    lst += [m1+d+m2 for m1 in MARKS for d in DOORS for m2 in MARKS]
    return lst
SFX = E()

def has_row(prefix): return all(prefix + s in results for s in SFX)
def row(prefix):     return tuple(label(prefix + s) for s in SFX)

# Our 6 base reps
reps = ["", "1", "4", "11", "42", "43"]

# Generate mark–door–mark probes
probes = []
for R in reps:
    for m1 in MARKS:
        for d in DOORS:
            for m2 in MARKS:
                probes.append(R + m1 + d + m2)
print("Planned probe count:", len(probes))
explore(probes)

# Recompute states
rows = {}
for R in reps:
    if has_row(R):
        rows.setdefault(row(R), []).append(R)
    for d in DOORS:
        if has_row(R+d):
            rows.setdefault(row(R+d), []).append(R+d)
for p in probes:
    if has_row(p):
        rows.setdefault(row(p), []).append(p)

reps2 = [min(v, key=len) for v in rows.values()]
reps2.sort(key=len)
idx = { row(r): i for i,r in enumerate(reps2) }

print("States discovered:", len(reps2))

# Build delta
delta = [[None]*6 for _ in range(len(reps2))]
for i,R in enumerate(reps2):
    for a,d in enumerate(DOORS):
        if has_row(R+d):
            delta[i][a] = idx[row(R+d)]

# Pair ports using maximum matching
ports=[(i,a) for i in range(len(reps2)) for a in range(6)]
pos={p:k for k,p in enumerate(ports)}
adj=[[] for _ in ports]
for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    backs = [b for b in range(6) if delta[j][b]==i] if j is not None else []
    for b in backs: adj[u].append(pos[(j,b)])

from collections import deque
N=len(ports); pairU=[-1]*N; pairV=[-1]*N; dist=[0]*N
def bfs():
    q=deque(); found=False
    for u in range(N):
        if pairU[u]==-1: dist[u]=0; q.append(u)
        else: dist[u]=10**9
    while q:
        u=q.popleft()
        for v in adj[u]:
            pu=pairV[v]
            if pu==-1: found=True
            elif dist[pu]==10**9: dist[pu]=dist[u]+1; q.append(pu)
    return found
def dfs(u):
    for v in adj[u]:
        pu=pairV[v]
        if pu==-1 or (dist[pu]==dist[u]+1 and dfs(pu)):
            pairU[u]=v; pairV[v]=u; return True
    dist[u]=10**9; return False

matching=0
while bfs():
    for u in range(N):
        if pairU[u]==-1 and dfs(u): matching+=1

print("Matching size:", matching, "/", N)

# Build final map
rooms=[label(R) for R in reps2]
start_idx = idx[row("")]
edges=set(); connections=[]
for u in range(N):
    v=pairU[u]
    if v==-1: continue
    i,a=ports[u]; j,b=ports[v]
    if delta[i][a]!=j or delta[j][b]!=i: continue
    key=tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a}, "to":{"room":j,"door":b}})

final_map={"rooms":rooms,"startingRoom":start_idx,"connections":connections}
print("=== MAP (aleph) ===")
print(json.dumps(final_map, indent=2))
print("Connections:", len(connections))

# Save snapshot
with open(SNAP,"w") as f: json.dump(results,f)
print("Saved:", SNAP, "plans:", len(results))


Loaded snapshot: 16080
Planned probe count: 576
States discovered: 6
Matching size: 36 / 36
=== MAP (aleph) ===
{
  "rooms": [
    0,
    2,
    1,
    0,
    3,
    1
  ],
  "startingRoom": 0,
  "connections": [
    {
      "from": {
        "room": 0,
        "door": 0
      },
      "to": {
        "room": 0,
        "door": 0
      }
    },
    {
      "from": {
        "room": 0,
        "door": 1
      },
      "to": {
        "room": 1,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 2
      },
      "to": {
        "room": 0,
        "door": 2
      }
    },
    {
      "from": {
        "room": 0,
        "door": 3
      },
      "to": {
        "room": 1,
        "door": 3
      }
    },
    {
      "from": {
        "room": 0,
        "door": 4
      },
      "to": {
        "room": 2,
        "door": 1
      }
    },
    {
      "from": {
        "room": 0,
        "door": 5
      },
      "to": {
        "room": 0,
        "door": 

In [ ]:
# === ALEPH: multi-mark probing experiment ===
import json, requests
from collections import deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

with open(SNAP) as f: results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status(); return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

def steps_from_str(plan):
    i=0; n=len(plan); c=0
    while i<n:
        ch=plan[i]
        if ch in '012345': c+=1; i+=1
        elif ch=='[' and i+2<n and plan[i+2]==']': c+=1; i+=3
        else: i+=1
    return c

def label(plan):
    seq = results[plan]; idx = steps_from_str(plan)
    if idx >= len(seq): idx = len(seq)-1
    return seq[idx] & 3

# Strong suffix set
def E():
    lst=[""]
    lst += MARKS + DOORS
    lst += [m+d for m in MARKS for d in DOORS]
    lst += [d+m for d in DOORS for m in MARKS]
    lst += [m1+d+m2 for m1 in MARKS for d in DOORS for m2 in MARKS]
    return lst
SFX = E()

def has_row(prefix): return all(prefix + s in results for s in SFX)
def row(prefix):     return tuple(label(prefix + s) for s in SFX)

# Base reps
reps = ["", "1", "4", "11", "42", "43"]

# ---- Multi-mark probe set ----
probes=set()
for R in reps:
    # consecutive marks
    for m1 in MARKS:
        for m2 in MARKS:
            probes.add(R+m1+m2)
    # mark-door-mark-door
    for m1 in MARKS:
        for d1 in DOORS:
            for m2 in MARKS:
                for d2 in DOORS:
                    probes.add(R+m1+d1+m2+d2)
    # mark-door-mark-door-mark
    for m1 in MARKS:
        for d1 in DOORS:
            for m2 in MARKS:
                for d2 in DOORS:
                    for m3 in MARKS:
                        probes.add(R+m1+d1+m2+d2+m3)

probes=list(probes)
print("Planned probe count:", len(probes))
explore(probes)

# Recompute states
rows={}
for R in reps:
    if has_row(R): rows.setdefault(row(R),[]).append(R)
    for d in DOORS:
        if has_row(R+d): rows.setdefault(row(R+d),[]).append(R+d)
for p in probes:
    if has_row(p): rows.setdefault(row(p),[]).append(p)

reps2=[min(v,key=len) for v in rows.values()]
reps2.sort(key=len)
idx={row(r):i for i,r in enumerate(reps2)}
print("States discovered:", len(reps2))

# Build delta
delta=[[None]*6 for _ in range(len(reps2))]
for i,R in enumerate(reps2):
    for a,d in enumerate(DOORS):
        if has_row(R+d):
            delta[i][a]=idx[row(R+d)]

# Pair ports
ports=[(i,a) for i in range(len(reps2)) for a in range(6)]
pos={p:k for k,p in enumerate(ports)}
adj=[[] for _ in ports]
for u,(i,a) in enumerate(ports):
    j=delta[i][a]
    backs=[b for b in range(6) if delta[j][b]==i] if j is not None else []
    for b in backs: adj[u].append(pos[(j,b)])

N=len(ports); pairU=[-1]*N; pairV=[-1]*N; dist=[0]*N
def bfs():
    from collections import deque
    q=deque(); found=False
    for u in range(N):
        if pairU[u]==-1: dist[u]=0; q.append(u)
        else: dist[u]=10**9
    while q:
        u=q.popleft()
        for v in adj[u]:
            pu=pairV[v]
            if pu==-1: found=True
            elif dist[pu]==10**9: dist[pu]=dist[u]+1; q.append(pu)
    return found
def dfs(u):
    for v in adj[u]:
        pu=pairV[v]
        if pu==-1 or (dist[pu]==dist[u]+1 and dfs(pu)):
            pairU[u]=v; pairV[v]=u; return True
    dist[u]=10**9; return False

matching=0
while bfs():
    for u in range(N):
        if pairU[u]==-1 and dfs(u): matching+=1
print("Matching size:", matching,"/",N)

# Build final map
rooms=[label(R) for R in reps2]
start_idx=idx[row("")]
edges=set(); connections=[]
for u in range(N):
    v=pairU[u]
    if v==-1: continue
    i,a=ports[u]; j,b=ports[v]
    if delta[i][a]!=j or delta[j][b]!=i: continue
    key=tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a},"to":{"room":j,"door":b}})
final_map={"rooms":rooms,"startingRoom":start_idx,"connections":connections}

print("=== MAP (aleph) ===")
print(json.dumps(final_map,indent=2))
print("Connections:",len(connections))

# Save snapshot
with open(SNAP,"w") as f: json.dump(results,f)
print("Saved:",SNAP,"plans:",len(results))


Loaded snapshot: 16080
Planned probe count: 17376


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/explore

In [ ]:
# === ALEPH: smaller multi-mark batch ===
import json, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

with open(SNAP) as f: results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status(); return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

# base reps
reps = ["", "1", "4", "11", "42", "43"]

# targeted multi-mark probes (much smaller set)
probes=set()
for R in reps:
    # consecutive marks
    for m1 in MARKS:
        for m2 in MARKS:
            probes.add(R+m1+m2)
    # mark-door-mark
    for m1 in MARKS:
        for d1 in DOORS:
            for m2 in MARKS:
                probes.add(R+m1+d1+m2)
    # mark-door-mark-door
    for m1 in MARKS:
        for d1 in DOORS:
            for m2 in MARKS:
                for d2 in DOORS:
                    probes.add(R+m1+d1+m2+d2)

probes=list(probes)
print("Planned probe count:", len(probes))  # should be ~1000
explore(probes)

# Save snapshot
with open(SNAP,"w") as f: json.dump(results,f)
print("Saved:",SNAP,"plans:",len(results))


Loaded snapshot: 16080
Planned probe count: 4128


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/explore

In [ ]:
# === ALEPH: safe probe batch ===
import json, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

with open(SNAP) as f: results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status(); return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

# base reps
reps = ["", "1", "4", "11", "42", "43"]

probes=set()
for R in reps:
    # door + mark
    for d in DOORS:
        for m in MARKS:
            probes.add(R+d+m)
    # mark + door
    for m in MARKS:
        for d in DOORS:
            probes.add(R+m+d)
    # mark + door + mark
    for m1 in MARKS:
        for d in DOORS:
            for m2 in MARKS:
                probes.add(R+m1+d+m2)

probes=list(probes)
print("Planned probe count:", len(probes))
explore(probes)

with open(SNAP,"w") as f: json.dump(results,f)
print("Saved:",SNAP,"plans:",len(results))


Loaded snapshot: 16080
Planned probe count: 864
Saved: /content/aleph_results.json plans: 16080


In [ ]:
# === ALEPH: mark–door–mark–door probes ===
import json, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

with open(SNAP) as f: results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status(); return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

# base reps (from your current map)
reps = ["", "1", "4", "11", "42", "43"]

# generate mark–door–mark–door probes
probes=set()
for R in reps:
    for m1 in MARKS:
        for d1 in DOORS:
            for m2 in MARKS:
                for d2 in DOORS:
                    probes.add(R+m1+d1+m2+d2)

probes=list(probes)
print("Planned probe count:", len(probes))  # expect ~576
explore(probes)

with open(SNAP,"w") as f: json.dump(results,f)
print("Saved:",SNAP,"plans:",len(results))


Loaded snapshot: 16080
Planned probe count: 3456


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/explore

In [ ]:
# === ALEPH: safe mark–door–mark–door probes ===
import json, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 200

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

with open(SNAP) as f: results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status(); return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

# base reps
reps = ["", "1", "4", "11", "42", "43"]

# build only valid mark–door–mark–door probes
probes=set()
for R in reps:
    for m1 in MARKS:
        for d1 in DOORS:
            for m2 in MARKS:
                for d2 in DOORS:
                    # valid pattern: ... mark door mark door ...
                    probes.add(R+m1+d1+m2+d2)

probes=list(probes)
print("Planned probe count:", len(probes))  # should be 6*4*6*4*6 = 3456 max
explore(probes)

with open(SNAP,"w") as f: json.dump(results,f)
print("Saved:",SNAP,"plans:",len(results))


Loaded snapshot: 16080
Planned probe count: 3456


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/explore

In [ ]:
# === ALEPH: mark–door–mark (safe) ===
import json, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 100   # keep even smaller

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

with open(SNAP) as f: results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status(); return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results and len(p) <= 20]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

# base reps
reps = ["", "1", "4", "11", "42", "43"]

# generate mark–door–mark probes only
probes=set()
for R in reps:
    for m1 in MARKS:
        for d in DOORS:
            for m2 in MARKS:
                probes.add(R+m1+d+m2)

probes=list(probes)
print("Planned probe count:", len(probes))  # expect 576
explore(probes)

with open(SNAP,"w") as f: json.dump(results,f)
print("Saved:",SNAP,"plans:",len(results))


Loaded snapshot: 16080
Planned probe count: 576
Saved: /content/aleph_results.json plans: 16080


In [ ]:
# === ALEPH: mark–door–mark–door–mark (depth 5) ===
import json, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 100  # keep small

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

# Load snapshot
with open(SNAP) as f: results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status(); return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results and len(p) <= 15]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

# base reps: only the shortest 3 to keep probe count reasonable
reps = ["", "1", "4"]

# Generate mark–door–mark–door–mark probes
probes=set()
for R in reps:
    for m1 in MARKS:
        for d1 in DOORS:
            for m2 in MARKS:
                for d2 in DOORS:
                    for m3 in MARKS:
                        probes.add(R+m1+d1+m2+d2+m3)

probes=list(probes)
print("Planned probe count:", len(probes))
explore(probes)

# Save updated snapshot
with open(SNAP,"w") as f: json.dump(results,f)
print("Saved:", SNAP, "plans:", len(results))

# --- Quick rebuild of states & connections (lightweight) ---
E = [""] + DOORS  # suffixes: empty and one-door
def row_of(plan): return tuple(results[plan+s][len(plan+s)] for s in E)

rows, reps_seen = {}, []
rows[row_of("")] = ""
reps_seen.append("")
while len(rows) < 10 and reps_seen:
    cur = reps_seen.pop()
    for d in DOORS:
        sp = cur+d
        if sp in results:
            r = row_of(sp)
            if r not in rows:
                rows[r] = sp
                reps_seen.append(sp)

print("States discovered:", len(rows))
print("Reps:", list(rows.values())[:10])


Loaded snapshot: 16080
Planned probe count: 6912


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/explore

In [ ]:
# === ALEPH: reduced safe mark–door–mark–door–mark probes ===
import json, requests

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"
CHUNK   = 100

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']

with open(SNAP) as f: results = json.load(f)
print("Loaded snapshot:", len(results))

def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status(); return r.json()

def explore(plans):
    todo = [p for p in plans if p not in results and len(p) <= 12]
    for i in range(0, len(todo), CHUNK):
        batch = todo[i:i+CHUNK]
        data = api("/explore", {"id": TEAM_ID, "plans": batch}, timeout=180)
        for p, seq in zip(batch, data["results"]):
            results[p] = [v & 3 for v in seq]
        print(f"  fetched {len(batch)} (queryCount={data.get('queryCount')}) | total={len(results)}")

# shorter reps only
reps = ["", "1"]

# generate mark–door–mark–door–mark–door (so ends with a door)
probes=set()
for R in reps:
    for m1 in MARKS:
        for d1 in DOORS:
            for m2 in MARKS:
                for d2 in DOORS:
                    for m3 in MARKS:
                        for d3 in DOORS:
                            probes.add(R+m1+d1+m2+d2+m3+d3)

probes=list(probes)
print("Planned probe count:", len(probes))
explore(probes)

with open(SNAP,"w") as f: json.dump(results,f)
print("Saved:", SNAP, "plans:", len(results))


Loaded snapshot: 16080
Planned probe count: 27648


HTTPError: 400 Client Error: Bad Request for url: https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com/explore

In [ ]:
# === ALEPH robust exploration with bisection retry + rebuild & map ===
import json, requests, re
from collections import deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']
TOKEN_RE = re.compile(r'^(?:[0-5]|\[[0-3]\])+$')   # server-safe tokens only

# ---------- load snapshot ----------
with open(SNAP, "r") as f:
    results = json.load(f)
print("Loaded snapshot:", len(results))

# ---------- api helpers ----------
def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def _explore_raw(plans):
    if not plans:
        return 0, 0
    data = api("/explore", {"id": TEAM_ID, "plans": plans}, timeout=180)
    for p, seq in zip(plans, data["results"]):
        results[p] = [v & 3 for v in seq]
    return len(plans), data.get("queryCount", None)

def explore_bisect(plans, max_batch=200, max_len_chars=24, verbose=True):
    """
    Sends plans with:
      - token validation (regex),
      - length cap,
      - bisection fallback for HTTP 400.
    Returns: (accepted_count, rejected_count)
    """
    # prefilter obvious invalids
    clean = [p for p in plans if TOKEN_RE.match(p) and len(p) <= max_len_chars and p not in results]
    rejected = set(p for p in plans if p not in clean and p not in results)
    accepted = 0

    def send_batch(batch):
        nonlocal accepted, rejected
        if not batch:
            return
        try:
            n, qc = _explore_raw(batch)
            accepted += n
            if verbose:
                print(f"  accepted {n:4d}  (queryCount={qc})  total={len(results)}")
        except requests.HTTPError as e:
            # 400 -> split; other codes -> re-raise
            if e.response is None or e.response.status_code != 400 or len(batch) == 1:
                # if single plan fails (or other error), drop it
                rejected.update(batch)
                if verbose:
                    print(f"  rejected {len(batch)} (HTTP {e.response.status_code if e.response else '??'})")
                return
            mid = len(batch)//2
            send_batch(batch[:mid])
            send_batch(batch[mid:])

    # chunk then send with bisection per chunk
    for i in range(0, len(clean), max_batch):
        send_batch(clean[i:i+max_batch])

    return accepted, len(rejected)

# ---------- label helpers ----------
def steps_from_str(plan):
    i=0; n=len(plan); c=0
    while i<n:
        ch=plan[i]
        if ch in '012345': c+=1; i+=1
        elif ch=='[' and i+2<n and plan[i+2]==']': c+=1; i+=3
        else: i+=1
    return c

def label(plan):
    seq = results[plan]
    idx = steps_from_str(plan)
    if idx >= len(seq): idx = len(seq)-1
    return seq[idx] & 3

# strong suffix set (kept moderate to classify reliably)
def suffixes_strong():
    E=[""]
    E += MARKS + DOORS
    E += [m+d for m in MARKS for d in DOORS]
    E += [d+m for d in DOORS for m in MARKS]
    E += [m1+d+m2 for m1 in MARKS for d in DOORS for m2 in MARKS]
    return E

E = suffixes_strong()

def has_row(prefix): return all((prefix + s) in results for s in E)
def row(prefix):     return tuple(label(prefix + s) for s in E)

# ---------- generate focused probes ----------
# Use your 6 base reps
reps = ["", "1", "4", "11", "42", "43"]

# Two symmetry-breaking patterns (VALID ONLY):
#   A) mark–door–mark–door
#   B) door–mark–door–mark
def gen_m_d_m_d(reps):
    S=set()
    for R in reps:
        for m1 in MARKS:
            for d1 in DOORS:
                for m2 in MARKS:
                    for d2 in DOORS:
                        S.add(R + m1 + d1 + m2 + d2)    # A
                        S.add(R + d1 + m1 + d2 + m2)    # B
    return sorted(S, key=len)

probes = gen_m_d_m_d(reps)
print("Planned probes (before filtering):", len(probes))

acc, rej = explore_bisect(probes, max_batch=200, max_len_chars=24, verbose=True)
print(f"Explore summary: accepted={acc}, rejected={rej}, snapshot now={len(results)}")

# ---------- rebuild states / transitions / map ----------
# Build state reps from anything with rows available
rows = {}
front = deque([""])
seen  = set([""])
if has_row(""):
    rows[row("")] = [""]

# conservative frontier over only VALID next prefixes we already fetched
def next_tokens(s):
    # allow appending either a door or a mark (no two marks directly)
    nxt=[]
    # last token was mark if string ends with ']' (safe heuristic)
    last_is_mark = s.endswith(']')
    # doors always allowed
    for d in DOORS:
        nxt.append(s + d)
    # marks only if last wasn't a mark
    if not last_is_mark:
        for m in MARKS:
            nxt.append(s + m)
    return nxt

# BFS up to reasonable depth, recording any prefix for which we already have full row
for depth in range(6):  # up to +6 tokens
    level=[]
    while front:
        cur = front.popleft()
        for nx in next_tokens(cur):
            if nx in seen: continue
            seen.add(nx)
            if has_row(nx):
                rows.setdefault(row(nx), []).append(nx)
                level.append(nx)
    if not level: break
    front = deque(level)
    if len(rows) >= 30: break

# choose the shortest prefix per signature
reps2 = [min(v, key=len) for v in rows.values()]
reps2.sort(key=len)
idx = {row(r): i for i,r in enumerate(reps2)}

# delta
n = len(reps2)
delta = [[None]*6 for _ in range(n)]
for i,R in enumerate(reps2):
    for a,d in enumerate(DOORS):
        sp = R + d
        if has_row(sp):
            delta[i][a] = idx[row(sp)]

# build bipartite graph on ports with mutual edges only
ports=[(i,a) for i in range(n) for a in range(6)]
pos={p:k for k,p in enumerate(ports)}
adj=[[] for _ in ports]
for u,(i,a) in enumerate(ports):
    j = delta[i][a]
    if j is None: continue
    backs = [b for b in range(6) if delta[j][b]==i]
    for b in backs:
        adj[u].append(pos[(j,b)])

# Hopcroft–Karp
from collections import deque
N=len(ports); pairU=[-1]*N; pairV=[-1]*N; dist=[0]*N; INF=10**9
def bfs():
    q=deque(); found=False
    for u in range(N):
        if pairU[u]==-1: dist[u]=0; q.append(u)
        else: dist[u]=INF
    while q:
        u=q.popleft()
        for v in adj[u]:
            pu=pairV[v]
            if pu==-1: found=True
            elif dist[pu]==INF:
                dist[pu]=dist[u]+1; q.append(pu)
    return found
def dfs(u):
    for v in adj[u]:
        pu=pairV[v]
        if pu==-1 or (dist[pu]==dist[u]+1 and dfs(pu)):
            pairU[u]=v; pairV[v]=u; return True
    dist[u]=INF; return False

matching=0
while bfs():
    for u in range(N):
        if pairU[u]==-1 and dfs(u): matching+=1

# assemble map
def room_label(prefix): return label(prefix)
rooms = [room_label(r) for r in reps2]
start_idx = idx[row("")]
edges=set(); connections=[]
for u in range(N):
    v=pairU[u]
    if v==-1: continue
    i,a=ports[u]; j,b=ports[v]
    if delta[i][a]!=j or delta[j][b]!=i: continue
    key=tuple(sorted([(i,a),(j,b)]))
    if key not in edges:
        edges.add(key)
        connections.append({"from":{"room":i,"door":a}, "to":{"room":j,"door":b}})

final_map = {"rooms": rooms, "startingRoom": start_idx, "connections": connections}

print("\n=== MAP (aleph, robust) ===")
print(json.dumps(final_map, indent=2))
print(f"States: {n} | connections: {len(connections)} | matching: {matching}/{N}")

# save updated snapshot
with open(SNAP, "w") as f:
    json.dump(results, f)
print(f"Saved: {SNAP}  plans: {len(results)}")


Loaded snapshot: 16080
Planned probes (before filtering): 6912
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)
  rejected 1 (HTTP ??)


KeyboardInterrupt: 

In [ ]:
# === ALEPH: adaptive, safe probing until split (>6 states) ===
import json, requests, re
from collections import deque

BASE    = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"
TEAM_ID = "tarstars@gmail.com YJRqfaVXC0Olk5m8c_WV5g"
SNAP    = "/content/aleph_results.json"

DOORS = ['0','1','2','3','4','5']
MARKS = ['[0]','[1]','[2]','[3]']
TOKEN_RE = re.compile(r'^(?:[0-5]|\[[0-3]\])+$')

# ---------- load snapshot ----------
with open(SNAP, "r") as f:
    results = json.load(f)
print("Loaded snapshot:", len(results))

# ---------- helpers ----------
def api(path, payload, timeout=120):
    r = requests.post(f"{BASE}{path}", json=payload, timeout=timeout)
    r.raise_for_status()
    return r.json()

def safe_explore_one(plan, max_len=24):
    if plan in results: return "cached"
    if not TOKEN_RE.match(plan): return "invalid"
    if len(plan) > max_len: return "too_long"
    try:
        data = api("/explore", {"id": TEAM_ID, "plans": [plan]}, timeout=180)
        seq = data["results"][0]
        results[plan] = [v & 3 for v in seq]
        return "accepted"
    except requests.HTTPError as e:
        if e.response is not None and e.response.status_code == 400:
            return "rejected"
        raise

def steps_from_str(plan):
    i=0; n=len(plan); c=0
    while i<n:
        ch=plan[i]
        if ch in '012345': c+=1; i+=1
        elif ch=='[' and i+2<n and plan[i+2]==']': c+=1; i+=3
        else: i+=1
    return c

def label(plan):
    seq = results[plan]
    idx = steps_from_str(plan)
    if idx >= len(seq): idx = len(seq)-1
    return seq[idx] & 3

def suffixes_strong():
    E=[""]
    E += MARKS + DOORS
    E += [m+d for m in MARKS for d in DOORS]
    E += [d+m for d in DOORS for m in MARKS]
    E += [m1+d+m2 for m1 in MARKS for d in DOORS for m2 in MARKS]
    return E

E = suffixes_strong()
def has_row(prefix): return all((prefix + s) in results for s in E)
def row(prefix):     return tuple(label(prefix + s) for s in E)

def rebuild_and_print(stop_on_split=True):
    # discover states from prefixes we already have rows for
    rows = {}
    front = deque([""])
    seen  = set([""])
    if has_row(""):
        rows[row("")] = [""]
    for depth in range(6):  # conservative breadth
        nxt=[]
        while front:
            cur = front.popleft()
            # expand only by valid next tokens (no two marks in a row)
            last_is_mark = cur.endswith(']')
            # doors always ok
            for d in DOORS:
                nx = cur + d
                if nx in seen: continue
                seen.add(nx)
                if has_row(nx):
                    rows.setdefault(row(nx), []).append(nx)
                    nxt.append(nx)
            # marks only if last wasn't a mark
            if not last_is_mark:
                for m in MARKS:
                    nx = cur + m
                    if nx in seen: continue
                    seen.add(nx)
                    if has_row(nx):
                        rows.setdefault(row(nx), []).append(nx)
                        nxt.append(nx)
        if not nxt: break
        front = deque(nxt)
        if len(rows) >= 40: break

    reps = [min(v, key=len) for v in rows.values()]
    reps.sort(key=len)
    idx = {row(r): i for i,r in enumerate(reps)}
    n = len(reps)

    # delta
    delta = [[None]*6 for _ in range(n)]
    for i,R in enumerate(reps):
        for a,d in enumerate(DOORS):
            sp = R + d
            if has_row(sp):
                delta[i][a] = idx[row(sp)]

    # mutual port graph
    ports=[(i,a) for i in range(n) for a in range(6)]
    pos={p:k for k,p in enumerate(ports)}
    adj=[[] for _ in ports]
    for u,(i,a) in enumerate(ports):
        j = delta[i][a]
        if j is None: continue
        backs = [b for b in range(6) if delta[j][b]==i]
        for b in backs:
            adj[u].append(pos[(j,b)])

    # Hopcroft–Karp
    from collections import deque as dq
    N=len(ports); pairU=[-1]*N; pairV=[-1]*N; dist=[0]*N; INF=10**9
    def bfs():
        q=dq(); found=False
        for u in range(N):
            if pairU[u]==-1: dist[u]=0; q.append(u)
            else: dist[u]=INF
        while q:
            u=q.popleft()
            for v in adj[u]:
                pu=pairV[v]
                if pu==-1: found=True
                elif dist[pu]==INF:
                    dist[pu]=dist[u]+1; q.append(pu)
        return found
    def dfs(u):
        for v in adj[u]:
            pu=pairV[v]
            if pu==-1 or (dist[pu]==dist[u]+1 and dfs(pu)):
                pairU[u]=v; pairV[v]=u; return True
        dist[u]=INF; return False
    matching=0
    while bfs():
        for u in range(N):
            if pairU[u]==-1 and dfs(u): matching+=1

    # map
    rooms=[label(r) for r in reps]
    start_idx = idx.get(row(""), 0)
    edges=set(); connections=[]
    for u in range(N):
        v=pairU[u]
        if v==-1: continue
        i,a=ports[u]; j,b=ports[v]
        if delta[i][a]!=j or delta[j][b]!=i: continue
        key=tuple(sorted([(i,a),(j,b)]))
        if key not in edges:
            edges.add(key)
            connections.append({"from":{"room":i,"door":a}, "to":{"room":j,"door":b}})

    print(f"[rebuild] states={n}  connections={len(connections)}  matching={matching}/{N}")
    if stop_on_split and (n>6 or len(connections)==18):
        print("Stopping: looks promising to submit.")
        return True, {"rooms":rooms,"startingRoom":start_idx,"connections":connections}
    return False, {"rooms":rooms,"startingRoom":start_idx,"connections":connections}

# ---------- probe schedule (adaptive) ----------
# Base reps: prefer the shortest ones; add more only if needed
base_reps = ["", "1", "4"]

def generate_stream():
    # Phase A: mark–door–mark–door
    for R in base_reps:
        for m1 in MARKS:
            for d1 in DOORS:
                for m2 in MARKS:
                    for d2 in DOORS:
                        yield R + m1 + d1 + m2 + d2
    # Phase B: door–mark–door–mark
    for R in base_reps:
        for d1 in DOORS:
            for m1 in MARKS:
                for d2 in DOORS:
                    for m2 in MARKS:
                        yield R + d1 + m1 + d2 + m2
    # Phase C: mark–door–mark–door–mark (but keep short)
    for R in base_reps:
        for m1 in MARKS:
            for d1 in DOORS:
                for m2 in MARKS:
                    for d2 in DOORS:
                        for m3 in MARKS:
                            yield R + m1 + d1 + m2 + d2 + m3

accepted=0; rejected=0; skipped=0
milestone=0
for plan in generate_stream():
    if plan in results:
        skipped+=1
        continue
    if not TOKEN_RE.match(plan) or len(plan)>24:
        skipped+=1
        continue
    res = safe_explore_one(plan, max_len=24)
    if res=="accepted":
        accepted+=1
        milestone+=1
    elif res in ("invalid","too_long","rejected"):
        rejected+=1
    # periodic rebuild
    if milestone>=100:
        milestone=0
        stop, current_map = rebuild_and_print(stop_on_split=True)
        # save progress every checkpoint
        with open(SNAP,"w") as f: json.dump(results,f)
        print(f"[checkpoint] saved snapshot: {len(results)} plans | accepted={accepted}, rejected={rejected}, skipped={skipped}")
        if stop:
            final_map = current_map
            break
else:
    # finished stream
    _, final_map = rebuild_and_print(stop_on_split=False)

print("\n=== MAP (aleph, current) ===")
print(json.dumps(final_map, indent=2))
print(f"Totals: accepted={accepted}, rejected={rejected}, skipped(cached/filtered)={skipped}")
with open(SNAP,"w") as f: json.dump(results,f)
print(f"Saved: {SNAP}  plans: {len(results)}")


Loaded snapshot: 16080
